# EXTRACT BIRD DATA

https://sabirds.app

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import json
import os
import pandas as pd

c:\ProgramData\miniconda3\envs\py\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
from bs4 import BeautifulSoup
import json

def extract_bird_details(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Comprehensive map of all colors from your filter list
    color_map = {
        "#333": "Black",
        "#f5f5f5": "White",
        "#888": "Grey",
        "#8b4513": "Brown",
        "#dc3545": "Red",
        "#fd7e14": "Orange",
        "#ffc107": "Yellow",
        "#28a745": "Green",
        "#007bff": "Blue",
        "#6f42c1": "Purple",
        "#e83e8c": "Pink",
        "#f5deb3": "Buff",
        "#fffdd0": "Cream",
        "#954535": "Chestnut",
        "#808000": "Olive",
        "#a52a2a": "Rufous",
        "linear-gradient(135deg, #ff006e, #8338ec, #3a86ff)": "Iridescent"
    }

    birds = []
    # Identify each card container
    cards = soup.find_all('div', class_='card')

    for card in cards:
        # 1. Extract Images & ID
        img_container = card.find('div', class_='card-image')
        img_tag = img_container.find('img') if img_container else None
        
        # 2. Extract and Map Colors
        color_dots = card.find_all('div', class_='card-color-dot')
        mapped_colors = []
        for dot in color_dots:
            style = dot.get('style', '')
            # Clean the style string to get the hex or gradient
            color_value = style.replace('background:', '').strip().lower()
            mapped_colors.append(color_map.get(color_value, "Unknown"))

        # 3. Extract Tags (Family, Size, Habitat)
        meta_tags = [t.get_text() for t in card.find_all('span', class_='meta-tag')]
        
        # 4. Compile Data Structure
        bird_data = {
            # "species_id": img_container.get('data-species') if img_container else None,
            "common_name": card.find('h3').get_text() if card.find('h3') else None,
            "scientific_name": card.find('p', class_='scientific').get_text() if card.find('p', class_='scientific') else None,
            "afrikaans_name": card.find('p', class_='afrikaans-name').get_text() if card.find('p', class_='afrikaans-name') else None,
            "image":  img_tag.get('src') if img_tag else None,
            "family": meta_tags[0] if len(meta_tags) > 0 else None,
            "size": meta_tags[1] if len(meta_tags) > 1 else None,
            "habitat": meta_tags[2] if len(meta_tags) > 2 else None,
            "colors": mapped_colors
        }
        birds.append(bird_data)
        
    return birds

In [3]:
html_content = \
'''
<div class="grid" id="birdGrid">
    <div class="card ">
                    <div class="card-image" data-species="Ciconia abdimii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/abdims-stork/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ciconia abdimii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Abdim's Stork</h3>
                        <p class="afrikaans-name">Kleinswartooievaar</p>
                        <p class="scientific">Ciconia abdimii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Storks</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div>
                </div>
                <div class="card ">
                    <div class="card-image" data-species="Tricholaema leucomelas"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/acacia-pied-barbet/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Tricholaema leucomelas')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Acacia Pied Barbet</h3>
                        <p class="afrikaans-name">Bonthoutkapper</p>
                        <p class="scientific">Tricholaema leucomelas</p>
                        <div class="card-meta">
                            <span class="meta-tag">Barbets</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Glaucidium capense"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-barred-owlet/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Glaucidium capense')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Barred Owlet</h3>
                        <p class="afrikaans-name">Gebande Uil</p>
                        <p class="scientific">Glaucidium capense</p>
                        <div class="card-meta">
                            <span class="meta-tag">Owls</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anas sparsa"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-black-duck/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anas sparsa')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Black Duck</h3>
                        <p class="afrikaans-name">Swarteend</p>
                        <p class="scientific">Anas sparsa</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ducks</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Apus barbatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-black-swift/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Apus barbatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Black Swift</h3>
                        <p class="afrikaans-name">Swartwindswael</p>
                        <p class="scientific">Apus barbatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Swifts</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Rocky</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Smithornis capensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-broadbill/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Smithornis capensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #e83e8c"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Broadbill</h3>
                        <p class="afrikaans-name">Breëbek</p>
                        <p class="scientific">Smithornis capensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Broadbills</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Crecopsis egregia"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-crake/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Crecopsis egregia')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Crake</h3>
                        <p class="afrikaans-name">Afrikaanse Riethaan</p>
                        <p class="scientific">Crecopsis egregia</p>
                        <div class="card-meta">
                            <span class="meta-tag">Crakes</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cuculus gularis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-cuckoo/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cuculus gularis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Cuckoo</h3>
                        <p class="afrikaans-name">Afrikaanse Koekoek</p>
                        <p class="scientific">Cuculus gularis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cuckoos</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Aviceda cuculoides"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-cuckoo-hawk/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Aviceda cuculoides')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Cuckoo Hawk</h3>
                        <p class="afrikaans-name">Koekoekvalk</p>
                        <p class="scientific">Aviceda cuculoides</p>
                        <div class="card-meta">
                            <span class="meta-tag">Hawks</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anhinga rufa"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-darter/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anhinga rufa')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Darter</h3>
                        <p class="afrikaans-name">Slanghalsvoël</p>
                        <p class="scientific">Anhinga rufa</p>
                        <div class="card-meta">
                            <span class="meta-tag">Darters</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Muscicapa adusta"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-dusky-flycatcher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Muscicapa adusta')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Dusky Flycatcher</h3>
                        <p class="afrikaans-name">Donkervlieëvanger</p>
                        <p class="scientific">Muscicapa adusta</p>
                        <div class="card-meta">
                            <span class="meta-tag">Flycatchers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Chrysococcyx cupreus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-emerald-cuckoo/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Chrysococcyx cupreus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Emerald Cuckoo</h3>
                        <p class="afrikaans-name">Mooimeisie</p>
                        <p class="scientific">Chrysococcyx cupreus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cuckoos</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Podica senegalensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-finfoot/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Podica senegalensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Finfoot</h3>
                        <p class="afrikaans-name">Watertrapper</p>
                        <p class="scientific">Podica senegalensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Finfoots</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Lagonosticta rubricata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-firefinch/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Lagonosticta rubricata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #e83e8c"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Firefinch</h3>
                        <p class="afrikaans-name">Kaapse Vuurvinkie</p>
                        <p class="scientific">Lagonosticta rubricata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Firefinches</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Haliaeetus vocifer"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-fish-eagle/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Haliaeetus vocifer')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Fish Eagle</h3>
                        <p class="afrikaans-name">Visarend</p>
                        <p class="scientific">Haliaeetus vocifer</p>
                        <div class="card-meta">
                            <span class="meta-tag">Eagles</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Oriolus auratus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-golden-oriole/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Oriolus auratus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Golden Oriole</h3>
                        <p class="afrikaans-name">Afrikaanse Wielewaal</p>
                        <p class="scientific">Oriolus auratus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Orioles</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Accipiter tachiro"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-goshawk/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Accipiter tachiro')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Goshawk</h3>
                        <p class="afrikaans-name">Afrikaanse Sperwer</p>
                        <p class="scientific">Accipiter tachiro</p>
                        <div class="card-meta">
                            <span class="meta-tag">Hawks</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Tyto capensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-grass-owl/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Tyto capensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Grass Owl</h3>
                        <p class="afrikaans-name">Grasuil</p>
                        <p class="scientific">Tyto capensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Owls</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Treron calvus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-green-pigeon/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Treron calvus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #6f42c1"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Green Pigeon</h3>
                        <p class="afrikaans-name">Papegaaiduif</p>
                        <p class="scientific">Treron calvus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pigeons</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Lophoceros nasutus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-grey-hornbill/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Lophoceros nasutus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Grey Hornbill</h3>
                        <p class="afrikaans-name">Grysneushoringvoël</p>
                        <p class="scientific">Lophoceros nasutus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Hornbills</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Polyboroides typus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-harrier-hawk/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Polyboroides typus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Harrier- Hawk</h3>
                        <p class="afrikaans-name">Kaalwangvalk</p>
                        <p class="scientific">Polyboroides typus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Hawks</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Aquila spilogaster"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-hawk-eagle/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Aquila spilogaster')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Hawk Eagle</h3>
                        <p class="afrikaans-name">Grootjagarend</p>
                        <p class="scientific">Aquila spilogaster</p>
                        <div class="card-meta">
                            <span class="meta-tag">Eagles</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Falco cuvierii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-hobby/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Falco cuvierii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Hobby</h3>
                        <p class="afrikaans-name">Afrikaanse Boomvalk</p>
                        <p class="scientific">Falco cuvierii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Falcons</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Upupa africana"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-hoopoe/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Upupa africana')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #954535"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Hoopoe</h3>
                        <p class="afrikaans-name">Hoephoep</p>
                        <p class="scientific">Upupa africana</p>
                        <div class="card-meta">
                            <span class="meta-tag">Hoopoes</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Savanna</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Actophilornis africanus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-jacana/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Actophilornis africanus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #007bff"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Jacana</h3>
                        <p class="afrikaans-name">Grootlangtoon</p>
                        <p class="scientific">Actophilornis africanus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Jacanas</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Circus ranivorus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-marsh-harrier/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Circus ranivorus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Marsh Harrier</h3>
                        <p class="afrikaans-name">Afrikaanse Vleivalk(Paddavreter)</p>
                        <p class="scientific">Circus ranivorus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Harriers</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Streptopelia decipiens"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-mourning-dove/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Streptopelia decipiens')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #e83e8c"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Mourning Dove</h3>
                        <p class="afrikaans-name">Rooioogtortelduif</p>
                        <p class="scientific">Streptopelia decipiens</p>
                        <div class="card-meta">
                            <span class="meta-tag">Doves</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Columba arquatrix"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-olive-pigeon/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Columba arquatrix')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #6f42c1"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Olive Pigeon</h3>
                        <p class="afrikaans-name">Geelbekbosduif</p>
                        <p class="scientific">Columba arquatrix</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pigeons</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anastomus lamelligerus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-openbill/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anastomus lamelligerus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Openbill</h3>
                        <p class="afrikaans-name">Oopbekooievaar</p>
                        <p class="scientific">Anastomus lamelligerus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Storks</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Haematopus moquini"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-oystercatcher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Haematopus moquini')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Oystercatcher</h3>
                        <p class="afrikaans-name">Swarttobie</p>
                        <p class="scientific">Haematopus moquini</p>
                        <div class="card-meta">
                            <span class="meta-tag">Oystercatchers</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cypsiurus parvus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-palm-swift/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cypsiurus parvus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Palm Swift</h3>
                        <p class="afrikaans-name">Palmwindswael</p>
                        <p class="scientific">Cypsiurus parvus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Swifts</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Terpsiphone viridis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-paradise-flycatcher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Terpsiphone viridis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Paradise Flycatcher</h3>
                        <p class="afrikaans-name">Paradysvlieëvanger</p>
                        <p class="scientific">Terpsiphone viridis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Paradise-Flycatchers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Spheniscus demersus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-penguin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Spheniscus demersus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Penguin</h3>
                        <p class="afrikaans-name">Brilpikkewyn</p>
                        <p class="scientific">Spheniscus demersus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Penguins</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Motacilla aguimp"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-pied-wagtail/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Motacilla aguimp')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Pied Wagtail</h3>
                        <p class="afrikaans-name">Bontkwikkie</p>
                        <p class="scientific">Motacilla aguimp</p>
                        <div class="card-meta">
                            <span class="meta-tag">Wagtails</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Rivers</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anthus cinnamomeus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-pipit/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anthus cinnamomeus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Pipit</h3>
                        <p class="afrikaans-name">Gewone Koester</p>
                        <p class="scientific">Anthus cinnamomeus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pipits</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pitta angolensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-pitta/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pitta angolensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Pitta</h3>
                        <p class="afrikaans-name">Angolapitta</p>
                        <p class="scientific">Pitta angolensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pittas</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Nettapus auritus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-pygmy-goose/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Nettapus auritus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Pygmy Goose</h3>
                        <p class="afrikaans-name">Dwerggans</p>
                        <p class="scientific">Nettapus auritus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ducks</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ispidina picta"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-pygmy-kingfisher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ispidina picta')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #6f42c1"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Pygmy Kingfisher</h3>
                        <p class="afrikaans-name">Dwergvisvanger</p>
                        <p class="scientific">Ispidina picta</p>
                        <div class="card-meta">
                            <span class="meta-tag">Kingfishers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Rallus caerulescens"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-rail/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Rallus caerulescens')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Rail</h3>
                        <p class="afrikaans-name">Grootriethaan</p>
                        <p class="scientific">Rallus caerulescens</p>
                        <div class="card-meta">
                            <span class="meta-tag">Rails</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pycnonotus nigricans"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-red-eyed-bulbul/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pycnonotus nigricans')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Red-eyed Bulbul</h3>
                        <p class="afrikaans-name">Rooioogtiptol</p>
                        <p class="scientific">Pycnonotus nigricans</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bulbuls</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anthus crenatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-rock-pipit/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anthus crenatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Rock Pipit</h3>
                        <p class="afrikaans-name">Klipkoester</p>
                        <p class="scientific">Anthus crenatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pipits</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Threskiornis aethiopicus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-sacred-ibis/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Threskiornis aethiopicus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Sacred Ibis</h3>
                        <p class="afrikaans-name">Skoorsteenveër</p>
                        <p class="scientific">Threskiornis aethiopicus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ibises</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Otus senegalensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-scops-owl/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Otus senegalensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Scops Owl</h3>
                        <p class="afrikaans-name">Skopsuil</p>
                        <p class="scientific">Otus senegalensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Owls</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Rynchops flavirostris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-skimmer/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Rynchops flavirostris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Skimmer</h3>
                        <p class="afrikaans-name">Waterploeër</p>
                        <p class="scientific">Rynchops flavirostris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Skimmers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Gallinago nigripennis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-snipe/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Gallinago nigripennis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Snipe</h3>
                        <p class="afrikaans-name">Afrikaanse Snip</p>
                        <p class="scientific">Gallinago nigripennis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Snipes</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Platalea alba"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-spoonbill/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Platalea alba')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #e83e8c"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Spoonbill</h3>
                        <p class="afrikaans-name">Lepelaar</p>
                        <p class="scientific">Platalea alba</p>
                        <div class="card-meta">
                            <span class="meta-tag">Spoonbills</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Salpornis salvadori"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-spotted-creeper/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Salpornis salvadori')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Spotted Creeper</h3>
                        <p class="afrikaans-name">Afrikaanse Boomkruiper</p>
                        <p class="scientific">Salpornis salvadori</p>
                        <div class="card-meta">
                            <span class="meta-tag">Creepers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Saxicola torquatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-stonechat/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Saxicola torquatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Stonechat</h3>
                        <p class="afrikaans-name">Gewone Bontrokkie</p>
                        <p class="scientific">Saxicola torquatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Chats</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Porphyrio madagascariensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-swamphen/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Porphyrio madagascariensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Swamphen</h3>
                        <p class="afrikaans-name">Grootkoningriethaan</p>
                        <p class="scientific">Porphyrio madagascariensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Swamphens</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Vanellus senegallus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-wattled-lapwing/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Vanellus senegallus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Wattled Lapwing</h3>
                        <p class="afrikaans-name">Lelkiewiet</p>
                        <p class="scientific">Vanellus senegallus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Lapwings</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Strix woodfordii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-wood-owl/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Strix woodfordii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Wood Owl</h3>
                        <p class="afrikaans-name">Bosuil</p>
                        <p class="scientific">Strix woodfordii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Owls</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ciconia microscelis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/african-woolly-necked-stork/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ciconia microscelis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>African Woolly-necked Stork</h3>
                        <p class="afrikaans-name">Wolnekooievaar</p>
                        <p class="scientific">Ciconia microscelis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Storks</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Certhilauda brevirostris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/agulhas-long-billed-lark/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Certhilauda brevirostris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Agulhas Long-billed Lark</h3>
                        <p class="afrikaans-name">Overberglangbeklewerik</p>
                        <p class="scientific">Certhilauda brevirostris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Porphyrio alleni"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/allens-gallinule/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Porphyrio alleni')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #6f42c1"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Allen's Gallinule</h3>
                        <p class="afrikaans-name">Kleinkoningriethaan</p>
                        <p class="scientific">Porphyrio alleni</p>
                        <div class="card-meta">
                            <span class="meta-tag">Swamphens</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Tachymarptis melba"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/alpine-swift/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Tachymarptis melba')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Alpine Swift</h3>
                        <p class="afrikaans-name">Witpenswindswael</p>
                        <p class="scientific">Tachymarptis melba</p>
                        <div class="card-meta">
                            <span class="meta-tag">Swifts</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Rocky</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pluvialis dominica"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/american-golden-plover/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pluvialis dominica')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>American Golden Plover</h3>
                        <p class="afrikaans-name">Amerikaanse Goue Strandkiewiet</p>
                        <p class="scientific">Pluvialis dominica</p>
                        <div class="card-meta">
                            <span class="meta-tag">Plovers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Porphyrio martinica"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/american-purple-gallinule/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Porphyrio martinica')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #6f42c1"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>American Purple Gallinule</h3>
                        <p class="afrikaans-name">Amerikaanse Koningriethaan</p>
                        <p class="scientific">Porphyrio martinica</p>
                        <div class="card-meta">
                            <span class="meta-tag">Swamphens</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Chalcomitra amethystina"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/amethyst-sunbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Chalcomitra amethystina')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #6f42c1"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Amethyst Sunbird</h3>
                        <p class="afrikaans-name">Swartsuikerbekkie</p>
                        <p class="scientific">Chalcomitra amethystina</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sunbirds</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Diomedea amsterdamensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/amsterdam-albatross/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Diomedea amsterdamensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Amsterdam Albatross</h3>
                        <p class="afrikaans-name">Amsterdamalbatros</p>
                        <p class="scientific">Diomedea amsterdamensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Albatrosses</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Falco amurensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/amur-falcon/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Falco amurensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Amur Falcon</h3>
                        <p class="afrikaans-name">Oostelike Rooipootvalk</p>
                        <p class="scientific">Falco amurensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Falcons</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Myrmecocichla formicivora"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/ant-eating-chat/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Myrmecocichla formicivora')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #e83e8c"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Ant-eating Chat</h3>
                        <p class="afrikaans-name">Swartpiek</p>
                        <p class="scientific">Myrmecocichla formicivora</p>
                        <div class="card-meta">
                            <span class="meta-tag">Chats</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Thalassoica antarctica"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/antarctic-petrel/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Thalassoica antarctica')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Antarctic Petrel</h3>
                        <p class="afrikaans-name">Antarktiese Stormvoël</p>
                        <p class="scientific">Thalassoica antarctica</p>
                        <div class="card-meta">
                            <span class="meta-tag">Petrels</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pachyptila desolata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/antarctic-prion/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pachyptila desolata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Antarctic Prion</h3>
                        <p class="afrikaans-name">Antarktiese Walvisvoël</p>
                        <p class="scientific">Pachyptila desolata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Prions</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sterna vittata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/antarctic-tern/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sterna vittata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Antarctic Tern</h3>
                        <p class="afrikaans-name">Grysborssterretjie</p>
                        <p class="scientific">Sterna vittata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Terns</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sterna paradisaea"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/arctic-tern/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sterna paradisaea')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Arctic Tern</h3>
                        <p class="afrikaans-name">Arktiese Sterretjie</p>
                        <p class="scientific">Sterna paradisaea</p>
                        <div class="card-meta">
                            <span class="meta-tag">Terns</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Myrmecocichla arnotti"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/arnots-chat/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Myrmecocichla arnotti')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Arnot's Chat</h3>
                        <p class="afrikaans-name">Bontpiek</p>
                        <p class="scientific">Myrmecocichla arnotti</p>
                        <div class="card-meta">
                            <span class="meta-tag">Chats</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Turdoides jardineii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/arrow-marked-babbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Turdoides jardineii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Arrow-marked Babbler</h3>
                        <p class="afrikaans-name">Pylvlekkatlagter</p>
                        <p class="scientific">Turdoides jardineii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Babblers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Muscicapa caerulescens"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/ashy-flycatcher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Muscicapa caerulescens')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Ashy Flycatcher</h3>
                        <p class="afrikaans-name">Blougrysvlieëvanger</p>
                        <p class="scientific">Muscicapa caerulescens</p>
                        <div class="card-meta">
                            <span class="meta-tag">Flycatchers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Savanna</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Melaniparus cinerascens"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/ashy-tit/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Melaniparus cinerascens')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Ashy Tit</h3>
                        <p class="afrikaans-name">Akasiagrysmees</p>
                        <p class="scientific">Melaniparus cinerascens</p>
                        <div class="card-meta">
                            <span class="meta-tag">Tits</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Limnodromus semipalmatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/asian-dowitcher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Limnodromus semipalmatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Asian Dowitcher</h3>
                        <p class="afrikaans-name">Asiatiese Snip</p>
                        <p class="scientific">Limnodromus semipalmatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pterodroma incerta"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/atlantic-petrel/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pterodroma incerta')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Atlantic Petrel</h3>
                        <p class="afrikaans-name">Atlantiese Stormvoël</p>
                        <p class="scientific">Pterodroma incerta</p>
                        <div class="card-meta">
                            <span class="meta-tag">Petrels</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Thalassarche chlororhynchos"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/atlantic-yellow-nosed-albatross/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Thalassarche chlororhynchos')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Atlantic Yellow-nosed Albatross</h3>
                        <p class="afrikaans-name">Atlantiese Geelneusalbatros (- malmok)</p>
                        <p class="scientific">Thalassarche chlororhynchos</p>
                        <div class="card-meta">
                            <span class="meta-tag">Albatrosses</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Morus serrator"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/australasian-gannet/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Morus serrator')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Australasian Gannet</h3>
                        <p class="afrikaans-name">Australiese Malgas</p>
                        <p class="scientific">Morus serrator</p>
                        <div class="card-meta">
                            <span class="meta-tag">Gannets</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Hieraaetus ayresii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/ayress-hawk-eagle/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Hieraaetus ayresii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Ayres's Hawk Eagle</h3>
                        <p class="afrikaans-name">Kleinjagarend</p>
                        <p class="scientific">Hieraaetus ayresii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Eagles</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Zapornia pusilla"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/baillons-crake/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Zapornia pusilla')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Baillon's Crake</h3>
                        <p class="afrikaans-name">Kleinriethaan</p>
                        <p class="scientific">Zapornia pusilla</p>
                        <div class="card-meta">
                            <span class="meta-tag">Crakes</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calidris bairdii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/bairds-sandpiper/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calidris bairdii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Baird's Sandpiper</h3>
                        <p class="afrikaans-name">Bairdse Strandloper</p>
                        <p class="scientific">Calidris bairdii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Neophedina cincta"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/banded-martin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Neophedina cincta')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Banded Martin</h3>
                        <p class="afrikaans-name">Gebande Oewerswael</p>
                        <p class="scientific">Neophedina cincta</p>
                        <div class="card-meta">
                            <span class="meta-tag">Martins</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Phalacrocorax neglectus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/bank-cormorant/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Phalacrocorax neglectus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Bank Cormorant</h3>
                        <p class="afrikaans-name">Bankduiker</p>
                        <p class="scientific">Phalacrocorax neglectus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cormorants</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Limosa lapponica"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/bar-tailed-godwit/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Limosa lapponica')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Bar-tailed Godwit</h3>
                        <p class="afrikaans-name">Bandstertgriet</p>
                        <p class="scientific">Limosa lapponica</p>
                        <div class="card-meta">
                            <span class="meta-tag">Godwits</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Apalis thoracica"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/bar-throated-apalis/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Apalis thoracica')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Bar-throated Apalis</h3>
                        <p class="afrikaans-name">Bandkeelkleinjantjie</p>
                        <p class="scientific">Apalis thoracica</p>
                        <div class="card-meta">
                            <span class="meta-tag">Apalises</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pterodroma baraui"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/baraus-petrel/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pterodroma baraui')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Barau's Petrel</h3>
                        <p class="afrikaans-name">Baraustormvoël</p>
                        <p class="scientific">Pterodroma baraui</p>
                        <div class="card-meta">
                            <span class="meta-tag">Petrels</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calendulauda barlowi"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/barlows-lark/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calendulauda barlowi')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #a52a2a"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Barlow's Lark</h3>
                        <p class="afrikaans-name">Barlowse Lewerik</p>
                        <p class="scientific">Calendulauda barlowi</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Shrubland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Hirundo rustica"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/barn-swallow/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Hirundo rustica')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Barn Swallow</h3>
                        <p class="afrikaans-name">Europese Swael</p>
                        <p class="scientific">Hirundo rustica</p>
                        <div class="card-meta">
                            <span class="meta-tag">Swallows</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Bradypterus barratti"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/barratts-warbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Bradypterus barratti')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Barratt's Warbler</h3>
                        <p class="afrikaans-name">Ruigtesanger</p>
                        <p class="scientific">Bradypterus barratti</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bush-Warblers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cercococcyx montanus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/barred-long-tailed-cuckoo/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cercococcyx montanus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Barred Long-tailed Cuckoo</h3>
                        <p class="afrikaans-name">Langstertkoekoek</p>
                        <p class="scientific">Cercococcyx montanus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cuckoos</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calamonastes fasciolatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/barred-wren-warbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calamonastes fasciolatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Barred Wren- Warbler</h3>
                        <p class="afrikaans-name">Gebande Sanger</p>
                        <p class="scientific">Calamonastes fasciolatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Warblers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Acrocephalus griseldis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/basra-reed-warbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Acrocephalus griseldis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Basra Reed Warbler</h3>
                        <p class="afrikaans-name">Basrarietsanger</p>
                        <p class="scientific">Acrocephalus griseldis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Reed-Warblers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Macheiramphus alcinus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/bat-hawk/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Macheiramphus alcinus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Bat Hawk</h3>
                        <p class="afrikaans-name">Vlermuisvalk</p>
                        <p class="scientific">Macheiramphus alcinus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Hawks</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Terathopius ecaudatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/bateleur/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Terathopius ecaudatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Bateleur</h3>
                        <p class="afrikaans-name">Berghaan</p>
                        <p class="scientific">Terathopius ecaudatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Snake-Eagles</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cercotrichas quadrivirgata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/bearded-scrub-robin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cercotrichas quadrivirgata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Bearded Scrub Robin</h3>
                        <p class="afrikaans-name">Baardwipstert</p>
                        <p class="scientific">Cercotrichas quadrivirgata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Scrub-Robins</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Savanna</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Gypaetus barbatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/bearded-vulture/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Gypaetus barbatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Bearded Vulture</h3>
                        <p class="afrikaans-name">Baardaasvoël</p>
                        <p class="scientific">Gypaetus barbatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Vultures</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Rocky</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Chloropicus namaquus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/bearded-woodpecker/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Chloropicus namaquus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Bearded Woodpecker</h3>
                        <p class="afrikaans-name">Baardspeg</p>
                        <p class="scientific">Chloropicus namaquus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Woodpeckers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Campethera bennettii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/bennetts-woodpecker/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Campethera bennettii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Bennett's Woodpecker</h3>
                        <p class="afrikaans-name">Bennettse Speg</p>
                        <p class="scientific">Campethera bennettii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Woodpeckers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Centropus grillii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-coucal/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Centropus grillii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black Coucal</h3>
                        <p class="afrikaans-name">Swartvleiloerie</p>
                        <p class="scientific">Centropus grillii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Coucals</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Zapornia flavirostra"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-crake/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Zapornia flavirostra')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black Crake</h3>
                        <p class="afrikaans-name">Swartriethaan</p>
                        <p class="scientific">Zapornia flavirostra</p>
                        <div class="card-meta">
                            <span class="meta-tag">Crakes</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cuculus clamosus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-cuckoo/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cuculus clamosus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black Cuckoo</h3>
                        <p class="afrikaans-name">Swartkoekoek</p>
                        <p class="scientific">Cuculus clamosus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cuckoos</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Campephaga flava"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-cuckooshrike/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Campephaga flava')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black Cuckooshrike</h3>
                        <p class="afrikaans-name">Swartkatakoeroe</p>
                        <p class="scientific">Campephaga flava</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cuckooshrikes</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Circus maurus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-harrier/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Circus maurus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black Harrier</h3>
                        <p class="afrikaans-name">Witkruisvleivalk (-paddavreter)</p>
                        <p class="scientific">Circus maurus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Harriers</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Egretta ardesiaca"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-heron/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Egretta ardesiaca')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black Heron</h3>
                        <p class="afrikaans-name">Swartreier</p>
                        <p class="scientific">Egretta ardesiaca</p>
                        <div class="card-meta">
                            <span class="meta-tag">Herons</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Milvus migrans"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-kite/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Milvus migrans')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black Kite</h3>
                        <p class="afrikaans-name">Geelbekwou</p>
                        <p class="scientific">Milvus migrans</p>
                        <div class="card-meta">
                            <span class="meta-tag">Kites</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Psalidoprocne pristoptera"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-saw-wing/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Psalidoprocne pristoptera')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black Saw-wing</h3>
                        <p class="afrikaans-name">Swartsaagvlerkswael</p>
                        <p class="scientific">Psalidoprocne pristoptera</p>
                        <div class="card-meta">
                            <span class="meta-tag">Saw-wings</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Rynchops niger"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-skimmer/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Rynchops niger')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black Skimmer</h3>
                        <p class="afrikaans-name">Amerikaanse Waterploeër</p>
                        <p class="scientific">Rynchops niger</p>
                        <div class="card-meta">
                            <span class="meta-tag">Skimmers</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Rivers</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Accipiter melanoleucus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-sparrowhawk/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Accipiter melanoleucus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black Sparrowhawk</h3>
                        <p class="afrikaans-name">Swartsperwer</p>
                        <p class="scientific">Accipiter melanoleucus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Hawks</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ciconia nigra"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-stork/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ciconia nigra')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black Stork</h3>
                        <p class="afrikaans-name">Grootswartooievaar</p>
                        <p class="scientific">Ciconia nigra</p>
                        <div class="card-meta">
                            <span class="meta-tag">Storks</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Chlidonias niger"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-tern/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Chlidonias niger')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black Tern</h3>
                        <p class="afrikaans-name">Swartsterretjie</p>
                        <p class="scientific">Chlidonias niger</p>
                        <div class="card-meta">
                            <span class="meta-tag">Terns</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Bias musicus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-and-white-flycatcher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Bias musicus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-and-white Flycatcher</h3>
                        <p class="afrikaans-name">Witpensvlieëvanger</p>
                        <p class="scientific">Bias musicus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Flycatchers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Dryoscopus cubla"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-backed-puffback/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Dryoscopus cubla')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-backed Puffback</h3>
                        <p class="afrikaans-name">Sneeubal</p>
                        <p class="scientific">Dryoscopus cubla</p>
                        <div class="card-meta">
                            <span class="meta-tag">Puffbacks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Lissotis melanogaster"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-bellied-bustard/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Lissotis melanogaster')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-bellied Bustard</h3>
                        <p class="afrikaans-name">Langbeenkorhaan</p>
                        <p class="scientific">Lissotis melanogaster</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bustards</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Notopholia corusca"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-bellied-starling/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Notopholia corusca')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-bellied Starling</h3>
                        <p class="afrikaans-name">Swartpensglansspreeu</p>
                        <p class="scientific">Notopholia corusca</p>
                        <div class="card-meta">
                            <span class="meta-tag">Starlings</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Fregetta tropica"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-bellied-storm-petrel/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Fregetta tropica')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-bellied Storm Petrel</h3>
                        <p class="afrikaans-name">Swartpensstormswael</p>
                        <p class="scientific">Fregetta tropica</p>
                        <div class="card-meta">
                            <span class="meta-tag">Storm-Petrels</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Thalassarche melanophris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-browed-albatross/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Thalassarche melanophris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-browed Albatross</h3>
                        <p class="afrikaans-name">Swartrugalbatros (-malmok)</p>
                        <p class="scientific">Thalassarche melanophris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Albatrosses</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Prinia flavicans"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-chested-prinia/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Prinia flavicans')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-chested Prinia</h3>
                        <p class="afrikaans-name">Swartbandlangstertjie</p>
                        <p class="scientific">Prinia flavicans</p>
                        <div class="card-meta">
                            <span class="meta-tag">Prinias</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Circaetus pectoralis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-chested-snake-eagle/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Circaetus pectoralis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-chested Snake Eagle</h3>
                        <p class="afrikaans-name">Swartborsslangarend</p>
                        <p class="scientific">Circaetus pectoralis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Snake-Eagles</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Lybius torquatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-collared-barbet/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Lybius torquatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-collared Barbet</h3>
                        <p class="afrikaans-name">Rooikophoutkapper</p>
                        <p class="scientific">Lybius torquatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Barbets</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Nycticorax nycticorax"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-crowned-night-heron/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Nycticorax nycticorax')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-crowned Night Heron</h3>
                        <p class="afrikaans-name">Gewone Nagreier</p>
                        <p class="scientific">Nycticorax nycticorax</p>
                        <div class="card-meta">
                            <span class="meta-tag">Herons</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Tchagra senegalus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-crowned-tchagra/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Tchagra senegalus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-crowned Tchagra</h3>
                        <p class="afrikaans-name">Swartkroontjagra</p>
                        <p class="scientific">Tchagra senegalus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Tchagras</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Eremopterix australis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-eared-sparrow-lark/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Eremopterix australis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-eared Sparrow- Lark</h3>
                        <p class="afrikaans-name">Swartoorlewerik</p>
                        <p class="scientific">Eremopterix australis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Brunhilda erythronotos"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-faced-waxbill/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Brunhilda erythronotos')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #e83e8c"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-faced Waxbill</h3>
                        <p class="afrikaans-name">Swartwangsysie</p>
                        <p class="scientific">Brunhilda erythronotos</p>
                        <div class="card-meta">
                            <span class="meta-tag">Waxbills</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Chlorophoneus nigrifrons"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-fronted-bushshrike/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Chlorophoneus nigrifrons')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-fronted Bushshrike</h3>
                        <p class="afrikaans-name">Swartoogboslaksman</p>
                        <p class="scientific">Chlorophoneus nigrifrons</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bush-shrikes</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Serinus alario"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-headed-canary/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Serinus alario')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-headed Canary</h3>
                        <p class="afrikaans-name">Swartkopkanarie</p>
                        <p class="scientific">Serinus alario</p>
                        <div class="card-meta">
                            <span class="meta-tag">Canaries</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Chroicocephalus ridibundus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-headed-gull/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Chroicocephalus ridibundus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-headed Gull</h3>
                        <p class="afrikaans-name">Swartkopmeeu</p>
                        <p class="scientific">Chroicocephalus ridibundus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Gulls</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ardea melanocephala"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-headed-heron/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ardea melanocephala')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-headed Heron</h3>
                        <p class="afrikaans-name">Swartkopreier</p>
                        <p class="scientific">Ardea melanocephala</p>
                        <div class="card-meta">
                            <span class="meta-tag">Herons</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Oriolus larvatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-headed-oriole/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Oriolus larvatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-headed Oriole</h3>
                        <p class="afrikaans-name">Swartkopwielewaal</p>
                        <p class="scientific">Oriolus larvatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Orioles</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Rissa tridactyla"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-legged-kittiwake/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Rissa tridactyla')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-legged Kittiwake</h3>
                        <p class="afrikaans-name">Swartpootbrandervoël</p>
                        <p class="scientific">Rissa tridactyla</p>
                        <div class="card-meta">
                            <span class="meta-tag">Gulls &amp; Terns</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sterna sumatrana"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-naped-tern/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sterna sumatrana')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-naped Tern</h3>
                        <p class="afrikaans-name">Swartneksterretjie</p>
                        <p class="scientific">Sterna sumatrana</p>
                        <div class="card-meta">
                            <span class="meta-tag">Terns</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Podiceps nigricollis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-necked-grebe/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Podiceps nigricollis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-necked Grebe</h3>
                        <p class="afrikaans-name">Swartnekdobbertjie</p>
                        <p class="scientific">Podiceps nigricollis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Grebes</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Turnix nanus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-rumped-buttonquail/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Turnix nanus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-rumped Buttonquail</h3>
                        <p class="afrikaans-name">Swartrugkwarteltjie</p>
                        <p class="scientific">Turnix nanus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Buttonquails</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Limosa limosa"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-tailed-godwit/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Limosa limosa')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-tailed Godwit</h3>
                        <p class="afrikaans-name">Swartstertgriet</p>
                        <p class="scientific">Limosa limosa</p>
                        <div class="card-meta">
                            <span class="meta-tag">Godwits</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Crithagra atrogularis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-throated-canary/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Crithagra atrogularis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-throated Canary</h3>
                        <p class="afrikaans-name">Bergkanarie</p>
                        <p class="scientific">Crithagra atrogularis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Canaries</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Platysteira peltata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-throated-wattle-eye/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Platysteira peltata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-throated Wattle-eye</h3>
                        <p class="afrikaans-name">Beloogbosbontrokkie</p>
                        <p class="scientific">Platysteira peltata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Wattle-eyes</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Elanus caeruleus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-winged-kite/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Elanus caeruleus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-winged Kite</h3>
                        <p class="afrikaans-name">Blouvalk</p>
                        <p class="scientific">Elanus caeruleus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Kites</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Vanellus melanopterus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-winged-lapwing/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Vanellus melanopterus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-winged Lapwing</h3>
                        <p class="afrikaans-name">Grootswartvlerkkiewiet</p>
                        <p class="scientific">Vanellus melanopterus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Lapwings</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Glareola nordmanni"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-winged-pratincole/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Glareola nordmanni')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-winged Pratincole</h3>
                        <p class="afrikaans-name">Swartvlerksprinkaanvoël</p>
                        <p class="scientific">Glareola nordmanni</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pratincoles</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Himantopus himantopus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/black-winged-stilt/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Himantopus himantopus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #e83e8c"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Black-winged Stilt</h3>
                        <p class="afrikaans-name">Rooipootelsie</p>
                        <p class="scientific">Himantopus himantopus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Stilts</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Vanellus armatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/blacksmith-lapwing/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Vanellus armatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Blacksmith Lapwing</h3>
                        <p class="afrikaans-name">Bontkiewiet</p>
                        <p class="scientific">Vanellus armatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Lapwings</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Grus paradisea"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/blue-crane/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Grus paradisea')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Blue Crane</h3>
                        <p class="afrikaans-name">Bloukraanvoël</p>
                        <p class="scientific">Grus paradisea</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cranes</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Eupodotis caerulescens"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/blue-korhaan/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Eupodotis caerulescens')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Blue Korhaan</h3>
                        <p class="afrikaans-name">Bloukorhaan</p>
                        <p class="scientific">Eupodotis caerulescens</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bustards</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Halobaena caerulea"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/blue-petrel/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Halobaena caerulea')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Blue Petrel</h3>
                        <p class="afrikaans-name">Bloustormvoël</p>
                        <p class="scientific">Halobaena caerulea</p>
                        <div class="card-meta">
                            <span class="meta-tag">Petrels</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Synoicus adansonii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/blue-quail/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Synoicus adansonii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Blue Quail</h3>
                        <p class="afrikaans-name">Bloukwartel</p>
                        <p class="scientific">Synoicus adansonii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Quails</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Hirundo atrocaerulea"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/blue-swallow/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Hirundo atrocaerulea')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Blue Swallow</h3>
                        <p class="afrikaans-name">Blouswael</p>
                        <p class="scientific">Hirundo atrocaerulea</p>
                        <div class="card-meta">
                            <span class="meta-tag">Swallows</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Uraeginthus angolensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/blue-waxbill/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Uraeginthus angolensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #e83e8c"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Blue Waxbill</h3>
                        <p class="afrikaans-name">Gewone Blousysie</p>
                        <p class="scientific">Uraeginthus angolensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Waxbills</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Spatula hottentota"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/blue-billed-teal/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Spatula hottentota')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Blue-billed Teal</h3>
                        <p class="afrikaans-name">Gevlekte Eend</p>
                        <p class="scientific">Spatula hottentota</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ducks</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Merops persicus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/blue-cheeked-bee-eater/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Merops persicus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Blue-cheeked Bee-eater</h3>
                        <p class="afrikaans-name">Blouwangbyvreter</p>
                        <p class="scientific">Merops persicus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bee-eaters</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Karoo</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Trochocercus cyanomelas"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/blue-mantled-crested-flycatcher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Trochocercus cyanomelas')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Blue-mantled Crested Flycatcher</h3>
                        <p class="afrikaans-name">Bloukuifvlieëvanger</p>
                        <p class="scientific">Trochocercus cyanomelas</p>
                        <div class="card-meta">
                            <span class="meta-tag">Flycatchers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Turtur afer"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/blue-spotted-wood-dove/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Turtur afer')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #6f42c1"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Blue-spotted Wood Dove</h3>
                        <p class="afrikaans-name">Blouvlekduifie</p>
                        <p class="scientific">Turtur afer</p>
                        <div class="card-meta">
                            <span class="meta-tag">Doves</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Telophorus zeylonus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/bokmakierie/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Telophorus zeylonus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Bokmakierie</h3>
                        <p class="afrikaans-name">Bokmakierie</p>
                        <p class="scientific">Telophorus zeylonus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bokmakieries</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Hieraaetus pennatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/booted-eagle/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Hieraaetus pennatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Booted Eagle</h3>
                        <p class="afrikaans-name">Dwergarend</p>
                        <p class="scientific">Hieraaetus pennatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Eagles</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Spizocorys fringillaris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/bothas-lark/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Spizocorys fringillaris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">CR</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Botha's Lark</h3>
                        <p class="afrikaans-name">Vaalrivierlewerik</p>
                        <p class="scientific">Spizocorys fringillaris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pinarornis plumosus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/boulder-chat/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pinarornis plumosus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Boulder Chat</h3>
                        <p class="afrikaans-name">Swartberglyster</p>
                        <p class="scientific">Pinarornis plumosus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Chats</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Apus bradfieldi"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/bradfields-swift/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Apus bradfieldi')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Bradfield's Swift</h3>
                        <p class="afrikaans-name">Muiskleurwindswael</p>
                        <p class="scientific">Apus bradfieldi</p>
                        <div class="card-meta">
                            <span class="meta-tag">Swifts</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Karoo</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Onychoprion anaethetus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/bridled-tern/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Onychoprion anaethetus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Bridled Tern</h3>
                        <p class="afrikaans-name">Brilsterretjie</p>
                        <p class="scientific">Onychoprion anaethetus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Terns</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Crithagra sulphurata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/brimstone-canary/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Crithagra sulphurata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Brimstone Canary</h3>
                        <p class="afrikaans-name">Dikbekkanarie</p>
                        <p class="scientific">Crithagra sulphurata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Canaries</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pachyptila vittata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/broad-billed-prion/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pachyptila vittata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Broad-billed Prion</h3>
                        <p class="afrikaans-name">Breëbekwalvisvoël</p>
                        <p class="scientific">Pachyptila vittata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Prions</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Eurystomus glaucurus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/broad-billed-roller/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Eurystomus glaucurus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #6f42c1"></div><div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Broad-billed Roller</h3>
                        <p class="afrikaans-name">Geelbektroupant</p>
                        <p class="scientific">Eurystomus glaucurus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Rollers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calidris falcinellus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/broad-billed-sandpiper/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calidris falcinellus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Broad-billed Sandpiper</h3>
                        <p class="afrikaans-name">Breëbekstrandloper</p>
                        <p class="scientific">Calidris falcinellus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Vidua obtusa"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/broad-tailed-paradise-whydah/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Vidua obtusa')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Broad-tailed Paradise Whydah</h3>
                        <p class="afrikaans-name">Breëstertparadysvink</p>
                        <p class="scientific">Vidua obtusa</p>
                        <div class="card-meta">
                            <span class="meta-tag">Whydahs</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Schoenicola brevirostris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/broad-tailed-warbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Schoenicola brevirostris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #a52a2a"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Broad-tailed Warbler</h3>
                        <p class="afrikaans-name">Breëstertsanger</p>
                        <p class="scientific">Schoenicola brevirostris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Warblers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Spermestes cucullata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/bronze-mannikin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Spermestes cucullata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Bronze Mannikin</h3>
                        <p class="afrikaans-name">Gewone Fret</p>
                        <p class="scientific">Spermestes cucullata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Mannikins</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Rhinoptilus chalcopterus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/bronze-winged-courser/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Rhinoptilus chalcopterus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Bronze-winged Courser</h3>
                        <p class="afrikaans-name">Bronsvlerkdrawwertjie</p>
                        <p class="scientific">Rhinoptilus chalcopterus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Coursers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sula leucogaster"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/brown-booby/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sula leucogaster')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Brown Booby</h3>
                        <p class="afrikaans-name">Bruinmalgas</p>
                        <p class="scientific">Sula leucogaster</p>
                        <div class="card-meta">
                            <span class="meta-tag">Boobies</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anous stolidus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/brown-noddy/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anous stolidus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Brown Noddy</h3>
                        <p class="afrikaans-name">Grootbruinsterretjie</p>
                        <p class="scientific">Anous stolidus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Noddies</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cercotrichas signata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/brown-scrub-robin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cercotrichas signata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Brown Scrub Robin</h3>
                        <p class="afrikaans-name">Bruinwipstert</p>
                        <p class="scientific">Cercotrichas signata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Scrub-Robins</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Savanna</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Circaetus cinereus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/brown-snake-eagle/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Circaetus cinereus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Brown Snake Eagle</h3>
                        <p class="afrikaans-name">Bruinslangarend</p>
                        <p class="scientific">Circaetus cinereus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Snake-Eagles</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Prodotiscus regulus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/brown-backed-honeybird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Prodotiscus regulus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Brown-backed Honeybird</h3>
                        <p class="afrikaans-name">Skerpbekheuningvoël</p>
                        <p class="scientific">Prodotiscus regulus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Honeyguides</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Tchagra australis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/brown-crowned-tchagra/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Tchagra australis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Brown-crowned Tchagra</h3>
                        <p class="afrikaans-name">Rooivlerktjagra</p>
                        <p class="scientific">Tchagra australis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Tchagras</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Poicephalus cryptoxanthus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/brown-headed-parrot/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Poicephalus cryptoxanthus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Brown-headed Parrot</h3>
                        <p class="afrikaans-name">Bruinkoppapegaai</p>
                        <p class="scientific">Poicephalus cryptoxanthus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Parrots</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Halcyon albiventris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/brown-hooded-kingfisher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Halcyon albiventris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Brown-hooded Kingfisher</h3>
                        <p class="afrikaans-name">Bruinkopvisvanger</p>
                        <p class="scientific">Halcyon albiventris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Kingfishers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Riparia paludicola"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/brown-throated-martin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Riparia paludicola')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Brown-throated Martin</h3>
                        <p class="afrikaans-name">Afrikaanse Oewerswael</p>
                        <p class="scientific">Riparia paludicola</p>
                        <div class="card-meta">
                            <span class="meta-tag">Martins</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Nilaus afer"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/brubru/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Nilaus afer')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Brubru</h3>
                        <p class="afrikaans-name">Bontroklaksman</p>
                        <p class="scientific">Nilaus afer</p>
                        <div class="card-meta">
                            <span class="meta-tag">Brubrus</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calidris subruficollis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/buff-breasted-sandpiper/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calidris subruficollis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Buff-breasted Sandpiper</h3>
                        <p class="afrikaans-name">Taanborsstrandloper</p>
                        <p class="scientific">Calidris subruficollis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sarothrura elegans"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/buff-spotted-flufftail/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sarothrura elegans')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Buff-spotted Flufftail</h3>
                        <p class="afrikaans-name">Gevlekte Vleikuiken</p>
                        <p class="scientific">Sarothrura elegans</p>
                        <div class="card-meta">
                            <span class="meta-tag">Flufftails</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Campicoloides bifasciatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/buff-streaked-chat/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Campicoloides bifasciatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Buff-streaked Chat</h3>
                        <p class="afrikaans-name">Bergklipwagter</p>
                        <p class="scientific">Campicoloides bifasciatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Chats</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Rocky</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anthus vaalensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/buffy-pipit/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anthus vaalensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Buffy Pipit</h3>
                        <p class="afrikaans-name">Vaalkoester</p>
                        <p class="scientific">Anthus vaalensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pipits</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Thalassarche bulleri"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/bullers-albatross/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Thalassarche bulleri')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Buller's Albatross</h3>
                        <p class="afrikaans-name">Witkroonalbatros (- malmok)</p>
                        <p class="scientific">Thalassarche bulleri</p>
                        <div class="card-meta">
                            <span class="meta-tag">Albatrosses</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Centropus burchellii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/burchells-coucal/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Centropus burchellii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Burchell's Coucal</h3>
                        <p class="afrikaans-name">Gewone Vleiloerie</p>
                        <p class="scientific">Centropus burchellii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Coucals</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cursorius rufus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/burchells-courser/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cursorius rufus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Burchell's Courser</h3>
                        <p class="afrikaans-name">Bloukopdrawwertjie</p>
                        <p class="scientific">Cursorius rufus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Coursers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pterocles burchelli"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/burchells-sandgrouse/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pterocles burchelli')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Burchell's Sandgrouse</h3>
                        <p class="afrikaans-name">Gevlekte Sandpatrys</p>
                        <p class="scientific">Pterocles burchelli</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandgrouse</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Lamprotornis australis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/burchells-starling/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Lamprotornis australis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Burchell's Starling</h3>
                        <p class="afrikaans-name">Grootglansspreeu</p>
                        <p class="scientific">Lamprotornis australis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Starlings</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Eremomela usticollis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/burnt-necked-eremomela/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Eremomela usticollis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Burnt-necked Eremomela</h3>
                        <p class="afrikaans-name">Bruinkeelbossanger</p>
                        <p class="scientific">Eremomela usticollis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Eremomelas</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sylvia nigricapillus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/bush-blackcap/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sylvia nigricapillus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Bush Blackcap</h3>
                        <p class="afrikaans-name">Rooibektiptol</p>
                        <p class="scientific">Sylvia nigricapillus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Warblers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anthus caffer"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/bushveld-pipit/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anthus caffer')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Bushveld Pipit</h3>
                        <p class="afrikaans-name">Bosveldkoester</p>
                        <p class="scientific">Anthus caffer</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pipits</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Neafrapus boehmi"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/böhms-spinetail/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Neafrapus boehmi')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Böhm's Spinetail</h3>
                        <p class="afrikaans-name">Witpensstekelstert</p>
                        <p class="scientific">Neafrapus boehmi</p>
                        <div class="card-meta">
                            <span class="meta-tag">Spinetails</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Batis capensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-batis/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Batis capensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Batis</h3>
                        <p class="afrikaans-name">Kaapse Bosbontrokkie</p>
                        <p class="scientific">Batis capensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Batises</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pycnonotus capensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-bulbul/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pycnonotus capensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Bulbul</h3>
                        <p class="afrikaans-name">Kaapse Tiptol</p>
                        <p class="scientific">Pycnonotus capensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bulbuls</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Emberiza capensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-bunting/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Emberiza capensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Bunting</h3>
                        <p class="afrikaans-name">Rooivlerkstreepkoppie</p>
                        <p class="scientific">Emberiza capensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Buntings</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Serinus canicollis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-canary/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Serinus canicollis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Canary</h3>
                        <p class="afrikaans-name">Kaapse Kanarie</p>
                        <p class="scientific">Serinus canicollis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Canaries</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Mirafra apiata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-clapper-lark/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Mirafra apiata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Clapper Lark</h3>
                        <p class="afrikaans-name">Kaapse Klappertjie</p>
                        <p class="scientific">Mirafra apiata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Phalacrocorax capensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-cormorant/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Phalacrocorax capensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Cormorant</h3>
                        <p class="afrikaans-name">Trekduiker</p>
                        <p class="scientific">Phalacrocorax capensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cormorants</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Corvus capensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-crow/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Corvus capensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Crow</h3>
                        <p class="afrikaans-name">Swartkraai</p>
                        <p class="scientific">Corvus capensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Crows</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Bubo capensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-eagle-owl/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Bubo capensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Eagle- Owl</h3>
                        <p class="afrikaans-name">Kaapse Ooruil</p>
                        <p class="scientific">Bubo capensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Owls</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Morus capensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-gannet/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Morus capensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Gannet</h3>
                        <p class="afrikaans-name">Witmalgas</p>
                        <p class="scientific">Morus capensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Gannets</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Lamprotornis nitens"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-glossy-starling/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Lamprotornis nitens')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #007bff"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Glossy Starling</h3>
                        <p class="afrikaans-name">Kleinglansspreeu</p>
                        <p class="scientific">Lamprotornis nitens</p>
                        <div class="card-meta">
                            <span class="meta-tag">Starlings</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sphenoeacus afer"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-grassbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sphenoeacus afer')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Grassbird</h3>
                        <p class="afrikaans-name">Grasvoël</p>
                        <p class="scientific">Sphenoeacus afer</p>
                        <div class="card-meta">
                            <span class="meta-tag">Grassbirds</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Certhilauda curvirostris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-long-billed-lark/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Certhilauda curvirostris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Long-billed Lark</h3>
                        <p class="afrikaans-name">Weskuslangbeklewerik</p>
                        <p class="scientific">Certhilauda curvirostris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Macronyx capensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-longclaw/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Macronyx capensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Longclaw</h3>
                        <p class="afrikaans-name">Oranjekeelkalkoentjie</p>
                        <p class="scientific">Macronyx capensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Longclaws</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Poicephalus robustus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-parrot/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Poicephalus robustus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            <span class="badge badge-threatened">VU</span>
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Parrot</h3>
                        <p class="afrikaans-name">Woudpapegaai</p>
                        <p class="scientific">Poicephalus robustus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Parrots</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anthoscopus minutus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-penduline-tit/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anthoscopus minutus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Penduline-tit</h3>
                        <p class="afrikaans-name">Kaapse Kapokvoël</p>
                        <p class="scientific">Anthoscopus minutus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Penduline-Tits</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Savanna</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cossypha caffra"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-robin-chat/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cossypha caffra')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Robin-chat</h3>
                        <p class="afrikaans-name">Gewone Janfrederik</p>
                        <p class="scientific">Cossypha caffra</p>
                        <div class="card-meta">
                            <span class="meta-tag">Robin-Chats</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Savanna</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Monticola rupestris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-rock-thrush/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Monticola rupestris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Rock Thrush</h3>
                        <p class="afrikaans-name">Kaapse Kliplyster</p>
                        <p class="scientific">Monticola rupestris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Thrushes</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Chaetops frenatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-rockjumper/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Chaetops frenatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Rockjumper</h3>
                        <p class="afrikaans-name">Kaapse Berglyster</p>
                        <p class="scientific">Chaetops frenatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Rockjumpers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Rocky</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Spatula smithii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-shoveler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Spatula smithii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Shoveler</h3>
                        <p class="afrikaans-name">Kaapse Slopeend</p>
                        <p class="scientific">Spatula smithii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ducks</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Crithagra totta"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-siskin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Crithagra totta')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Siskin</h3>
                        <p class="afrikaans-name">Kaapse Pietjiekanarie</p>
                        <p class="scientific">Crithagra totta</p>
                        <div class="card-meta">
                            <span class="meta-tag">Canaries</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Passer melanurus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-sparrow/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Passer melanurus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Sparrow</h3>
                        <p class="afrikaans-name">Gewone Mossie</p>
                        <p class="scientific">Passer melanurus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sparrows</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pternistis capensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-spurfowl/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pternistis capensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Spurfowl</h3>
                        <p class="afrikaans-name">Kaapse Fisant</p>
                        <p class="scientific">Pternistis capensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Francolins</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Promerops cafer"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-sugarbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Promerops cafer')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Sugarbird</h3>
                        <p class="afrikaans-name">Kaapse Suikervoël</p>
                        <p class="scientific">Promerops cafer</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sugarbirds</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anas capensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-teal/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anas capensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #e83e8c"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Teal</h3>
                        <p class="afrikaans-name">Teeleend</p>
                        <p class="scientific">Anas capensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ducks</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Streptopelia capicola"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-turtle-dove/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Streptopelia capicola')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Turtle Dove</h3>
                        <p class="afrikaans-name">Gewone Tortelduif</p>
                        <p class="scientific">Streptopelia capicola</p>
                        <div class="card-meta">
                            <span class="meta-tag">Doves</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Gyps coprotheres"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-vulture/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Gyps coprotheres')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Vulture</h3>
                        <p class="afrikaans-name">Kransaasvoël</p>
                        <p class="scientific">Gyps coprotheres</p>
                        <div class="card-meta">
                            <span class="meta-tag">Vultures</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Motacilla capensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-wagtail/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Motacilla capensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Wagtail</h3>
                        <p class="afrikaans-name">Gewone Kwikkie</p>
                        <p class="scientific">Motacilla capensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Wagtails</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ploceus capensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-weaver/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ploceus capensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape Weaver</h3>
                        <p class="afrikaans-name">Kaapse Wewer</p>
                        <p class="scientific">Ploceus capensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Weavers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Zosterops virens"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cape-white-eye/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Zosterops virens')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cape White-eye</h3>
                        <p class="afrikaans-name">Kaapse Glasogie</p>
                        <p class="scientific">Zosterops virens</p>
                        <div class="card-meta">
                            <span class="meta-tag">White-eyes</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Oenanthe pileata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/capped-wheatear/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Oenanthe pileata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Capped Wheatear</h3>
                        <p class="afrikaans-name">Hoëveldskaapwagter</p>
                        <p class="scientific">Oenanthe pileata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Wheatears</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Dendropicos fuscescens"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cardinal-woodpecker/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Dendropicos fuscescens')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cardinal Woodpecker</h3>
                        <p class="afrikaans-name">Kardinaalspeg</p>
                        <p class="scientific">Dendropicos fuscescens</p>
                        <div class="card-meta">
                            <span class="meta-tag">Woodpeckers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Charadrius asiaticus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/caspian-plover/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Charadrius asiaticus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Caspian Plover</h3>
                        <p class="afrikaans-name">Asiatiese Strandkiewiet</p>
                        <p class="scientific">Charadrius asiaticus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Plovers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Karoo</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Hydroprogne caspia"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/caspian-tern/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Hydroprogne caspia')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Caspian Tern</h3>
                        <p class="afrikaans-name">Reusesterretjie</p>
                        <p class="scientific">Hydroprogne caspia</p>
                        <div class="card-meta">
                            <span class="meta-tag">Terns</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Melaenornis infuscatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/chat-flycatcher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Melaenornis infuscatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Chat Flycatcher</h3>
                        <p class="afrikaans-name">Grootvlieëvanger</p>
                        <p class="scientific">Melaenornis infuscatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Flycatchers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Savanna</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Thalassarche eremita"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/chatham-albatross/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Thalassarche eremita')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Chatham Albatross</h3>
                        <p class="afrikaans-name">Chathamalbatros (-malmok)</p>
                        <p class="scientific">Thalassarche eremita</p>
                        <div class="card-meta">
                            <span class="meta-tag">Albatrosses</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ploceus rubiginosus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/chestnut-weaver/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ploceus rubiginosus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Chestnut Weaver</h3>
                        <p class="afrikaans-name">Bruinwewer</p>
                        <p class="scientific">Ploceus rubiginosus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Weavers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Eremopterix leucotis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/chestnut-backed-sparrow-lark/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Eremopterix leucotis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Chestnut-backed Sparrow- Lark</h3>
                        <p class="afrikaans-name">Rooiruglewerik</p>
                        <p class="scientific">Eremopterix leucotis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Charadrius pallidus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/chestnut-banded-plover/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Charadrius pallidus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #954535"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Chestnut-banded Plover</h3>
                        <p class="afrikaans-name">Rooibandstrandkiewiet</p>
                        <p class="scientific">Charadrius pallidus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Plovers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Prionops scopifrons"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/chestnut-fronted-helmet-shrike/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Prionops scopifrons')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Chestnut-fronted Helmet-shrike</h3>
                        <p class="afrikaans-name">Stekelkophelmlaksman</p>
                        <p class="scientific">Prionops scopifrons</p>
                        <div class="card-meta">
                            <span class="meta-tag">Helmet-shrikes</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Curruca subcoerulea"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/chestnut-vented-tit-babbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Curruca subcoerulea')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Chestnut-vented Tit-babbler</h3>
                        <p class="afrikaans-name">Bosveldtjeriktik</p>
                        <p class="scientific">Curruca subcoerulea</p>
                        <div class="card-meta">
                            <span class="meta-tag">Warblers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Batis molitor"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/chinspot-batis/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Batis molitor')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Chinspot Batis</h3>
                        <p class="afrikaans-name">Witliesbosbontrokkie</p>
                        <p class="scientific">Batis molitor</p>
                        <div class="card-meta">
                            <span class="meta-tag">Batises</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cossypha dichroa"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/chorister-robin-chat/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cossypha dichroa')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Chorister Robin-chat</h3>
                        <p class="afrikaans-name">Lawaaimakerjanfrederik</p>
                        <p class="scientific">Cossypha dichroa</p>
                        <div class="card-meta">
                            <span class="meta-tag">Robin-Chats</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Fregata andrewsi"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/christmas-frigatebird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Fregata andrewsi')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Christmas Frigatebird</h3>
                        <p class="afrikaans-name">Kerseilandfregatvoël</p>
                        <p class="scientific">Fregata andrewsi</p>
                        <div class="card-meta">
                            <span class="meta-tag">Frigatebirds</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Alectoris chukar"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/chukar-partridge/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Alectoris chukar')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Chukar Partridge</h3>
                        <p class="afrikaans-name">Asiatiese Patrys</p>
                        <p class="scientific">Alectoris chukar</p>
                        <div class="card-meta">
                            <span class="meta-tag">Gamebirds</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Emberiza tahapisi"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cinnamon-breasted-bunting/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Emberiza tahapisi')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cinnamon-breasted Bunting</h3>
                        <p class="afrikaans-name">Klipstreepkoppie</p>
                        <p class="scientific">Emberiza tahapisi</p>
                        <div class="card-meta">
                            <span class="meta-tag">Buntings</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Rocky</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Euryptila subcinnamomea"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cinnamon-breasted-warbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Euryptila subcinnamomea')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cinnamon-breasted Warbler</h3>
                        <p class="afrikaans-name">Kaneelborssanger</p>
                        <p class="scientific">Euryptila subcinnamomea</p>
                        <div class="card-meta">
                            <span class="meta-tag">Warblers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Motacilla citreola"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/citrine-wagtail/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Motacilla citreola')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Citrine Wagtail</h3>
                        <p class="afrikaans-name">Sitrienkwikkie</p>
                        <p class="scientific">Motacilla citreola</p>
                        <div class="card-meta">
                            <span class="meta-tag">Wagtails</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cisticola textrix"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/cloud-cisticola/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cisticola textrix')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cloud Cisticola</h3>
                        <p class="afrikaans-name">Gevlekte Klopkloppie</p>
                        <p class="scientific">Cisticola textrix</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cisticolas</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ficedula albicollis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/collared-flycatcher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ficedula albicollis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Collared Flycatcher</h3>
                        <p class="afrikaans-name">Withalsvlieëvanger</p>
                        <p class="scientific">Ficedula albicollis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Flycatchers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cichladusa arquata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/collared-palm-thrush/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cichladusa arquata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Collared Palm Thrush</h3>
                        <p class="afrikaans-name">Palmmôrelyster</p>
                        <p class="scientific">Cichladusa arquata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Robin-Chats</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Glareola pratincola"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/collared-pratincole/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Glareola pratincola')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Collared Pratincole</h3>
                        <p class="afrikaans-name">Rooivlerksprinkaanvoël</p>
                        <p class="scientific">Glareola pratincola</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pratincoles</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Hedydipna collaris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/collared-sunbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Hedydipna collaris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #6f42c1"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Collared Sunbird</h3>
                        <p class="afrikaans-name">Kortbeksuikerbekkie</p>
                        <p class="scientific">Hedydipna collaris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sunbirds</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Turnix sylvaticus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/common-buttonquail/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Turnix sylvaticus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Common Buttonquail</h3>
                        <p class="afrikaans-name">Bosveldkwarteltjie</p>
                        <p class="scientific">Turnix sylvaticus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Buttonquails</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Buteo buteo"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/common-buzzard/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Buteo buteo')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Common Buzzard</h3>
                        <p class="afrikaans-name">Bruinjakkalsvoël</p>
                        <p class="scientific">Buteo buteo</p>
                        <div class="card-meta">
                            <span class="meta-tag">Buzzards</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cuculus canorus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/common-cuckoo/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cuculus canorus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Common Cuckoo</h3>
                        <p class="afrikaans-name">Europese Koekoek</p>
                        <p class="scientific">Cuculus canorus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cuckoos</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Tringa nebularia"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/common-greenshank/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Tringa nebularia')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Common Greenshank</h3>
                        <p class="afrikaans-name">Groenpootruiter</p>
                        <p class="scientific">Tringa nebularia</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Gallinula chloropus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/common-moorhen/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Gallinula chloropus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Common Moorhen</h3>
                        <p class="afrikaans-name">Grootwaterhoender</p>
                        <p class="scientific">Gallinula chloropus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Moorhens</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Acridotheres tristis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/common-myna/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Acridotheres tristis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Common Myna</h3>
                        <p class="afrikaans-name">Indiese Spreeu</p>
                        <p class="scientific">Acridotheres tristis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Mynas</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Gardens</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Struthio camelus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/common-ostrich/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Struthio camelus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #e83e8c"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Common Ostrich</h3>
                        <p class="afrikaans-name">Volstruis</p>
                        <p class="scientific">Struthio camelus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ostriches</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Coturnix coturnix"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/common-quail/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Coturnix coturnix')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Common Quail</h3>
                        <p class="afrikaans-name">Afrikaanse Kwartel</p>
                        <p class="scientific">Coturnix coturnix</p>
                        <div class="card-meta">
                            <span class="meta-tag">Quails</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Tringa totanus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/common-redshank/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Tringa totanus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Common Redshank</h3>
                        <p class="afrikaans-name">Rooipootruiter</p>
                        <p class="scientific">Tringa totanus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Phoenicurus phoenicurus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/common-redstart/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Phoenicurus phoenicurus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Common Redstart</h3>
                        <p class="afrikaans-name">Europese Rooistert</p>
                        <p class="scientific">Phoenicurus phoenicurus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Flycatchers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Acrocephalus scirpaceus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/common-reed-warbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Acrocephalus scirpaceus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Common Reed Warbler</h3>
                        <p class="afrikaans-name">Hermanse Rietsanger</p>
                        <p class="scientific">Acrocephalus scirpaceus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Reed-Warblers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Charadrius hiaticula"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/common-ringed-plover/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Charadrius hiaticula')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Common Ringed Plover</h3>
                        <p class="afrikaans-name">Ringnekstrandkiewiet</p>
                        <p class="scientific">Charadrius hiaticula</p>
                        <div class="card-meta">
                            <span class="meta-tag">Plovers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Actitis hypoleucos"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/common-sandpiper/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Actitis hypoleucos')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Common Sandpiper</h3>
                        <p class="afrikaans-name">Gewone Ruiter</p>
                        <p class="scientific">Actitis hypoleucos</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Rhinopomastus cyanomelas"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/common-scimitarbill/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Rhinopomastus cyanomelas')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #6f42c1"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Common Scimitarbill</h3>
                        <p class="afrikaans-name">Swartbekkakelaar</p>
                        <p class="scientific">Rhinopomastus cyanomelas</p>
                        <div class="card-meta">
                            <span class="meta-tag">Scimitarbills</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sturnus vulgaris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/common-starling/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sturnus vulgaris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: linear-gradient(135deg, #ff006e, #8338ec, #3a86ff)"></div><div class="card-color-dot" style="background: #007bff"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Common Starling</h3>
                        <p class="afrikaans-name">Europese Spreeu</p>
                        <p class="scientific">Sturnus vulgaris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Starlings</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Gardens</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Apus apus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/common-swift/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Apus apus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Common Swift</h3>
                        <p class="afrikaans-name">Europese Windswael</p>
                        <p class="scientific">Apus apus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Swifts</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sterna hirundo"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/common-tern/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sterna hirundo')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Common Tern</h3>
                        <p class="afrikaans-name">Gewone Sterretjie</p>
                        <p class="scientific">Sterna hirundo</p>
                        <div class="card-meta">
                            <span class="meta-tag">Terns</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Estrilda astrild"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/common-waxbill/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Estrilda astrild')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #e83e8c"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Common Waxbill</h3>
                        <p class="afrikaans-name">Rooibeksysie</p>
                        <p class="scientific">Estrilda astrild</p>
                        <div class="card-meta">
                            <span class="meta-tag">Waxbills</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Curruca communis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/common-whitethroat/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Curruca communis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Common Whitethroat</h3>
                        <p class="afrikaans-name">Witkeelsanger</p>
                        <p class="scientific">Curruca communis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Warblers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Campocolinus coqui"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/coqui-francolin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Campocolinus coqui')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Coqui Francolin</h3>
                        <p class="afrikaans-name">Swempie</p>
                        <p class="scientific">Campocolinus coqui</p>
                        <div class="card-meta">
                            <span class="meta-tag">Francolins</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Crex crex"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/corn-crake/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Crex crex')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Corn Crake</h3>
                        <p class="afrikaans-name">Kwartelkoning</p>
                        <p class="scientific">Crex crex</p>
                        <div class="card-meta">
                            <span class="meta-tag">Crakes</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calonectris borealis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/corys-shearwater/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calonectris borealis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cory's Shearwater</h3>
                        <p class="afrikaans-name">Geelbekpylstormvoël</p>
                        <p class="scientific">Calonectris borealis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Shearwaters</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Dromas ardeola">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Dromas ardeola')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Crab- Plover</h3>
                        <p class="afrikaans-name">Krapvreter</p>
                        <p class="scientific">Dromas ardeola</p>
                        <div class="card-meta">
                            <span class="meta-tag">Crab-Plovers</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Trachyphonus vaillantii">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Trachyphonus vaillantii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Crested Barbet</h3>
                        <p class="afrikaans-name">Kuifkophoutkapper</p>
                        <p class="scientific">Trachyphonus vaillantii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Barbets</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ortygornis sephaena">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ortygornis sephaena')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Crested Francolin</h3>
                        <p class="afrikaans-name">Bospatrys</p>
                        <p class="scientific">Ortygornis sephaena</p>
                        <div class="card-meta">
                            <span class="meta-tag">Francolins</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Guttera pucherani">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Guttera pucherani')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Crested Guineafowl</h3>
                        <p class="afrikaans-name">Kuifkoptarentaal</p>
                        <p class="scientific">Guttera pucherani</p>
                        <div class="card-meta">
                            <span class="meta-tag">Guineafowl</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pernis ptilorhynchus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pernis ptilorhynchus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Crested Honey Buzzard</h3>
                        <p class="afrikaans-name">Oosterse Wespedief</p>
                        <p class="scientific">Pernis ptilorhynchus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Buzzards</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Laniarius atrococcineus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Laniarius atrococcineus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Crimson-breasted Shrike</h3>
                        <p class="afrikaans-name">Rooiborslaksman</p>
                        <p class="scientific">Laniarius atrococcineus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Boubous</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cisticola natalensis">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cisticola natalensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Croaking Cisticola</h3>
                        <p class="afrikaans-name">Groottinktinkie</p>
                        <p class="scientific">Cisticola natalensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cisticolas</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Microcarbo coronatus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Microcarbo coronatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Crowned Cormorant</h3>
                        <p class="afrikaans-name">Kuifkopduiker</p>
                        <p class="scientific">Microcarbo coronatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cormorants</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Stephanoaetus coronatus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Stephanoaetus coronatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Crowned Eagle</h3>
                        <p class="afrikaans-name">Kroonarend</p>
                        <p class="scientific">Stephanoaetus coronatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Eagles</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Lophoceros alboterminatus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Lophoceros alboterminatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Crowned Hornbill</h3>
                        <p class="afrikaans-name">Gekroonde Neushoringvoël</p>
                        <p class="scientific">Lophoceros alboterminatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Hornbills</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Vanellus coronatus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Vanellus coronatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Crowned Lapwing</h3>
                        <p class="afrikaans-name">Kroonkiewiet</p>
                        <p class="scientific">Vanellus coronatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Lapwings</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anomalospiza imberbis">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anomalospiza imberbis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cuckoo Finch</h3>
                        <p class="afrikaans-name">Koekoekvink</p>
                        <p class="scientific">Anomalospiza imberbis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cuckoos</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calidris ferruginea">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calidris ferruginea')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Curlew Sandpiper</h3>
                        <p class="afrikaans-name">Krombekstrandloper</p>
                        <p class="scientific">Calidris ferruginea</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Amadina fasciata">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Amadina fasciata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Cut-throat Finch</h3>
                        <p class="afrikaans-name">Bandkeelvink</p>
                        <p class="scientific">Amadina fasciata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Finches</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sternula balaenarum">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sternula balaenarum')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Damara Tern</h3>
                        <p class="afrikaans-name">Damarasterretjie</p>
                        <p class="scientific">Sternula balaenarum</p>
                        <div class="card-meta">
                            <span class="meta-tag">Terns</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Melierax metabates">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Melierax metabates')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #e83e8c"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Dark Chanting Goshawk</h3>
                        <p class="afrikaans-name">Donkersingvalk</p>
                        <p class="scientific">Melierax metabates</p>
                        <div class="card-meta">
                            <span class="meta-tag">Goshawks</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ploceus bicolor">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ploceus bicolor')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Dark-backed Weaver</h3>
                        <p class="afrikaans-name">Bosmusikant</p>
                        <p class="scientific">Ploceus bicolor</p>
                        <div class="card-meta">
                            <span class="meta-tag">Weavers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pycnonotus tricolor">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pycnonotus tricolor')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Dark-capped Bulbul</h3>
                        <p class="afrikaans-name">Swartoogtiptol</p>
                        <p class="scientific">Pycnonotus tricolor</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bulbuls</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Gardens</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Iduna natalensis">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Iduna natalensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Dark-capped Yellow Warbler</h3>
                        <p class="afrikaans-name">Geelsanger</p>
                        <p class="scientific">Iduna natalensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Warblers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Neotis denhami">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Neotis denhami')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Denham's Bustard</h3>
                        <p class="afrikaans-name">Veldpou</p>
                        <p class="scientific">Neotis denhami</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bustards</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cisticola aridulus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cisticola aridulus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Desert Cisticola</h3>
                        <p class="afrikaans-name">Woestynklopkloppie</p>
                        <p class="scientific">Cisticola aridulus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cisticolas</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Falco dickinsoni">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Falco dickinsoni')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Dickinson's Kestrel</h3>
                        <p class="afrikaans-name">Dickinsonse Grysvalk</p>
                        <p class="scientific">Falco dickinsoni</p>
                        <div class="card-meta">
                            <span class="meta-tag">Falcons</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Chrysococcyx caprius">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Chrysococcyx caprius')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Diederik Cuckoo</h3>
                        <p class="afrikaans-name">Diederikkie</p>
                        <p class="scientific">Chrysococcyx caprius</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cuckoos</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Rhinoptilus africanus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Rhinoptilus africanus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Double-banded Courser</h3>
                        <p class="afrikaans-name">Dubbelbanddrawwertjie</p>
                        <p class="scientific">Rhinoptilus africanus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Coursers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pterocles bicinctus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pterocles bicinctus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Double-banded Sandgrouse</h3>
                        <p class="afrikaans-name">Dubbelbandsandpatrys</p>
                        <p class="scientific">Pterocles bicinctus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandgrouse</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Prinia hypoxantha">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Prinia hypoxantha')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Drakensberg Prinia</h3>
                        <p class="afrikaans-name">Drakensberglangstertjie</p>
                        <p class="scientific">Prinia hypoxantha</p>
                        <div class="card-meta">
                            <span class="meta-tag">Prinias</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Chaetops aurantius">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Chaetops aurantius')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Drakensberg Rockjumper</h3>
                        <p class="afrikaans-name">Oranjeborsberglyster</p>
                        <p class="scientific">Chaetops aurantius</p>
                        <div class="card-meta">
                            <span class="meta-tag">Rockjumpers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Crithagra symonsi">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Crithagra symonsi')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Drakensberg Siskin</h3>
                        <p class="afrikaans-name">Bergpietjiekanarie</p>
                        <p class="scientific">Crithagra symonsi</p>
                        <div class="card-meta">
                            <span class="meta-tag">Canaries</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calidris alpina"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/dunlin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calidris alpina')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Dunlin</h3>
                        <p class="afrikaans-name">Bontstrandloper</p>
                        <p class="scientific">Calidris alpina</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Vidua funerea"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/dusky-indigobird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Vidua funerea')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Dusky Indigobird</h3>
                        <p class="afrikaans-name">Gewone Blouvinkie</p>
                        <p class="scientific">Vidua funerea</p>
                        <div class="card-meta">
                            <span class="meta-tag">Whydahs</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pinarocorys nigricans"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/dusky-lark/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pinarocorys nigricans')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Dusky Lark</h3>
                        <p class="afrikaans-name">Donkerlewerik</p>
                        <p class="scientific">Pinarocorys nigricans</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cinnyris fuscus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/dusky-sunbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cinnyris fuscus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Dusky Sunbird</h3>
                        <p class="afrikaans-name">Namakwasuikerbekkie</p>
                        <p class="scientific">Cinnyris fuscus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sunbirds</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Karoo</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ixobrychus sturmii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/dwarf-bittern/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ixobrychus sturmii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Dwarf Bittern</h3>
                        <p class="afrikaans-name">Dwergrietreier</p>
                        <p class="scientific">Ixobrychus sturmii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bitterns</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Columba delegorguei"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/eastern-bronze-naped-pigeon/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Columba delegorguei')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #e83e8c"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Eastern Bronze-naped Pigeon</h3>
                        <p class="afrikaans-name">Withalsbosduif</p>
                        <p class="scientific">Columba delegorguei</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pigeons</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Mirafra fasciolata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/eastern-clapper-lark/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Mirafra fasciolata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Eastern Clapper Lark</h3>
                        <p class="afrikaans-name">Hoëveldklappertjie</p>
                        <p class="scientific">Mirafra fasciolata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Certhilauda semitorquata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/eastern-long-billed-lark/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Certhilauda semitorquata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Eastern Long-billed Lark</h3>
                        <p class="afrikaans-name">Grasveldlangbeklewerik</p>
                        <p class="scientific">Certhilauda semitorquata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Nicator gularis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/eastern-nicator/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Nicator gularis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Eastern Nicator</h3>
                        <p class="afrikaans-name">Geelvleknikator</p>
                        <p class="scientific">Nicator gularis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Nicators</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Alopochen aegyptiaca"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/egyptian-goose/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Alopochen aegyptiaca')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #e83e8c"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Egyptian Goose</h3>
                        <p class="afrikaans-name">Kolgans</p>
                        <p class="scientific">Alopochen aegyptiaca</p>
                        <div class="card-meta">
                            <span class="meta-tag">Geese</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Neophron percnopterus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/egyptian-vulture/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Neophron percnopterus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Egyptian Vulture</h3>
                        <p class="afrikaans-name">Egiptiese Aasvoël</p>
                        <p class="scientific">Neophron percnopterus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Vultures</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Thalasseus elegans"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/elegant-tern/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Thalasseus elegans')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Elegant Tern</h3>
                        <p class="afrikaans-name">Gevlekte Vleikuiken</p>
                        <p class="scientific">Thalasseus elegans</p>
                        <div class="card-meta">
                            <span class="meta-tag">Terns</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Falco eleonorae"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/eleonoras-falcon/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Falco eleonorae')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Eleonora's Falcon</h3>
                        <p class="afrikaans-name">Eleonoravalk</p>
                        <p class="scientific">Falco eleonorae</p>
                        <div class="card-meta">
                            <span class="meta-tag">Falcons</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Turtur chalcospilos"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/emerald-spotted-wood-dove/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Turtur chalcospilos')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #6f42c1"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Emerald-spotted Wood Dove</h3>
                        <p class="afrikaans-name">Groenvlekduifie</p>
                        <p class="scientific">Turtur chalcospilos</p>
                        <div class="card-meta">
                            <span class="meta-tag">Doves</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Botaurus stellaris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/eurasian-bittern/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Botaurus stellaris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Eurasian Bittern</h3>
                        <p class="afrikaans-name">Grootrietreier (Roerdomp)</p>
                        <p class="scientific">Botaurus stellaris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bitterns</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sylvia atricapilla"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/eurasian-blackcap/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sylvia atricapilla')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Eurasian Blackcap</h3>
                        <p class="afrikaans-name">Swartkroonsanger</p>
                        <p class="scientific">Sylvia atricapilla</p>
                        <div class="card-meta">
                            <span class="meta-tag">Warblers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Fringilla coelebs"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/eurasian-chaffinch/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Fringilla coelebs')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Eurasian Chaffinch</h3>
                        <p class="afrikaans-name">Gryskoppie</p>
                        <p class="scientific">Fringilla coelebs</p>
                        <div class="card-meta">
                            <span class="meta-tag">Finches</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Numenius arquata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/eurasian-curlew/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Numenius arquata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Eurasian Curlew</h3>
                        <p class="afrikaans-name">Grootwulp</p>
                        <p class="scientific">Numenius arquata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Curlews</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Oriolus oriolus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/eurasian-golden-oriole/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Oriolus oriolus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Eurasian Golden Oriole</h3>
                        <p class="afrikaans-name">Europese Wielewaal</p>
                        <p class="scientific">Oriolus oriolus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Orioles</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Falco subbuteo"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/eurasian-hobby/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Falco subbuteo')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Eurasian Hobby</h3>
                        <p class="afrikaans-name">Europese Boomvalk</p>
                        <p class="scientific">Falco subbuteo</p>
                        <div class="card-meta">
                            <span class="meta-tag">Falcons</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Haematopus ostralegus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/eurasian-oystercatcher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Haematopus ostralegus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Eurasian Oystercatcher</h3>
                        <p class="afrikaans-name">Bonttobie</p>
                        <p class="scientific">Haematopus ostralegus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Oystercatchers</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Otus scops"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/eurasian-scops-owl/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Otus scops')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Eurasian Scops Owl</h3>
                        <p class="afrikaans-name">Skopsuil</p>
                        <p class="scientific">Otus scops</p>
                        <div class="card-meta">
                            <span class="meta-tag">Owls</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Numenius phaeopus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/eurasian-whimbrel/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Numenius phaeopus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Eurasian Whimbrel</h3>
                        <p class="afrikaans-name">Kleinwulp</p>
                        <p class="scientific">Numenius phaeopus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Curlews</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Merops apiaster"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/european-bee-eater/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Merops apiaster')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #007bff"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>European Bee-eater</h3>
                        <p class="afrikaans-name">Europese Byvreter</p>
                        <p class="scientific">Merops apiaster</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bee-eaters</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pernis apivorus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/european-honey-buzzard/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pernis apivorus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>European Honey Buzzard</h3>
                        <p class="afrikaans-name">Wespedief</p>
                        <p class="scientific">Pernis apivorus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Buzzards</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Caprimulgus europaeus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/european-nightjar/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Caprimulgus europaeus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>European Nightjar</h3>
                        <p class="afrikaans-name">Europese Naguil</p>
                        <p class="scientific">Caprimulgus europaeus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Nightjars</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ficedula hypoleuca"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/european-pied-flycatcher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ficedula hypoleuca')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>European Pied Flycatcher</h3>
                        <p class="afrikaans-name">Europese Bontvlieëvanger</p>
                        <p class="scientific">Ficedula hypoleuca</p>
                        <div class="card-meta">
                            <span class="meta-tag">Flycatchers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Coracias garrulus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/european-roller/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Coracias garrulus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>European Roller</h3>
                        <p class="afrikaans-name">Europese Troupant</p>
                        <p class="scientific">Coracias garrulus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Rollers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Hydrobates pelagicus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/european-storm-petrel/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Hydrobates pelagicus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>European Storm Petrel</h3>
                        <p class="afrikaans-name">Europese Stormswael</p>
                        <p class="scientific">Hydrobates pelagicus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Storm-Petrels</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Streptopelia turtur"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/european-turtle-dove/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Streptopelia turtur')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #e83e8c"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>European Turtle Dove</h3>
                        <p class="afrikaans-name">Europese Tortelduif</p>
                        <p class="scientific">Streptopelia turtur</p>
                        <div class="card-meta">
                            <span class="meta-tag">Doves</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Stenostira scita">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Stenostira scita')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Fairy Flycatcher</h3>
                        <p class="afrikaans-name">Feevlieëvanger</p>
                        <p class="scientific">Stenostira scita</p>
                        <div class="card-meta">
                            <span class="meta-tag">Fairy-Flycatchers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pachyptila turtur">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pachyptila turtur')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Fairy Prion</h3>
                        <p class="afrikaans-name">Swartstertwalvisvoël</p>
                        <p class="scientific">Pachyptila turtur</p>
                        <div class="card-meta">
                            <span class="meta-tag">Prions</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Oenanthe familiaris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/familiar-chat/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Oenanthe familiaris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Familiar Chat</h3>
                        <p class="afrikaans-name">Gewone Spekvreter</p>
                        <p class="scientific">Oenanthe familiaris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Wheatears</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Euplectes axillaris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/fan-tailed-widowbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Euplectes axillaris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Fan-tailed Widowbird</h3>
                        <p class="afrikaans-name">Kortstertflap</p>
                        <p class="scientific">Euplectes axillaris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Widowbirds</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calendulauda africanoides"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/fawn-coloured-lark/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calendulauda africanoides')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Fawn-coloured Lark</h3>
                        <p class="afrikaans-name">Vaalbruinlewerik</p>
                        <p class="scientific">Calendulauda africanoides</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Caprimulgus pectoralis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/fiery-necked-nightjar/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Caprimulgus pectoralis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Fiery-necked Nightjar</h3>
                        <p class="afrikaans-name">Afrikaanse Naguil</p>
                        <p class="scientific">Caprimulgus pectoralis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Nightjars</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Melaenornis silens"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/fiscal-flycatcher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Melaenornis silens')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Fiscal Flycatcher</h3>
                        <p class="afrikaans-name">Fiskaalvlieëvanger</p>
                        <p class="scientific">Melaenornis silens</p>
                        <div class="card-meta">
                            <span class="meta-tag">Flycatchers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Mirafra rufocinnamomea"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/flappet-lark/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Mirafra rufocinnamomea')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Flappet Lark</h3>
                        <p class="afrikaans-name">Laeveldklappertjie</p>
                        <p class="scientific">Mirafra rufocinnamomea</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ardenna carneipes">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ardenna carneipes')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Flesh-footed Shearwater</h3>
                        <p class="afrikaans-name">Bruinpylstormvoël</p>
                        <p class="scientific">Ardenna carneipes</p>
                        <div class="card-meta">
                            <span class="meta-tag">Shearwaters</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Buteo trizonatus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Buteo trizonatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Forest Buzzard</h3>
                        <p class="afrikaans-name">Bosjakkalsvoël</p>
                        <p class="scientific">Buteo trizonatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Buzzards</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Crithagra scotops">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Crithagra scotops')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Forest Canary</h3>
                        <p class="afrikaans-name">Gestreepte Kanarie</p>
                        <p class="scientific">Crithagra scotops</p>
                        <div class="card-meta">
                            <span class="meta-tag">Canaries</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Dicrurus adsimilis">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Dicrurus adsimilis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Fork-tailed Drongo</h3>
                        <p class="afrikaans-name">Mikstertbyvanger</p>
                        <p class="scientific">Dicrurus adsimilis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Drongos</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Leucophaeus pipixcan">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Leucophaeus pipixcan')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Franklin's Gull</h3>
                        <p class="afrikaans-name">Franklinse Meeu</p>
                        <p class="scientific">Leucophaeus pipixcan</p>
                        <div class="card-meta">
                            <span class="meta-tag">Gulls</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Caprimulgus tristigma">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Caprimulgus tristigma')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Freckled Nightjar</h3>
                        <p class="afrikaans-name">Donkernaguil</p>
                        <p class="scientific">Caprimulgus tristigma</p>
                        <div class="card-meta">
                            <span class="meta-tag">Nightjars</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Rocky</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Dendrocygna bicolor">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Dendrocygna bicolor')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Fulvous Whistling Duck</h3>
                        <p class="afrikaans-name">Fluiteend</p>
                        <p class="scientific">Dendrocygna bicolor</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ducks</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Turnix hottentottus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Turnix hottentottus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Fynbos Buttonquail</h3>
                        <p class="afrikaans-name">Kaapse Kwarteltjie</p>
                        <p class="scientific">Turnix hottentottus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Buttonquails</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Micronisus gabar"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/gabar-goshawk/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Micronisus gabar')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Gabar Goshawk</h3>
                        <p class="afrikaans-name">Witkruissperwer (Kleinsingvalk)</p>
                        <p class="scientific">Micronisus gabar</p>
                        <div class="card-meta">
                            <span class="meta-tag">Hawks</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sylvia borin"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/garden-warbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sylvia borin')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Garden Warbler</h3>
                        <p class="afrikaans-name">Tuinsanger</p>
                        <p class="scientific">Sylvia borin</p>
                        <div class="card-meta">
                            <span class="meta-tag">Warblers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Spatula querquedula"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/garganey/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Spatula querquedula')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Garganey</h3>
                        <p class="afrikaans-name">Somereend</p>
                        <p class="scientific">Spatula querquedula</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ducks</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Megaceryle maxima"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/giant-kingfisher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Megaceryle maxima')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Giant Kingfisher</h3>
                        <p class="afrikaans-name">Reusevisvanger</p>
                        <p class="scientific">Megaceryle maxima</p>
                        <div class="card-meta">
                            <span class="meta-tag">Kingfishers</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Plegadis falcinellus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/glossy-ibis/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Plegadis falcinellus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #6f42c1"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Glossy Ibis</h3>
                        <p class="afrikaans-name">Glansibis</p>
                        <p class="scientific">Plegadis falcinellus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ibises</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Tmetothylacus tenellus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/golden-pipit/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Tmetothylacus tenellus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Golden Pipit</h3>
                        <p class="afrikaans-name">Goudkoester</p>
                        <p class="scientific">Tmetothylacus tenellus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pipits</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Emberiza flaviventris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/golden-breasted-bunting/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Emberiza flaviventris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Golden-breasted Bunting</h3>
                        <p class="afrikaans-name">Rooirugstreepkoppie</p>
                        <p class="scientific">Emberiza flaviventris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Buntings</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Campethera abingoni"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/golden-tailed-woodpecker/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Campethera abingoni')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Golden-tailed Woodpecker</h3>
                        <p class="afrikaans-name">Goudstertspeg</p>
                        <p class="scientific">Campethera abingoni</p>
                        <div class="card-meta">
                            <span class="meta-tag">Woodpeckers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ardea goliath"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/goliath-heron/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ardea goliath')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Goliath Heron</h3>
                        <p class="afrikaans-name">Reusereier</p>
                        <p class="scientific">Ardea goliath</p>
                        <div class="card-meta">
                            <span class="meta-tag">Herons</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Telophorus viridis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/gorgeous-bushshrike/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Telophorus viridis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Gorgeous Bushshrike</h3>
                        <p class="afrikaans-name">Konkoit</p>
                        <p class="scientific">Telophorus viridis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bokmakieries</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Podiceps cristatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/great-crested-grebe/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Podiceps cristatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Great Crested Grebe</h3>
                        <p class="afrikaans-name">Kuifkopdobbertjie</p>
                        <p class="scientific">Podiceps cristatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Grebes</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ardea alba"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/great-egret/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ardea alba')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Great Egret</h3>
                        <p class="afrikaans-name">Grootwitreier</p>
                        <p class="scientific">Ardea alba</p>
                        <div class="card-meta">
                            <span class="meta-tag">Herons</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Fregata minor">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Fregata minor')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Great Frigatebird</h3>
                        <p class="afrikaans-name">Grootfregatvoël</p>
                        <p class="scientific">Fregata minor</p>
                        <div class="card-meta">
                            <span class="meta-tag">Frigatebirds</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calidris tenuirostris">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calidris tenuirostris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Great Knot</h3>
                        <p class="afrikaans-name">Grootknoet</p>
                        <p class="scientific">Calidris tenuirostris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Acrocephalus arundinaceus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/great-reed-warbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Acrocephalus arundinaceus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Great Reed Warbler</h3>
                        <p class="afrikaans-name">Grootrietsanger</p>
                        <p class="scientific">Acrocephalus arundinaceus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Reed-Warblers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ardenna gravis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/great-shearwater/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ardenna gravis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Great Shearwater</h3>
                        <p class="afrikaans-name">Grootpylstormvoël</p>
                        <p class="scientific">Ardenna gravis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Shearwaters</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Gallinago media"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/great-snipe/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Gallinago media')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Great Snipe</h3>
                        <p class="afrikaans-name">Dubbelsnip</p>
                        <p class="scientific">Gallinago media</p>
                        <div class="card-meta">
                            <span class="meta-tag">Snipes</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Passer motitensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/great-sparrow/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Passer motitensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Great Sparrow</h3>
                        <p class="afrikaans-name">Grootmossie</p>
                        <p class="scientific">Passer motitensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sparrows</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Clamator glandarius"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/great-spotted-cuckoo/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Clamator glandarius')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Great Spotted Cuckoo</h3>
                        <p class="afrikaans-name">Gevlekte Koekoek</p>
                        <p class="scientific">Clamator glandarius</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cuckoos</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pelecanus onocrotalus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/great-white-pelican/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pelecanus onocrotalus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #e83e8c"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Great White Pelican</h3>
                        <p class="afrikaans-name">Witpelikaan</p>
                        <p class="scientific">Pelecanus onocrotalus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pelicans</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pterodroma macroptera"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/great-winged-petrel/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pterodroma macroptera')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Great-winged Petrel</h3>
                        <p class="afrikaans-name">Langvlerkstormvoël</p>
                        <p class="scientific">Pterodroma macroptera</p>
                        <div class="card-meta">
                            <span class="meta-tag">Petrels</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Lamprotornis chalybaeus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/greater-blue-eared-starling/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Lamprotornis chalybaeus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Greater Blue-eared Starling</h3>
                        <p class="afrikaans-name">Groot-blouoorglansspreeu</p>
                        <p class="scientific">Lamprotornis chalybaeus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Starlings</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cinnyris afer"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/greater-double-collared-sunbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cinnyris afer')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #007bff"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Greater Double-collared Sunbird</h3>
                        <p class="afrikaans-name">Groot-rooibandsuikerbekkie</p>
                        <p class="scientific">Cinnyris afer</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sunbirds</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Phoenicopterus roseus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/greater-flamingo/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Phoenicopterus roseus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #e83e8c"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Greater Flamingo</h3>
                        <p class="afrikaans-name">Grootflamink</p>
                        <p class="scientific">Phoenicopterus roseus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Flamingos</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Indicator indicator"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/greater-honeyguide/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Indicator indicator')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Greater Honeyguide</h3>
                        <p class="afrikaans-name">Grootheuningwyser</p>
                        <p class="scientific">Indicator indicator</p>
                        <div class="card-meta">
                            <span class="meta-tag">Honeyguides</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Falco rupicoloides"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/greater-kestrel/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Falco rupicoloides')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Greater Kestrel</h3>
                        <p class="afrikaans-name">Grootrooivalk</p>
                        <p class="scientific">Falco rupicoloides</p>
                        <div class="card-meta">
                            <span class="meta-tag">Falcons</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Rostratula benghalensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/greater-painted-snipe/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Rostratula benghalensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Greater Painted-snipe</h3>
                        <p class="afrikaans-name">Goudsnip</p>
                        <p class="scientific">Rostratula benghalensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Painted-Snipes</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Charadrius leschenaultii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/greater-sand-plover/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Charadrius leschenaultii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Greater Sand Plover</h3>
                        <p class="afrikaans-name">Grootstrandkiewiet</p>
                        <p class="scientific">Charadrius leschenaultii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Plovers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cecropis cucullata">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cecropis cucullata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Greater Striped Swallow</h3>
                        <p class="afrikaans-name">Grootstreepswael</p>
                        <p class="scientific">Cecropis cucullata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Swallows</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Stactolaema olivacea">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Stactolaema olivacea')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Green Barbet</h3>
                        <p class="afrikaans-name">Groenhoutkapper</p>
                        <p class="scientific">Stactolaema olivacea</p>
                        <div class="card-meta">
                            <span class="meta-tag">Barbets</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Savanna</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ceuthmochares australis">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ceuthmochares australis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Green Malkoha</h3>
                        <p class="afrikaans-name">Groenvleiloerie</p>
                        <p class="scientific">Ceuthmochares australis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cuckoos</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Tringa ochropus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Tringa ochropus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Green Sandpiper</h3>
                        <p class="afrikaans-name">Witgatruiter</p>
                        <p class="scientific">Tringa ochropus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Mandingoa nitidula">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Mandingoa nitidula')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Green Twinspot</h3>
                        <p class="afrikaans-name">Groenkolpensie</p>
                        <p class="scientific">Mandingoa nitidula</p>
                        <div class="card-meta">
                            <span class="meta-tag">Twinspots</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Phoeniculus purpureus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Phoeniculus purpureus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Green Woodhoopoe</h3>
                        <p class="afrikaans-name">Rooibekkakelaar</p>
                        <p class="scientific">Phoeniculus purpureus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Wood-hoopoes</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Camaroptera brachyura">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Camaroptera brachyura')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Green-backed Camaroptera</h3>
                        <p class="afrikaans-name">Groenrugkwêkwêvoël</p>
                        <p class="scientific">Camaroptera brachyura</p>
                        <div class="card-meta">
                            <span class="meta-tag">Camaropteras</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Butorides striata">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Butorides striata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Green-backed Heron</h3>
                        <p class="afrikaans-name">Groenrugreier</p>
                        <p class="scientific">Butorides striata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Herons</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Eremomela scotops">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Eremomela scotops')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Green-capped Eremomela</h3>
                        <p class="afrikaans-name">Donkerwangbossanger</p>
                        <p class="scientific">Eremomela scotops</p>
                        <div class="card-meta">
                            <span class="meta-tag">Eremomelas</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pytilia melba">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pytilia melba')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Green-winged Pytilia</h3>
                        <p class="afrikaans-name">Gewone Melba</p>
                        <p class="scientific">Pytilia melba</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pytilias</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Balearica regulorum">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Balearica regulorum')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Grey Crowned Crane</h3>
                        <p class="afrikaans-name">Mahem</p>
                        <p class="scientific">Balearica regulorum</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cranes</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ceblepyris caesius">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ceblepyris caesius')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Grey Cuckooshrike</h3>
                        <p class="afrikaans-name">Bloukatakoeroe</p>
                        <p class="scientific">Ceblepyris caesius</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cuckooshrikes</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Crinifer concolor">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Crinifer concolor')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Grey Go-away-bird</h3>
                        <p class="afrikaans-name">Kwêvoël</p>
                        <p class="scientific">Crinifer concolor</p>
                        <div class="card-meta">
                            <span class="meta-tag">Go-away-birds</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ardea cinerea">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ardea cinerea')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Grey Heron</h3>
                        <p class="afrikaans-name">Bloureier</p>
                        <p class="scientific">Ardea cinerea</p>
                        <div class="card-meta">
                            <span class="meta-tag">Herons</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anthoscopus caroli">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anthoscopus caroli')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Grey Penduline-tit</h3>
                        <p class="afrikaans-name">Gryskapokvoël</p>
                        <p class="scientific">Anthoscopus caroli</p>
                        <div class="card-meta">
                            <span class="meta-tag">Penduline-Tits</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Procellaria cinerea">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Procellaria cinerea')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Grey Petrel</h3>
                        <p class="afrikaans-name">Pediunker</p>
                        <p class="scientific">Procellaria cinerea</p>
                        <div class="card-meta">
                            <span class="meta-tag">Petrels</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pluvialis squatarola">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pluvialis squatarola')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Grey Plover</h3>
                        <p class="afrikaans-name">Grysstrandkiewiet</p>
                        <p class="scientific">Pluvialis squatarola</p>
                        <div class="card-meta">
                            <span class="meta-tag">Plovers</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cyanomitra veroxii">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cyanomitra veroxii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Grey Sunbird</h3>
                        <p class="afrikaans-name">Gryssuikerbekkie</p>
                        <p class="scientific">Cyanomitra veroxii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sunbirds</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Melaniparus afer">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Melaniparus afer')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Grey Tit</h3>
                        <p class="afrikaans-name">Piet-tjou-tjou-grysmees</p>
                        <p class="scientific">Melaniparus afer</p>
                        <div class="card-meta">
                            <span class="meta-tag">Tits</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Myioparus plumbeus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Myioparus plumbeus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Grey Tit- Flycatcher</h3>
                        <p class="afrikaans-name">Waaierstertvlieëvanger</p>
                        <p class="scientific">Myioparus plumbeus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Flycatchers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Motacilla cinerea">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Motacilla cinerea')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Grey Wagtail</h3>
                        <p class="afrikaans-name">Gryskwikkie</p>
                        <p class="scientific">Motacilla cinerea</p>
                        <div class="card-meta">
                            <span class="meta-tag">Wagtails</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Rivers</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Glaucestrilda perreini">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Glaucestrilda perreini')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #e83e8c"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Grey Waxbill</h3>
                        <p class="afrikaans-name">Gryssysie</p>
                        <p class="scientific">Glaucestrilda perreini</p>
                        <div class="card-meta">
                            <span class="meta-tag">Waxbills</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Camaroptera brevicaudata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/grey-backed-camaroptera/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Camaroptera brevicaudata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Grey-backed Camaroptera</h3>
                        <p class="afrikaans-name">Grysrugkwêkwêvoël</p>
                        <p class="scientific">Camaroptera brevicaudata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Camaropteras</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cisticola subruficapilla"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/grey-backed-cisticola/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cisticola subruficapilla')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Grey-backed Cisticola</h3>
                        <p class="afrikaans-name">Grysrugtinktinkie</p>
                        <p class="scientific">Cisticola subruficapilla</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cisticolas</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Eremopterix verticalis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/grey-backed-sparrow-lark/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Eremopterix verticalis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Grey-backed Sparrow- Lark</h3>
                        <p class="afrikaans-name">Grysruglewerik</p>
                        <p class="scientific">Eremopterix verticalis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Garrodia nereis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/grey-backed-storm-petrel/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Garrodia nereis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Grey-backed Storm Petrel</h3>
                        <p class="afrikaans-name">Grysstormswael</p>
                        <p class="scientific">Garrodia nereis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Storm-Petrels</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Thalassarche chrysostoma"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/grey-headed-albatross/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Thalassarche chrysostoma')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Grey-headed Albatross</h3>
                        <p class="afrikaans-name">Gryskopalbatros (-malmok)</p>
                        <p class="scientific">Thalassarche chrysostoma</p>
                        <div class="card-meta">
                            <span class="meta-tag">Albatrosses</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Malaconotus blanchoti"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/grey-headed-bushshrike/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Malaconotus blanchoti')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Grey-headed Bushshrike</h3>
                        <p class="afrikaans-name">Spookvoël</p>
                        <p class="scientific">Malaconotus blanchoti</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bush-shrikes</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Chroicocephalus cirrocephalus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/grey-headed-gull/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Chroicocephalus cirrocephalus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Grey-headed Gull</h3>
                        <p class="afrikaans-name">Gryskopmeeu</p>
                        <p class="scientific">Chroicocephalus cirrocephalus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Gulls</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Halcyon leucocephala"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/grey-headed-kingfisher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Halcyon leucocephala')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Grey-headed Kingfisher</h3>
                        <p class="afrikaans-name">Gryskopvisvanger</p>
                        <p class="scientific">Halcyon leucocephala</p>
                        <div class="card-meta">
                            <span class="meta-tag">Kingfishers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Poicephalus fuscicollis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/grey-headed-parrot/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Poicephalus fuscicollis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Grey-headed Parrot</h3>
                        <p class="afrikaans-name">Savannepapegaai</p>
                        <p class="scientific">Poicephalus fuscicollis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Parrots</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pseudhirundo griseopyga"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/grey-rumped-swallow/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pseudhirundo griseopyga')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Grey-rumped Swallow</h3>
                        <p class="afrikaans-name">Gryskruisswael</p>
                        <p class="scientific">Pseudhirundo griseopyga</p>
                        <div class="card-meta">
                            <span class="meta-tag">Swallows</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Scleroptila afra"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/grey-winged-francolin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Scleroptila afra')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Grey-winged Francolin</h3>
                        <p class="afrikaans-name">Bergpatrys</p>
                        <p class="scientific">Scleroptila afra</p>
                        <div class="card-meta">
                            <span class="meta-tag">Francolins</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Geocolaptes olivaceus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/ground-woodpecker/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Geocolaptes olivaceus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Ground Woodpecker</h3>
                        <p class="afrikaans-name">Grondspeg</p>
                        <p class="scientific">Geocolaptes olivaceus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Woodpeckers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Rocky</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Turdus litsitsirupa"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/groundscraper-thrush/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Turdus litsitsirupa')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Groundscraper Thrush</h3>
                        <p class="afrikaans-name">Gevlekte Lyster</p>
                        <p class="scientific">Turdus litsitsirupa</p>
                        <div class="card-meta">
                            <span class="meta-tag">Thrushes</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Gelochelidon nilotica"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/gull-billed-tern/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Gelochelidon nilotica')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Gull-billed Tern</h3>
                        <p class="afrikaans-name">Oostelike Sterretjie</p>
                        <p class="scientific">Gelochelidon nilotica</p>
                        <div class="card-meta">
                            <span class="meta-tag">Terns</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Promerops gurneyi"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/gurneys-sugarbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Promerops gurneyi')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Gurney's Sugarbird</h3>
                        <p class="afrikaans-name">Rooiborssuikervoël</p>
                        <p class="scientific">Promerops gurneyi</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sugarbirds</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Bostrychia hagedash"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/hadeda-ibis/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Bostrychia hagedash')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Hadeda Ibis</h3>
                        <p class="afrikaans-name">Hadeda</p>
                        <p class="scientific">Bostrychia hagedash</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ibises</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Alcedo semitorquata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/half-collared-kingfisher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Alcedo semitorquata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Half-collared Kingfisher</h3>
                        <p class="afrikaans-name">Blouvisvanger</p>
                        <p class="scientific">Alcedo semitorquata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Kingfishers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Scopus umbretta"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/hamerkop/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Scopus umbretta')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Hamerkop</h3>
                        <p class="afrikaans-name">Hamerkop</p>
                        <p class="scientific">Scopus umbretta</p>
                        <div class="card-meta">
                            <span class="meta-tag">Hamerkop</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Coturnix delegorguei"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/harlequin-quail/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Coturnix delegorguei')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Harlequin Quail</h3>
                        <p class="afrikaans-name">Bontkwartel</p>
                        <p class="scientific">Coturnix delegorguei</p>
                        <div class="card-meta">
                            <span class="meta-tag">Quails</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Chroicocephalus hartlaubii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/hartlaubs-gull/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Chroicocephalus hartlaubii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Hartlaub's Gull</h3>
                        <p class="afrikaans-name">Hartlaubse Meeu</p>
                        <p class="scientific">Chroicocephalus hartlaubii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Gulls</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Numida meleagris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/helmeted-guineafowl/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Numida meleagris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #007bff"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Helmeted Guineafowl</h3>
                        <p class="afrikaans-name">Gewone Tarentaal</p>
                        <p class="scientific">Numida meleagris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Guineafowl</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ploceus xanthops"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/holubs-golden-weaver/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ploceus xanthops')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Holub's Golden Weaver</h3>
                        <p class="afrikaans-name">Goudwewer</p>
                        <p class="scientific">Ploceus xanthops</p>
                        <div class="card-meta">
                            <span class="meta-tag">Weavers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Necrosyrtes monachus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/hooded-vulture/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Necrosyrtes monachus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">CR</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #e83e8c"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Hooded Vulture</h3>
                        <p class="afrikaans-name">Monnikaasvoël</p>
                        <p class="scientific">Necrosyrtes monachus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Vultures</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Apus horus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/horus-swift/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Apus horus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Horus Swift</h3>
                        <p class="afrikaans-name">Horuswindswael</p>
                        <p class="scientific">Apus horus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Swifts</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Corvus splendens"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/house-crow/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Corvus splendens')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>House Crow</h3>
                        <p class="afrikaans-name">Huiskraai</p>
                        <p class="scientific">Corvus splendens</p>
                        <div class="card-meta">
                            <span class="meta-tag">Crows</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Gardens</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Passer domesticus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/house-sparrow/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Passer domesticus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>House Sparrow</h3>
                        <p class="afrikaans-name">Huismossie</p>
                        <p class="scientific">Passer domesticus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sparrows</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Gardens</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Limosa haemastica"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/hudsonian-godwit/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Limosa haemastica')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Hudsonian Godwit</h3>
                        <p class="afrikaans-name">Hudsonbaaigriet</p>
                        <p class="scientific">Limosa haemastica</p>
                        <div class="card-meta">
                            <span class="meta-tag">Godwits</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Hippolais icterina"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/icterine-warbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Hippolais icterina')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Icterine Warbler</h3>
                        <p class="afrikaans-name">Spotsanger</p>
                        <p class="scientific">Hippolais icterina</p>
                        <div class="card-meta">
                            <span class="meta-tag">Warblers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pavo cristatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/indian-peafowl/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pavo cristatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Indian Peafowl</h3>
                        <p class="afrikaans-name">Makpou</p>
                        <p class="scientific">Pavo cristatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pheasants</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Thalassarche carteri"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/indian-yellow-nosed-albatross/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Thalassarche carteri')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Indian Yellow-nosed Albatross</h3>
                        <p class="afrikaans-name">Indiese Geelneusalbatros (-malmok)</p>
                        <p class="scientific">Thalassarche carteri</p>
                        <div class="card-meta">
                            <span class="meta-tag">Albatrosses</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Buteo rufofuscus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/jackal-buzzard/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Buteo rufofuscus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Jackal Buzzard</h3>
                        <p class="afrikaans-name">Rooiborsjakkalsvoël</p>
                        <p class="scientific">Buteo rufofuscus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Buzzards</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Clamator jacobinus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/jacobin-cuckoo/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Clamator jacobinus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Jacobin Cuckoo</h3>
                        <p class="afrikaans-name">Bontnuwejaarsvoël</p>
                        <p class="scientific">Clamator jacobinus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cuckoos</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Lagonosticta rhodopareia"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/jamesons-firefinch/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Lagonosticta rhodopareia')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Jameson's Firefinch</h3>
                        <p class="afrikaans-name">Jamesonse Vuurvinkie</p>
                        <p class="scientific">Lagonosticta rhodopareia</p>
                        <div class="card-meta">
                            <span class="meta-tag">Firefinches</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cercotrichas paena"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/kalahari-scrub-robin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cercotrichas paena')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Kalahari Scrub Robin</h3>
                        <p class="afrikaans-name">Kalahariwipstert</p>
                        <p class="scientific">Cercotrichas paena</p>
                        <div class="card-meta">
                            <span class="meta-tag">Scrub-Robins</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Emarginata schlegelii">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Emarginata schlegelii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Karoo Chat</h3>
                        <p class="afrikaans-name">Karoospekvreter</p>
                        <p class="scientific">Emarginata schlegelii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Chats</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Eremomela gregalis">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Eremomela gregalis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Karoo Eremomela</h3>
                        <p class="afrikaans-name">Groenbossanger</p>
                        <p class="scientific">Eremomela gregalis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Eremomelas</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Eupodotis vigorsii">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Eupodotis vigorsii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Karoo Korhaan</h3>
                        <p class="afrikaans-name">Vaalkorhaan</p>
                        <p class="scientific">Eupodotis vigorsii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bustards</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calendulauda albescens">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calendulauda albescens')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Karoo Lark</h3>
                        <p class="afrikaans-name">Karoolewerik</p>
                        <p class="scientific">Calendulauda albescens</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Karoo</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Certhilauda subcoronata">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Certhilauda subcoronata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Karoo Long-billed Lark</h3>
                        <p class="afrikaans-name">Karoolangbeklewerik</p>
                        <p class="scientific">Certhilauda subcoronata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Prinia maculosa">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Prinia maculosa')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Karoo Prinia</h3>
                        <p class="afrikaans-name">Karoolangstertjie</p>
                        <p class="scientific">Prinia maculosa</p>
                        <div class="card-meta">
                            <span class="meta-tag">Prinias</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cercotrichas coryphoeus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cercotrichas coryphoeus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Karoo Scrub Robin</h3>
                        <p class="afrikaans-name">Slangverklikker</p>
                        <p class="scientific">Cercotrichas coryphoeus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Scrub-Robins</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Savanna</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Turdus smithi">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Turdus smithi')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Karoo Thrush</h3>
                        <p class="afrikaans-name">Geelbeklyster</p>
                        <p class="scientific">Turdus smithi</p>
                        <div class="card-meta">
                            <span class="meta-tag">Thrushes</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Larus dominicanus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Larus dominicanus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Kelp Gull</h3>
                        <p class="afrikaans-name">Kelpmeeu</p>
                        <p class="scientific">Larus dominicanus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Gulls</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Aphrodroma brevirostris">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Aphrodroma brevirostris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Kerguelen Petrel</h3>
                        <p class="afrikaans-name">Kerguelense Stormvoël</p>
                        <p class="scientific">Aphrodroma brevirostris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Petrels</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Aptenodytes patagonicus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Aptenodytes patagonicus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>King Penguin</h3>
                        <p class="afrikaans-name">Koningpikkewyn</p>
                        <p class="scientific">Aptenodytes patagonicus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Penguins</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Charadrius pecuarius">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Charadrius pecuarius')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Kittlitz's Plover</h3>
                        <p class="afrikaans-name">Geelborsstrandkiewiet</p>
                        <p class="scientific">Charadrius pecuarius</p>
                        <div class="card-meta">
                            <span class="meta-tag">Plovers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Chrysococcyx klaas">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Chrysococcyx klaas')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Klaas's Cuckoo</h3>
                        <p class="afrikaans-name">Meitjie</p>
                        <p class="scientific">Chrysococcyx klaas</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cuckoos</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sarkidiornis melanotos">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sarkidiornis melanotos')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Knob-billed Duck</h3>
                        <p class="afrikaans-name">Knobbeleend</p>
                        <p class="scientific">Sarkidiornis melanotos</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ducks</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Tauraco corythaix"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/knysna-turaco/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Tauraco corythaix')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #007bff"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Knysna Turaco</h3>
                        <p class="afrikaans-name">Knysnaloerie</p>
                        <p class="scientific">Tauraco corythaix</p>
                        <div class="card-meta">
                            <span class="meta-tag">Turacos</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Bradypterus sylvaticus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/knysna-warbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Bradypterus sylvaticus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            <span class="badge badge-threatened">VU</span>
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Knysna Warbler</h3>
                        <p class="afrikaans-name">Knysnaruigtesanger</p>
                        <p class="scientific">Bradypterus sylvaticus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bush-Warblers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Campethera notata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/knysna-woodpecker/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Campethera notata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Knysna Woodpecker</h3>
                        <p class="afrikaans-name">Knysnaspeg</p>
                        <p class="scientific">Campethera notata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Woodpeckers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ardeotis kori"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/kori-bustard/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ardeotis kori')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Kori Bustard</h3>
                        <p class="afrikaans-name">Gompou</p>
                        <p class="scientific">Ardeotis kori</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bustards</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Turdus libonyana"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/kurrichane-thrush/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Turdus libonyana')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Kurrichane Thrush</h3>
                        <p class="afrikaans-name">Rooibeklyster</p>
                        <p class="scientific">Turdus libonyana</p>
                        <div class="card-meta">
                            <span class="meta-tag">Thrushes</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Falco biarmicus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/lanner-falcon/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Falco biarmicus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lanner Falcon</h3>
                        <p class="afrikaans-name">Edelvalk</p>
                        <p class="scientific">Falco biarmicus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Falcons</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Torgos tracheliotos"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/lappet-faced-vulture/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Torgos tracheliotos')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #e83e8c"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lappet-faced Vulture</h3>
                        <p class="afrikaans-name">Swartaasvoël</p>
                        <p class="scientific">Torgos tracheliotos</p>
                        <div class="card-meta">
                            <span class="meta-tag">Vultures</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Galerida magnirostris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/large-billed-lark/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Galerida magnirostris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Large-billed Lark</h3>
                        <p class="afrikaans-name">Dikbeklewerik</p>
                        <p class="scientific">Galerida magnirostris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Emberiza impetuani"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/lark-like-bunting/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Emberiza impetuani')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lark-like Bunting</h3>
                        <p class="afrikaans-name">Vaalstreepkoppie</p>
                        <p class="scientific">Emberiza impetuani</p>
                        <div class="card-meta">
                            <span class="meta-tag">Buntings</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Karoo</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Spilopelia senegalensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/laughing-dove/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Spilopelia senegalensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #e83e8c"></div><div class="card-color-dot" style="background: #007bff"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Laughing Dove</h3>
                        <p class="afrikaans-name">Rooiborsduifie</p>
                        <p class="scientific">Spilopelia senegalensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Doves</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Leucophaeus atricilla"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/laughing-gull/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Leucophaeus atricilla')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Laughing Gull</h3>
                        <p class="afrikaans-name">Lagmeeu</p>
                        <p class="scientific">Leucophaeus atricilla</p>
                        <div class="card-meta">
                            <span class="meta-tag">Gulls</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Curruca layardi"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/layards-tit-babbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Curruca layardi')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Layard's Tit-babbler</h3>
                        <p class="afrikaans-name">Grystjeriktik</p>
                        <p class="scientific">Curruca layardi</p>
                        <div class="card-meta">
                            <span class="meta-tag">Warblers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Phoebastria immutabilis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/laysan-albatross/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Phoebastria immutabilis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Laysan Albatross</h3>
                        <p class="afrikaans-name">Swartwangalbatros (-malmok)</p>
                        <p class="scientific">Phoebastria immutabilis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Albatrosses</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cisticola aberrans"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/lazy-cisticola/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cisticola aberrans')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lazy Cisticola</h3>
                        <p class="afrikaans-name">Luitinktinkie</p>
                        <p class="scientific">Cisticola aberrans</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cisticolas</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Rocky</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Hydrobates leucorhous"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/leachs-storm-petrel/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Hydrobates leucorhous')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">CR</span>
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Leach's Storm Petrel</h3>
                        <p class="afrikaans-name">Swaalstertstormswael</p>
                        <p class="scientific">Hydrobates leucorhous</p>
                        <div class="card-meta">
                            <span class="meta-tag">Storm-Petrels</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Columba larvata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/lemon-dove/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Columba larvata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lemon Dove</h3>
                        <p class="afrikaans-name">Kaneelduifie</p>
                        <p class="scientific">Columba larvata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pigeons</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Crithagra citrinipectus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/lemon-breasted-canary/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Crithagra citrinipectus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lemon-breasted Canary</h3>
                        <p class="afrikaans-name">Geelborskanarie</p>
                        <p class="scientific">Crithagra citrinipectus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Canaries</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Larus fuscus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/lesser-black-backed-gull/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Larus fuscus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lesser Black-backed Gull</h3>
                        <p class="afrikaans-name">Kleinswartrugmeeu</p>
                        <p class="scientific">Larus fuscus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Gulls</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Thalasseus bengalensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/lesser-crested-tern/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Thalasseus bengalensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lesser Crested Tern</h3>
                        <p class="afrikaans-name">Kuifkopsterretjie</p>
                        <p class="scientific">Thalasseus bengalensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Terns</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cuculus poliocephalus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/lesser-cuckoo/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cuculus poliocephalus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lesser Cuckoo</h3>
                        <p class="afrikaans-name">Kleinkoekoek</p>
                        <p class="scientific">Cuculus poliocephalus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cuckoos</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Phoeniconaias minor"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/lesser-flamingo/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Phoeniconaias minor')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #e83e8c"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lesser Flamingo</h3>
                        <p class="afrikaans-name">Kleinflamink</p>
                        <p class="scientific">Phoeniconaias minor</p>
                        <div class="card-meta">
                            <span class="meta-tag">Flamingos</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Fregata ariel"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/lesser-frigatebird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Fregata ariel')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lesser Frigatebird</h3>
                        <p class="afrikaans-name">Kleinfregatvoël</p>
                        <p class="scientific">Fregata ariel</p>
                        <div class="card-meta">
                            <span class="meta-tag">Frigatebirds</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Lanius minor">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Lanius minor')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lesser Grey Shrike</h3>
                        <p class="afrikaans-name">Gryslaksman</p>
                        <p class="scientific">Lanius minor</p>
                        <div class="card-meta">
                            <span class="meta-tag">Shrikes</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Indicator minor">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Indicator minor')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lesser Honeyguide</h3>
                        <p class="afrikaans-name">Kleinheuningwyser</p>
                        <p class="scientific">Indicator minor</p>
                        <div class="card-meta">
                            <span class="meta-tag">Honeyguides</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Microparra capensis">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Microparra capensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lesser Jacana</h3>
                        <p class="afrikaans-name">Dwerglangtoon</p>
                        <p class="scientific">Microparra capensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Jacanas</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Falco naumanni">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Falco naumanni')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lesser Kestrel</h3>
                        <p class="afrikaans-name">Kleinrooivalk</p>
                        <p class="scientific">Falco naumanni</p>
                        <div class="card-meta">
                            <span class="meta-tag">Falcons</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ploceus intermedius">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ploceus intermedius')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lesser Masked Weaver</h3>
                        <p class="afrikaans-name">Kleingeelvink</p>
                        <p class="scientific">Ploceus intermedius</p>
                        <div class="card-meta">
                            <span class="meta-tag">Weavers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Paragallinula angulata">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Paragallinula angulata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lesser Moorhen</h3>
                        <p class="afrikaans-name">Kleinwaterhoender</p>
                        <p class="scientific">Paragallinula angulata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Moorhens</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anous tenuirostris">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anous tenuirostris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lesser Noddy</h3>
                        <p class="afrikaans-name">Kleinbruinsterretjie</p>
                        <p class="scientific">Anous tenuirostris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Noddies</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Charadrius mongolus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Charadrius mongolus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lesser Sand Plover</h3>
                        <p class="afrikaans-name">Mongoolse Strandkiewiet</p>
                        <p class="scientific">Charadrius mongolus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Plovers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Clanga pomarina">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Clanga pomarina')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lesser Spotted Eagle</h3>
                        <p class="afrikaans-name">Gevlekte Arend</p>
                        <p class="scientific">Clanga pomarina</p>
                        <div class="card-meta">
                            <span class="meta-tag">Eagles</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cecropis abyssinica">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cecropis abyssinica')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lesser Striped Swallow</h3>
                        <p class="afrikaans-name">Kleinstreepswael</p>
                        <p class="scientific">Cecropis abyssinica</p>
                        <div class="card-meta">
                            <span class="meta-tag">Swallows</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Acrocephalus gracilirostris">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Acrocephalus gracilirostris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lesser Swamp Warbler</h3>
                        <p class="afrikaans-name">Kaapse Rietsanger</p>
                        <p class="scientific">Acrocephalus gracilirostris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Reed-Warblers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Curruca curruca">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Curruca curruca')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lesser Whitethroat</h3>
                        <p class="afrikaans-name">Klein Witkeelsanger</p>
                        <p class="scientific">Curruca curruca</p>
                        <div class="card-meta">
                            <span class="meta-tag">Warblers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Tringa flavipes">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Tringa flavipes')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lesser Yellowlegs</h3>
                        <p class="afrikaans-name">Kleingeelpootruiter</p>
                        <p class="scientific">Tringa flavipes</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cisticola tinniens">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cisticola tinniens')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Levaillant's Cisticola</h3>
                        <p class="afrikaans-name">Vleitinktinkie</p>
                        <p class="scientific">Cisticola tinniens</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cisticolas</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Clamator levaillantii">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Clamator levaillantii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Levaillant's Cuckoo</h3>
                        <p class="afrikaans-name">Gestreepte Nuwejaarsvoël</p>
                        <p class="scientific">Clamator levaillantii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cuckoos</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Phoebetria palpebrata">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Phoebetria palpebrata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Light-mantled Albatross</h3>
                        <p class="afrikaans-name">Swartkopalbatros (-malmok)</p>
                        <p class="scientific">Phoebetria palpebrata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Albatrosses</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Coracias caudatus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Coracias caudatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #6f42c1"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lilac-breasted Roller</h3>
                        <p class="afrikaans-name">Gewone Troupant</p>
                        <p class="scientific">Coracias caudatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Rollers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Merops pusillus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Merops pusillus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Little Bee-eater</h3>
                        <p class="afrikaans-name">Kleinbyvreter</p>
                        <p class="scientific">Merops pusillus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bee-eaters</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ixobrychus minutus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ixobrychus minutus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Little Bittern</h3>
                        <p class="afrikaans-name">Kleinrietreier (Woudapie)</p>
                        <p class="scientific">Ixobrychus minutus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bitterns</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Egretta caerulea">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Egretta caerulea')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Little Blue Heron</h3>
                        <p class="afrikaans-name">Kleinbloureier</p>
                        <p class="scientific">Egretta caerulea</p>
                        <div class="card-meta">
                            <span class="meta-tag">Herons</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Zapornia parva">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Zapornia parva')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Little Crake</h3>
                        <p class="afrikaans-name">Kleinriethaan</p>
                        <p class="scientific">Zapornia parva</p>
                        <div class="card-meta">
                            <span class="meta-tag">Crakes</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Egretta garzetta">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Egretta garzetta')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Little Egret</h3>
                        <p class="afrikaans-name">Kleinwitreier</p>
                        <p class="scientific">Egretta garzetta</p>
                        <div class="card-meta">
                            <span class="meta-tag">Herons</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Tachybaptus ruficollis">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Tachybaptus ruficollis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Little Grebe</h3>
                        <p class="afrikaans-name">Kleindobbertjie</p>
                        <p class="scientific">Tachybaptus ruficollis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Grebes</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Charadrius dubius">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Charadrius dubius')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Little Ringed Plover</h3>
                        <p class="afrikaans-name">Kleinringnekstrandkiewiet</p>
                        <p class="scientific">Charadrius dubius</p>
                        <div class="card-meta">
                            <span class="meta-tag">Plovers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Bradypterus baboecala">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Bradypterus baboecala')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Little Rush Warbler</h3>
                        <p class="afrikaans-name">Kaapse Vleisanger</p>
                        <p class="scientific">Bradypterus baboecala</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bush-Warblers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Puffinus assimilis">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Puffinus assimilis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Little Shearwater</h3>
                        <p class="afrikaans-name">Kleinpylstormvoël</p>
                        <p class="scientific">Puffinus assimilis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Shearwaters</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Accipiter minullus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/little-sparrowhawk/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Accipiter minullus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Little Sparrowhawk</h3>
                        <p class="afrikaans-name">Kleinsperwer</p>
                        <p class="scientific">Accipiter minullus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Hawks</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calidris minuta"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/little-stint/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calidris minuta')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Little Stint</h3>
                        <p class="afrikaans-name">Kleinstrandloper</p>
                        <p class="scientific">Calidris minuta</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Apus affinis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/little-swift/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Apus affinis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Little Swift</h3>
                        <p class="afrikaans-name">Kleinwindswael</p>
                        <p class="scientific">Apus affinis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Swifts</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Rocky</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sternula albifrons"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/little-tern/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sternula albifrons')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Little Tern</h3>
                        <p class="afrikaans-name">Kleinsterretjie</p>
                        <p class="scientific">Sternula albifrons</p>
                        <div class="card-meta">
                            <span class="meta-tag">Terns</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Erythrocercus livingstonei"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/livingstones-flycatcher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Erythrocercus livingstonei')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Livingstone's Flycatcher</h3>
                        <p class="afrikaans-name">Rooistertvlieëvanger</p>
                        <p class="scientific">Erythrocercus livingstonei</p>
                        <div class="card-meta">
                            <span class="meta-tag">Flycatchers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Tauraco livingstonii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/livingstones-turaco/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Tauraco livingstonii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Livingstone's Turaco</h3>
                        <p class="afrikaans-name">Mosambiekloerie</p>
                        <p class="scientific">Tauraco livingstonii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Turacos</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Kaupifalco monogrammicus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/lizard-buzzard/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Kaupifalco monogrammicus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Lizard Buzzard</h3>
                        <p class="afrikaans-name">Akkedisvalk</p>
                        <p class="scientific">Kaupifalco monogrammicus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Buzzards</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sylvietta rufescens"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/long-billed-crombec/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sylvietta rufescens')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Long-billed Crombec</h3>
                        <p class="afrikaans-name">Bosveldstompstert</p>
                        <p class="scientific">Sylvietta rufescens</p>
                        <div class="card-meta">
                            <span class="meta-tag">Crombecs</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Lophaetus occipitalis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/long-crested-eagle/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Lophaetus occipitalis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Long-crested Eagle</h3>
                        <p class="afrikaans-name">Langkuifarend</p>
                        <p class="scientific">Lophaetus occipitalis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Eagles</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Stercorarius longicaudus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/long-tailed-jaeger/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Stercorarius longicaudus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Long-tailed Jaeger</h3>
                        <p class="afrikaans-name">Langstertroofmeeu</p>
                        <p class="scientific">Stercorarius longicaudus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Skuas</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Vidua paradisaea"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/long-tailed-paradise-whydah/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Vidua paradisaea')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Long-tailed Paradise Whydah</h3>
                        <p class="afrikaans-name">Gewone Paradysvink</p>
                        <p class="scientific">Vidua paradisaea</p>
                        <div class="card-meta">
                            <span class="meta-tag">Whydahs</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Euplectes progne"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/long-tailed-widowbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Euplectes progne')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Long-tailed Widowbird</h3>
                        <p class="afrikaans-name">Langstertflap</p>
                        <p class="scientific">Euplectes progne</p>
                        <div class="card-meta">
                            <span class="meta-tag">Widowbirds</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Vanellus crassirostris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/long-toed-lapwing/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Vanellus crassirostris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Long-toed Lapwing</h3>
                        <p class="afrikaans-name">Witvlerkkiewiet</p>
                        <p class="scientific">Vanellus crassirostris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Lapwings</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calidris subminuta"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/long-toed-stint/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calidris subminuta')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Long-toed Stint</h3>
                        <p class="afrikaans-name">Langtoonstrandloper</p>
                        <p class="scientific">Calidris subminuta</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Neotis ludwigii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/ludwigs-bustard/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Neotis ludwigii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Ludwig's Bustard</h3>
                        <p class="afrikaans-name">Ludwigse Pou</p>
                        <p class="scientific">Neotis ludwigii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bustards</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Eudyptes chrysolophus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/macaroni-penguin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Eudyptes chrysolophus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Macaroni Penguin</h3>
                        <p class="afrikaans-name">Langkuifpikkewyn</p>
                        <p class="scientific">Eudyptes chrysolophus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Penguins</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Oxyura maccoa"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/maccoa-duck/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Oxyura maccoa')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #007bff"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Maccoa Duck</h3>
                        <p class="afrikaans-name">Bloubekeend</p>
                        <p class="scientific">Oxyura maccoa</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ducks</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cuculus rochii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/madagascar-cuckoo/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cuculus rochii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Madagascar Cuckoo</h3>
                        <p class="afrikaans-name">Madagaskarkoekoek</p>
                        <p class="scientific">Cuculus rochii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cuckoos</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Glareola ocularis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/madagascar-pratincole/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Glareola ocularis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #FFFDD0"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Madagascar Pratincole</h3>
                        <p class="afrikaans-name">Brilwewer</p>
                        <p class="scientific">Glareola ocularis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pratincoles</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Spermestes fringilloides"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/magpie-mannikin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Spermestes fringilloides')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Magpie Mannikin</h3>
                        <p class="afrikaans-name">Dikbekfret</p>
                        <p class="scientific">Spermestes fringilloides</p>
                        <div class="card-meta">
                            <span class="meta-tag">Mannikins</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Urolestes melanoleucus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/magpie-shrike/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Urolestes melanoleucus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Magpie Shrike</h3>
                        <p class="afrikaans-name">Langstertlaksman</p>
                        <p class="scientific">Urolestes melanoleucus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Shrikes</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Corythornis cristatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/malachite-kingfisher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Corythornis cristatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Malachite Kingfisher</h3>
                        <p class="afrikaans-name">Kuifkopvisvanger</p>
                        <p class="scientific">Corythornis cristatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Kingfishers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Nectarinia famosa"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/malachite-sunbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Nectarinia famosa')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Malachite Sunbird</h3>
                        <p class="afrikaans-name">Jangroentjie</p>
                        <p class="scientific">Nectarinia famosa</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sunbirds</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ardeola idae"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/malagasy-pond-heron/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ardeola idae')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Malagasy Pond Heron</h3>
                        <p class="afrikaans-name">Madagaskar-ralreier</p>
                        <p class="scientific">Ardeola idae</p>
                        <div class="card-meta">
                            <span class="meta-tag">Herons</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anas platyrhynchos"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/mallard/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anas platyrhynchos')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Mallard</h3>
                        <p class="afrikaans-name">Groenkopeend</p>
                        <p class="scientific">Anas platyrhynchos</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ducks</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Halcyon senegaloides"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/mangrove-kingfisher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Halcyon senegaloides')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Mangrove Kingfisher</h3>
                        <p class="afrikaans-name">Manglietvisvanger</p>
                        <p class="scientific">Halcyon senegaloides</p>
                        <div class="card-meta">
                            <span class="meta-tag">Kingfishers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Puffinus puffinus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/manx-shearwater/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Puffinus puffinus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Manx Shearwater</h3>
                        <p class="afrikaans-name">Swartbekpylstormvoël</p>
                        <p class="scientific">Puffinus puffinus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Shearwaters</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Leptoptilos crumenifer"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/marabou-stork/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Leptoptilos crumenifer')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Marabou Stork</h3>
                        <p class="afrikaans-name">Maraboe</p>
                        <p class="scientific">Leptoptilos crumenifer</p>
                        <div class="card-meta">
                            <span class="meta-tag">Storks</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Melaenornis mariquensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/marico-flycatcher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Melaenornis mariquensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Marico Flycatcher</h3>
                        <p class="afrikaans-name">Maricovlieëvanger</p>
                        <p class="scientific">Melaenornis mariquensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Flycatchers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Savanna</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cinnyris mariquensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/marico-sunbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cinnyris mariquensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #6f42c1"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Marico Sunbird</h3>
                        <p class="afrikaans-name">Maricosuikerbekkie</p>
                        <p class="scientific">Cinnyris mariquensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sunbirds</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Asio capensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/marsh-owl/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Asio capensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Marsh Owl</h3>
                        <p class="afrikaans-name">Vlei-uil</p>
                        <p class="scientific">Asio capensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Owls</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Tringa stagnatilis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/marsh-sandpiper/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Tringa stagnatilis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Marsh Sandpiper</h3>
                        <p class="afrikaans-name">Moerasruiter</p>
                        <p class="scientific">Tringa stagnatilis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Acrocephalus palustris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/marsh-warbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Acrocephalus palustris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Marsh Warbler</h3>
                        <p class="afrikaans-name">Europese Rietsanger</p>
                        <p class="scientific">Acrocephalus palustris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Reed-Warblers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Polemaetus bellicosus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/martial-eagle/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Polemaetus bellicosus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Martial Eagle</h3>
                        <p class="afrikaans-name">Breëkoparend</p>
                        <p class="scientific">Polemaetus bellicosus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Eagles</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sula dactylatra"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/masked-booby/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sula dactylatra')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Masked Booby</h3>
                        <p class="afrikaans-name">Witmalgas</p>
                        <p class="scientific">Sula dactylatra</p>
                        <div class="card-meta">
                            <span class="meta-tag">Boobies</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Hydrobates matsudairae"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/matsudairas-storm-petrel/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Hydrobates matsudairae')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Matsudaira's Storm Petrel</h3>
                        <p class="afrikaans-name">Oosterse Stormswael</p>
                        <p class="scientific">Hydrobates matsudairae</p>
                        <div class="card-meta">
                            <span class="meta-tag">Storm-Petrels</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Mirafra cheniana"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/melodious-lark/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Mirafra cheniana')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Melodious Lark</h3>
                        <p class="afrikaans-name">Spotlewerik</p>
                        <p class="scientific">Mirafra cheniana</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Lamprotornis mevesii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/mevess-starling/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Lamprotornis mevesii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #6f42c1"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Meves's Starling</h3>
                        <p class="afrikaans-name">Langstertglansspreeu</p>
                        <p class="scientific">Lamprotornis mevesii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Starlings</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Poicephalus meyeri">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Poicephalus meyeri')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Meyer's Parrot</h3>
                        <p class="afrikaans-name">Bosveldpapegaai</p>
                        <p class="scientific">Poicephalus meyeri</p>
                        <div class="card-meta">
                            <span class="meta-tag">Parrots</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Lamprotornis elisabeth">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Lamprotornis elisabeth')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #6f42c1"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Miombo Blue-eared Starling</h3>
                        <p class="afrikaans-name">Klein-blouoorglansspreeu</p>
                        <p class="scientific">Lamprotornis elisabeth</p>
                        <div class="card-meta">
                            <span class="meta-tag">Starlings</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Savanna</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Thamnolaea cinnamomeiventris">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Thamnolaea cinnamomeiventris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Mocking Cliff Chat</h3>
                        <p class="afrikaans-name">Dassievoël</p>
                        <p class="scientific">Thamnolaea cinnamomeiventris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Chats</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Rocky</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Mirafra passerina">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Mirafra passerina')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Monotonous Lark</h3>
                        <p class="afrikaans-name">Bosveldlewerik</p>
                        <p class="scientific">Mirafra passerina</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Circus pygargus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Circus pygargus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Montagu's Harrier</h3>
                        <p class="afrikaans-name">Blouvleivalk (-paddavreter)</p>
                        <p class="scientific">Circus pygargus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Harriers</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cecropis senegalensis">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cecropis senegalensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Mosque Swallow</h3>
                        <p class="afrikaans-name">Moskeeswael</p>
                        <p class="scientific">Cecropis senegalensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Swallows</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Telacanthura ussheri">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Telacanthura ussheri')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Mottled Spinetail</h3>
                        <p class="afrikaans-name">Gevlekte Stekelstert</p>
                        <p class="scientific">Telacanthura ussheri</p>
                        <div class="card-meta">
                            <span class="meta-tag">Spinetails</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anthus hoeschi">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anthus hoeschi')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Mountain Pipit</h3>
                        <p class="afrikaans-name">Bergkoester</p>
                        <p class="scientific">Anthus hoeschi</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pipits</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Motacilla clara"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/mountain-wagtail/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Motacilla clara')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Mountain Wagtail</h3>
                        <p class="afrikaans-name">Bergkwikkie</p>
                        <p class="scientific">Motacilla clara</p>
                        <div class="card-meta">
                            <span class="meta-tag">Wagtails</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Myrmecocichla monticola"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/mountain-wheatear/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Myrmecocichla monticola')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Mountain Wheatear</h3>
                        <p class="afrikaans-name">Bergwagter</p>
                        <p class="scientific">Myrmecocichla monticola</p>
                        <div class="card-meta">
                            <span class="meta-tag">Chats</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cygnus olor"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/mute-swan/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cygnus olor')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Mute Swan</h3>
                        <p class="afrikaans-name">Makousiswaan</p>
                        <p class="scientific">Cygnus olor</p>
                        <div class="card-meta">
                            <span class="meta-tag">Swans</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Oena capensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/namaqua-dove/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Oena capensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #007bff"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Namaqua Dove</h3>
                        <p class="afrikaans-name">Namakwaduifie</p>
                        <p class="scientific">Oena capensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Doves</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pterocles namaqua"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/namaqua-sandgrouse/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pterocles namaqua')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Namaqua Sandgrouse</h3>
                        <p class="afrikaans-name">Kelkiewyn</p>
                        <p class="scientific">Pterocles namaqua</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandgrouse</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Karoo</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Phragmacia substriata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/namaqua-warbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Phragmacia substriata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Namaqua Warbler</h3>
                        <p class="afrikaans-name">Namakwalangstertjie</p>
                        <p class="scientific">Phragmacia substriata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Warblers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Apaloderma narina"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/narina-trogon/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Apaloderma narina')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Narina Trogon</h3>
                        <p class="afrikaans-name">Bosloerie</p>
                        <p class="scientific">Apaloderma narina</p>
                        <div class="card-meta">
                            <span class="meta-tag">Trogons</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pternistis natalensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/natal-spurfowl/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pternistis natalensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Natal Spurfowl</h3>
                        <p class="afrikaans-name">Natalse Fisant</p>
                        <p class="scientific">Pternistis natalensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Francolins</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cisticola fulvicapilla"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/neddicky/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cisticola fulvicapilla')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Neddicky</h3>
                        <p class="afrikaans-name">Neddikkie</p>
                        <p class="scientific">Cisticola fulvicapilla</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cisticolas</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cinnyris neergaardi"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/neergaards-sunbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cinnyris neergaardi')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Neergaard's Sunbird</h3>
                        <p class="afrikaans-name">Bloukruissuikerbekkie</p>
                        <p class="scientific">Cinnyris neergaardi</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sunbirds</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anthus nicholsoni"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/nicholsons-pipit/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anthus nicholsoni')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Nicholson's Pipit</h3>
                        <p class="afrikaans-name">Nicholsonse Koester</p>
                        <p class="scientific">Anthus nicholsoni</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pipits</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Afrotis afraoides"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/northern-black-korhaan/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Afrotis afraoides')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Northern Black Korhaan</h3>
                        <p class="afrikaans-name">Witvlerkkorhaan</p>
                        <p class="scientific">Afrotis afraoides</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bustards</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Macronectes halli"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/northern-giant-petrel/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Macronectes halli')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Northern Giant Petrel</h3>
                        <p class="afrikaans-name">Grootnellie</p>
                        <p class="scientific">Macronectes halli</p>
                        <div class="card-meta">
                            <span class="meta-tag">Petrels</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anas acuta"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/northern-pintail/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anas acuta')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Northern Pintail</h3>
                        <p class="afrikaans-name">Pylsterteend</p>
                        <p class="scientific">Anas acuta</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ducks</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Eudyptes moseleyi"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/northern-rockhopper-penguin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Eudyptes moseleyi')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Northern Rockhopper Penguin</h3>
                        <p class="afrikaans-name">Noordelike Geelkuifpikkewyn</p>
                        <p class="scientific">Eudyptes moseleyi</p>
                        <div class="card-meta">
                            <span class="meta-tag">Penguins</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Diomedea sanfordi"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/northern-royal-albatross/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Diomedea sanfordi')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Northern Royal Albatross</h3>
                        <p class="afrikaans-name">Swartvlerkkoningalbatros (-malmok)</p>
                        <p class="scientific">Diomedea sanfordi</p>
                        <div class="card-meta">
                            <span class="meta-tag">Albatrosses</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Oenanthe oenanthe"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/northern-wheatear/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Oenanthe oenanthe')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Northern Wheatear</h3>
                        <p class="afrikaans-name">Europese Skaapwagter</p>
                        <p class="scientific">Oenanthe oenanthe</p>
                        <div class="card-meta">
                            <span class="meta-tag">Wheatears</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Merops superciliosus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/olive-bee-eater/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Merops superciliosus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Olive Bee-eater</h3>
                        <p class="afrikaans-name">Olyfbyvreter</p>
                        <p class="scientific">Merops superciliosus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bee-eaters</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Chlorophoneus olivaceus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/olive-bushshrike/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Chlorophoneus olivaceus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Olive Bushshrike</h3>
                        <p class="afrikaans-name">Olyfboslaksman</p>
                        <p class="scientific">Chlorophoneus olivaceus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bush-shrikes</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cyanomitra olivacea"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/olive-sunbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cyanomitra olivacea')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Olive Sunbird</h3>
                        <p class="afrikaans-name">Olyfsuikerbekkie</p>
                        <p class="scientific">Cyanomitra olivacea</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sunbirds</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Turdus olivaceus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/olive-thrush/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Turdus olivaceus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Olive Thrush</h3>
                        <p class="afrikaans-name">Olyflyster</p>
                        <p class="scientific">Turdus olivaceus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Thrushes</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Dendropicos griseocephalus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/olive-woodpecker/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Dendropicos griseocephalus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Olive Woodpecker</h3>
                        <p class="afrikaans-name">Gryskopspeg</p>
                        <p class="scientific">Dendropicos griseocephalus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Woodpeckers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Hippolais olivetorum"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/olive-tree-warbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Hippolais olivetorum')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Olive-tree Warbler</h3>
                        <p class="afrikaans-name">Olyfboomsanger</p>
                        <p class="scientific">Hippolais olivetorum</p>
                        <div class="card-meta">
                            <span class="meta-tag">Warblers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Geokichla gurneyi"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/orange-ground-thrush/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Geokichla gurneyi')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Orange Ground Thrush</h3>
                        <p class="afrikaans-name">Oranjelyster</p>
                        <p class="scientific">Geokichla gurneyi</p>
                        <div class="card-meta">
                            <span class="meta-tag">Thrushes</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Scleroptila gutturalis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/orange-river-francolin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Scleroptila gutturalis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Orange River Francolin</h3>
                        <p class="afrikaans-name">Kalaharipatrys</p>
                        <p class="scientific">Scleroptila gutturalis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Francolins</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Zosterops pallidus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/orange-river-white-eye/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Zosterops pallidus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Orange River White-eye</h3>
                        <p class="afrikaans-name">Gariepglasogie</p>
                        <p class="scientific">Zosterops pallidus</p>
                        <div class="card-meta">
                            <span class="meta-tag">White-eyes</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Chlorophoneus sulfureopectus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/orange-breasted-bushshrike/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Chlorophoneus sulfureopectus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Orange-breasted Bushshrike</h3>
                        <p class="afrikaans-name">Oranjeborsboslaksman</p>
                        <p class="scientific">Chlorophoneus sulfureopectus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bush-shrikes</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anthobaphes violacea"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/orange-breasted-sunbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anthobaphes violacea')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #6f42c1"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Orange-breasted Sunbird</h3>
                        <p class="afrikaans-name">Oranjeborssuikerbekkie</p>
                        <p class="scientific">Anthobaphes violacea</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sunbirds</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Amandava subflava">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Amandava subflava')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Orange-breasted Waxbill</h3>
                        <p class="afrikaans-name">Rooiassie</p>
                        <p class="scientific">Amandava subflava</p>
                        <div class="card-meta">
                            <span class="meta-tag">Avadavats</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pytilia afra">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pytilia afra')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Orange-winged Pytilia</h3>
                        <p class="afrikaans-name">Oranjevlerkmelba</p>
                        <p class="scientific">Pytilia afra</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pytilias</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pandion haliaetus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pandion haliaetus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Osprey</h3>
                        <p class="afrikaans-name">Visvalk</p>
                        <p class="scientific">Pandion haliaetus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ospreys</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Accipiter ovampensis">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Accipiter ovampensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Ovambo Sparrowhawk</h3>
                        <p class="afrikaans-name">Ovambosperwer</p>
                        <p class="scientific">Accipiter ovampensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Hawks</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pluvialis fulva">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pluvialis fulva')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pacific Golden Plover</h3>
                        <p class="afrikaans-name">Asiatiese Goue Strandkiewiet</p>
                        <p class="scientific">Pluvialis fulva</p>
                        <div class="card-meta">
                            <span class="meta-tag">Plovers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Melierax canorus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Melierax canorus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #e83e8c"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pale Chanting Goshawk</h3>
                        <p class="afrikaans-name">Bleeksingvalk</p>
                        <p class="scientific">Melierax canorus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Goshawks</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Melaenornis pallidus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/pale-flycatcher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Melaenornis pallidus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pale Flycatcher</h3>
                        <p class="afrikaans-name">Muiskleurvlieëvanger</p>
                        <p class="scientific">Melaenornis pallidus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Flycatchers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Savanna</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cisticola cinnamomeus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/pale-crowned-cisticola/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cisticola cinnamomeus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pale-crowned Cisticola</h3>
                        <p class="afrikaans-name">Bleekkopklopkloppie</p>
                        <p class="scientific">Cisticola cinnamomeus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cisticolas</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Onychognathus nabouroup"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/pale-winged-starling/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Onychognathus nabouroup')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pale-winged Starling</h3>
                        <p class="afrikaans-name">Bleekvlerkspreeu</p>
                        <p class="scientific">Onychognathus nabouroup</p>
                        <div class="card-meta">
                            <span class="meta-tag">Starlings</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Rocky</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Circus macrourus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/pallid-harrier/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Circus macrourus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pallid Harrier</h3>
                        <p class="afrikaans-name">Witborsvleivalk (-paddavreter)</p>
                        <p class="scientific">Circus macrourus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Harriers</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Gypohierax angolensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/palm-nut-vulture/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Gypohierax angolensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Palm-nut Vulture</h3>
                        <p class="afrikaans-name">Witaasvoël</p>
                        <p class="scientific">Gypohierax angolensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Vultures</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Stercorarius parasiticus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/parasitic-jaeger/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Stercorarius parasiticus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Parasitic Jaeger</h3>
                        <p class="afrikaans-name">Arktiese Roofmeeu</p>
                        <p class="scientific">Stercorarius parasiticus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Skuas</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Hirundo dimidiata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/pearl-breasted-swallow/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Hirundo dimidiata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pearl-breasted Swallow</h3>
                        <p class="afrikaans-name">Pêrelborsswael</p>
                        <p class="scientific">Hirundo dimidiata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Swallows</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Glaucidium perlatum"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/pearl-spotted-owlet/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Glaucidium perlatum')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pearl-spotted Owlet</h3>
                        <p class="afrikaans-name">Witkoluil</p>
                        <p class="scientific">Glaucidium perlatum</p>
                        <div class="card-meta">
                            <span class="meta-tag">Owls</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calidris melanotos"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/pectoral-sandpiper/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calidris melanotos')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pectoral Sandpiper</h3>
                        <p class="afrikaans-name">Geelpootstrandloper</p>
                        <p class="scientific">Calidris melanotos</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Scotopelia peli"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/pels-fishing-owl/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Scotopelia peli')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pel's Fishing Owl</h3>
                        <p class="afrikaans-name">Visuil</p>
                        <p class="scientific">Scotopelia peli</p>
                        <div class="card-meta">
                            <span class="meta-tag">Owls</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Caprimulgus vexillarius"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/pennant-winged-nightjar/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Caprimulgus vexillarius')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pennant-winged Nightjar</h3>
                        <p class="afrikaans-name">Wimpelvlerknaguil</p>
                        <p class="scientific">Caprimulgus vexillarius</p>
                        <div class="card-meta">
                            <span class="meta-tag">Nightjars</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Falco peregrinus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/peregrine-falcon/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Falco peregrinus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Peregrine Falcon</h3>
                        <p class="afrikaans-name">Swerfvalk</p>
                        <p class="scientific">Falco peregrinus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Falcons</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Recurvirostra avosetta"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/pied-avocet/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Recurvirostra avosetta')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pied Avocet</h3>
                        <p class="afrikaans-name">Bontelsie</p>
                        <p class="scientific">Recurvirostra avosetta</p>
                        <div class="card-meta">
                            <span class="meta-tag">Avocets</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Corvus albus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/pied-crow/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Corvus albus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pied Crow</h3>
                        <p class="afrikaans-name">Witborskraai</p>
                        <p class="scientific">Corvus albus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Crows</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ceryle rudis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/pied-kingfisher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ceryle rudis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pied Kingfisher</h3>
                        <p class="afrikaans-name">Bontvisvanger</p>
                        <p class="scientific">Ceryle rudis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Kingfishers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Lamprotornis bicolor"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/pied-starling/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Lamprotornis bicolor')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pied Starling</h3>
                        <p class="afrikaans-name">Witgatspreeu</p>
                        <p class="scientific">Lamprotornis bicolor</p>
                        <div class="card-meta">
                            <span class="meta-tag">Starlings</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Oenanthe pleschanka"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/pied-wheatear/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Oenanthe pleschanka')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pied Wheatear</h3>
                        <p class="afrikaans-name">Bontskaapwagter</p>
                        <p class="scientific">Oenanthe pleschanka</p>
                        <div class="card-meta">
                            <span class="meta-tag">Wheatears</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Rocky</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Vidua macroura"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/pin-tailed-whydah/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Vidua macroura')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pin-tailed Whydah</h3>
                        <p class="afrikaans-name">Koningrooibekkie</p>
                        <p class="scientific">Vidua macroura</p>
                        <div class="card-meta">
                            <span class="meta-tag">Whydahs</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Oceanites pincoyae"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/pincoya-storm-petrel/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Oceanites pincoyae')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pincoya Storm Petrel</h3>
                        <p class="afrikaans-name">Pincoyastormswael</p>
                        <p class="scientific">Oceanites pincoyae</p>
                        <div class="card-meta">
                            <span class="meta-tag">Storm-Petrels</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pelecanus rufescens"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/pink-backed-pelican/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pelecanus rufescens')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #e83e8c"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pink-backed Pelican</h3>
                        <p class="afrikaans-name">Kleinpelikaan</p>
                        <p class="scientific">Pelecanus rufescens</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pelicans</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Spizocorys conirostris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/pink-billed-lark/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Spizocorys conirostris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #e83e8c"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pink-billed Lark</h3>
                        <p class="afrikaans-name">Pienkbeklewerik</p>
                        <p class="scientific">Spizocorys conirostris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Hypargos margaritatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/pink-throated-twinspot/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Hypargos margaritatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #e83e8c"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pink-throated Twinspot</h3>
                        <p class="afrikaans-name">Rooskeelkolpensie</p>
                        <p class="scientific">Hypargos margaritatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Twinspots</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Daption capense"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/pintado-petrel/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Daption capense')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pintado Petrel</h3>
                        <p class="afrikaans-name">Seeduifstormvoël</p>
                        <p class="scientific">Daption capense</p>
                        <div class="card-meta">
                            <span class="meta-tag">Petrels</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anthus leucophrys"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/plain-backed-pipit/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anthus leucophrys')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Plain-backed Pipit</h3>
                        <p class="afrikaans-name">Donkerkoester</p>
                        <p class="scientific">Anthus leucophrys</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pipits</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anthreptes reichenowi"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/plain-backed-sunbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anthreptes reichenowi')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #6f42c1"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: linear-gradient(135deg, #ff006e, #8338ec, #3a86ff)"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Plain-backed Sunbird</h3>
                        <p class="afrikaans-name">Bloukeelsuikerbekkie</p>
                        <p class="scientific">Anthreptes reichenowi</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sunbirds</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Stercorarius pomarinus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/pomarine-jaeger/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Stercorarius pomarinus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pomarine Jaeger</h3>
                        <p class="afrikaans-name">Knopstertroofmeeu</p>
                        <p class="scientific">Stercorarius pomarinus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Skuas</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Batis pririt"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/pririt-batis/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Batis pririt')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pririt Batis</h3>
                        <p class="afrikaans-name">Priritbosbontrokkie</p>
                        <p class="scientific">Batis pririt</p>
                        <div class="card-meta">
                            <span class="meta-tag">Batises</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Crithagra leucoptera"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/protea-canary/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Crithagra leucoptera')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Protea Canary</h3>
                        <p class="afrikaans-name">Proteakanarie</p>
                        <p class="scientific">Crithagra leucoptera</p>
                        <div class="card-meta">
                            <span class="meta-tag">Canaries</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ardea purpurea"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/purple-heron/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ardea purpurea')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Purple Heron</h3>
                        <p class="afrikaans-name">Rooireier</p>
                        <p class="scientific">Ardea purpurea</p>
                        <div class="card-meta">
                            <span class="meta-tag">Herons</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Vidua purpurascens"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/purple-indigobird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Vidua purpurascens')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #6f42c1"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Purple Indigobird</h3>
                        <p class="afrikaans-name">Witpootblouvinkie</p>
                        <p class="scientific">Vidua purpurascens</p>
                        <div class="card-meta">
                            <span class="meta-tag">Whydahs</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Coracias naevius"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/purple-roller/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Coracias naevius')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #6f42c1"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Purple Roller</h3>
                        <p class="afrikaans-name">Groottroupant</p>
                        <p class="scientific">Coracias naevius</p>
                        <div class="card-meta">
                            <span class="meta-tag">Rollers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cinnyris bifasciatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/purple-banded-sunbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cinnyris bifasciatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #6f42c1"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Purple-banded Sunbird</h3>
                        <p class="afrikaans-name">Purperbandsuikerbekkie</p>
                        <p class="scientific">Cinnyris bifasciatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sunbirds</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Gallirex porphyreolophus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/purple-crested-turaco/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Gallirex porphyreolophus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #6f42c1"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Purple-crested Turaco</h3>
                        <p class="afrikaans-name">Bloukuifloerie</p>
                        <p class="scientific">Gallirex porphyreolophus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Turacos</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Polihierax semitorquatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/pygmy-falcon/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Polihierax semitorquatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Pygmy Falcon</h3>
                        <p class="afrikaans-name">Dwergvalk</p>
                        <p class="scientific">Polihierax semitorquatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Falcons</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ortygospiza atricollis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/quailfinch/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ortygospiza atricollis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Quailfinch</h3>
                        <p class="afrikaans-name">Gewone Kwartelvinkie</p>
                        <p class="scientific">Ortygospiza atricollis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Quailfinches</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Coracias spatulatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/racket-tailed-roller/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Coracias spatulatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #6f42c1"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Racket-tailed Roller</h3>
                        <p class="afrikaans-name">Knopsterttroupant</p>
                        <p class="scientific">Coracias spatulatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Rollers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cisticola chiniana">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cisticola chiniana')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Rattling Cisticola</h3>
                        <p class="afrikaans-name">Bosveldtinktinkie</p>
                        <p class="scientific">Cisticola chiniana</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cisticolas</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calidris canutus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calidris canutus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red Knot</h3>
                        <p class="afrikaans-name">Knoet</p>
                        <p class="scientific">Calidris canutus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calendulauda burra">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calendulauda burra')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red Lark</h3>
                        <p class="afrikaans-name">Rooilewerik</p>
                        <p class="scientific">Calendulauda burra</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Phalaropus fulicarius">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Phalaropus fulicarius')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red Phalarope</h3>
                        <p class="afrikaans-name">Grysfraiingpoot</p>
                        <p class="scientific">Phalaropus fulicarius</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Spermestes bicolor">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Spermestes bicolor')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-backed Mannikin</h3>
                        <p class="afrikaans-name">Rooirugfret</p>
                        <p class="scientific">Spermestes bicolor</p>
                        <div class="card-meta">
                            <span class="meta-tag">Mannikins</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Lanius collurio">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Lanius collurio')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-backed Shrike</h3>
                        <p class="afrikaans-name">Rooiruglaksman</p>
                        <p class="scientific">Lanius collurio</p>
                        <div class="card-meta">
                            <span class="meta-tag">Shrikes</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Bubalornis niger">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Bubalornis niger')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-billed Buffalo Weaver</h3>
                        <p class="afrikaans-name">Buffelwewer</p>
                        <p class="scientific">Bubalornis niger</p>
                        <div class="card-meta">
                            <span class="meta-tag">Buffalo-Weavers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Lagonosticta senegala">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Lagonosticta senegala')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #e83e8c"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-billed Firefinch</h3>
                        <p class="afrikaans-name">Rooibekvuurvinkie</p>
                        <p class="scientific">Lagonosticta senegala</p>
                        <div class="card-meta">
                            <span class="meta-tag">Firefinches</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Buphagus erythrorynchus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Buphagus erythrorynchus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-billed Oxpecker</h3>
                        <p class="afrikaans-name">Rooibekrenostervoël</p>
                        <p class="scientific">Buphagus erythrorynchus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Oxpeckers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Quelea quelea">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Quelea quelea')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-billed Quelea</h3>
                        <p class="afrikaans-name">Rooibekkwelea</p>
                        <p class="scientific">Quelea quelea</p>
                        <div class="card-meta">
                            <span class="meta-tag">Queleas</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pternistis adspersus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-billed-spurfowl/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pternistis adspersus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-billed Spurfowl</h3>
                        <p class="afrikaans-name">Rooibekfisant</p>
                        <p class="scientific">Pternistis adspersus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Francolins</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anas erythrorhyncha"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-billed-teal/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anas erythrorhyncha')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-billed Teal</h3>
                        <p class="afrikaans-name">Rooibekeend</p>
                        <p class="scientific">Anas erythrorhyncha</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ducks</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Phaethon aethereus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-billed-tropicbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Phaethon aethereus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-billed Tropicbird</h3>
                        <p class="afrikaans-name">Rooibekpylstert</p>
                        <p class="scientific">Phaethon aethereus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Tropicbirds</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cecropis semirufa"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-breasted-swallow/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cecropis semirufa')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-breasted Swallow</h3>
                        <p class="afrikaans-name">Rooiborsswael</p>
                        <p class="scientific">Cecropis semirufa</p>
                        <div class="card-meta">
                            <span class="meta-tag">Swallows</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calandrella cinerea"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-capped-lark/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calandrella cinerea')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-capped Lark</h3>
                        <p class="afrikaans-name">Rooikoplewerik</p>
                        <p class="scientific">Calandrella cinerea</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cossypha natalensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-capped-robin-chat/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cossypha natalensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-capped Robin-chat</h3>
                        <p class="afrikaans-name">Nataljanfrederik</p>
                        <p class="scientific">Cossypha natalensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Robin-Chats</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cuculus solitarius"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-chested-cuckoo/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cuculus solitarius')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-chested Cuckoo</h3>
                        <p class="afrikaans-name">Piet-my-vrou</p>
                        <p class="scientific">Cuculus solitarius</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cuckoos</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sarothrura rufa"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-chested-flufftail/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sarothrura rufa')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-chested Flufftail</h3>
                        <p class="afrikaans-name">Rooiborsvleikuiken</p>
                        <p class="scientific">Sarothrura rufa</p>
                        <div class="card-meta">
                            <span class="meta-tag">Flufftails</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Euplectes ardens"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-collared-widowbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Euplectes ardens')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-collared Widowbird</h3>
                        <p class="afrikaans-name">Rooikeelflap</p>
                        <p class="scientific">Euplectes ardens</p>
                        <div class="card-meta">
                            <span class="meta-tag">Widowbirds</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Lophotis ruficrista"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-crested-korhaan/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Lophotis ruficrista')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-crested Korhaan</h3>
                        <p class="afrikaans-name">Boskorhaan</p>
                        <p class="scientific">Lophotis ruficrista</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bustards</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Streptopelia semitorquata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-eyed-dove/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Streptopelia semitorquata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #e83e8c"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-eyed Dove</h3>
                        <p class="afrikaans-name">Grootringduif</p>
                        <p class="scientific">Streptopelia semitorquata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Doves</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cisticola erythrops"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-faced-cisticola/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cisticola erythrops')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-faced Cisticola</h3>
                        <p class="afrikaans-name">Rooiwangtinktinkie</p>
                        <p class="scientific">Cisticola erythrops</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cisticolas</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Urocolius indicus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-faced-mousebird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Urocolius indicus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-faced Mousebird</h3>
                        <p class="afrikaans-name">Rooiwangmuisvoël</p>
                        <p class="scientific">Urocolius indicus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Mousebirds</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sula sula"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-footed-booby/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sula sula')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-footed Booby</h3>
                        <p class="afrikaans-name">Rooipootmalgas</p>
                        <p class="scientific">Sula sula</p>
                        <div class="card-meta">
                            <span class="meta-tag">Boobies</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Falco vespertinus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-footed-falcon/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Falco vespertinus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-footed Falcon</h3>
                        <p class="afrikaans-name">Westelike Rooipootvalk</p>
                        <p class="scientific">Falco vespertinus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Falcons</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pogoniulus pusillus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-fronted-tinkerbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pogoniulus pusillus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-fronted Tinkerbird</h3>
                        <p class="afrikaans-name">Rooiblestinker</p>
                        <p class="scientific">Pogoniulus pusillus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Tinkerbirds</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Amadina erythrocephala"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-headed-finch/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Amadina erythrocephala')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-headed Finch</h3>
                        <p class="afrikaans-name">Rooikopvink</p>
                        <p class="scientific">Amadina erythrocephala</p>
                        <div class="card-meta">
                            <span class="meta-tag">Finches</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Quelea erythrops"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-headed-quelea/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Quelea erythrops')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-headed Quelea</h3>
                        <p class="afrikaans-name">Rooikopkwelea</p>
                        <p class="scientific">Quelea erythrops</p>
                        <div class="card-meta">
                            <span class="meta-tag">Queleas</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anaplectes rubriceps"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-headed-weaver/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anaplectes rubriceps')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-headed Weaver</h3>
                        <p class="afrikaans-name">Rooikopwewer</p>
                        <p class="scientific">Anaplectes rubriceps</p>
                        <div class="card-meta">
                            <span class="meta-tag">Weavers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Fulica cristata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-knobbed-coot/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Fulica cristata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-knobbed Coot</h3>
                        <p class="afrikaans-name">Bleshoender</p>
                        <p class="scientific">Fulica cristata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Coots</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Buteo auguralis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-necked-buzzard/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Buteo auguralis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-necked Buzzard</h3>
                        <p class="afrikaans-name">Rooinekajakkalvoël</p>
                        <p class="scientific">Buteo auguralis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Buzzards</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Falco chicquera"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-necked-falcon/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Falco chicquera')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-necked Falcon</h3>
                        <p class="afrikaans-name">Rooinekvalk</p>
                        <p class="scientific">Falco chicquera</p>
                        <div class="card-meta">
                            <span class="meta-tag">Falcons</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Phalaropus lobatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-necked-phalarope/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Phalaropus lobatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-necked Phalarope</h3>
                        <p class="afrikaans-name">Rooihalsfraiingpoot</p>
                        <p class="scientific">Phalaropus lobatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pternistis afer"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-necked-spurfowl/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pternistis afer')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-necked Spurfowl</h3>
                        <p class="afrikaans-name">Rooikeelfisant</p>
                        <p class="scientific">Pternistis afer</p>
                        <div class="card-meta">
                            <span class="meta-tag">Francolins</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calidris ruficollis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-necked-stint/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calidris ruficollis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-necked Stint</h3>
                        <p class="afrikaans-name">Rooinekstrandloper</p>
                        <p class="scientific">Calidris ruficollis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cecropis daurica"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-rumped-swallow/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cecropis daurica')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-rumped Swallow</h3>
                        <p class="afrikaans-name">Rooinekswael</p>
                        <p class="scientific">Cecropis daurica</p>
                        <div class="card-meta">
                            <span class="meta-tag">Swallows</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Lanius phoenicuroides"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-tailed-shrike/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Lanius phoenicuroides')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-tailed Shrike</h3>
                        <p class="afrikaans-name">Rooistertklauwier</p>
                        <p class="scientific">Lanius phoenicuroides</p>
                        <div class="card-meta">
                            <span class="meta-tag">Shrikes</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Phaethon rubricauda"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-tailed-tropicbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Phaethon rubricauda')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-tailed Tropicbird</h3>
                        <p class="afrikaans-name">Rooipylstert</p>
                        <p class="scientific">Phaethon rubricauda</p>
                        <div class="card-meta">
                            <span class="meta-tag">Tropicbirds</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anthus cervinus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-throated-pipit/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anthus cervinus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-throated Pipit</h3>
                        <p class="afrikaans-name">Rooikeelkoester</p>
                        <p class="scientific">Anthus cervinus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pipits</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Hypargos niveoguttatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-throated-twinspot/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Hypargos niveoguttatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-throated Twinspot</h3>
                        <p class="afrikaans-name">Rooikeelkolpensie</p>
                        <p class="scientific">Hypargos niveoguttatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Twinspots</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Jynx ruficollis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-throated-wryneck/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Jynx ruficollis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-throated Wryneck</h3>
                        <p class="afrikaans-name">Draaihals</p>
                        <p class="scientific">Jynx ruficollis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Wrynecks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Scleroptila levaillantii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-winged-francolin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Scleroptila levaillantii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-winged Francolin</h3>
                        <p class="afrikaans-name">Rooivlerkpatrys</p>
                        <p class="scientific">Scleroptila levaillantii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Francolins</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Onychognathus morio"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/red-winged-starling/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Onychognathus morio')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Red-winged Starling</h3>
                        <p class="afrikaans-name">Rooivlerkspreeu</p>
                        <p class="scientific">Onychognathus morio</p>
                        <div class="card-meta">
                            <span class="meta-tag">Starlings</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Rocky</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Microcarbo africanus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/reed-cormorant/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Microcarbo africanus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Reed Cormorant</h3>
                        <p class="afrikaans-name">Rietduiker</p>
                        <p class="scientific">Microcarbo africanus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cormorants</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Prionops retzii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/retzs-helmet-shrike/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Prionops retzii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Retz's Helmet-shrike</h3>
                        <p class="afrikaans-name">Swarthelmlaksman</p>
                        <p class="scientific">Prionops retzii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Helmet-shrikes</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Locustella fluviatilis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/river-warbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Locustella fluviatilis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>River Warbler</h3>
                        <p class="afrikaans-name">Sprinkaansanger</p>
                        <p class="scientific">Locustella fluviatilis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Grassbirds</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Columba livia"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/rock-dove/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Columba livia')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #6f42c1"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Rock Dove</h3>
                        <p class="afrikaans-name">Tuinduif</p>
                        <p class="scientific">Columba livia</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pigeons</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Gardens</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Falco rupicolus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/rock-kestrel/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Falco rupicolus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Rock Kestrel</h3>
                        <p class="afrikaans-name">Kransvalk</p>
                        <p class="scientific">Falco rupicolus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Falcons</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Savanna</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ptyonoprogne fuligula"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/rock-martin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ptyonoprogne fuligula')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Rock Martin</h3>
                        <p class="afrikaans-name">Kransswael</p>
                        <p class="scientific">Ptyonoprogne fuligula</p>
                        <div class="card-meta">
                            <span class="meta-tag">Martins</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Rocky</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Glareola nuchalis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/rock-pratincole/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Glareola nuchalis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #FFFDD0"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Rock Pratincole</h3>
                        <p class="afrikaans-name">Withalssprinkaanvoël</p>
                        <p class="scientific">Glareola nuchalis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pratincoles</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Rocky</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Psittacula krameri"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/rose-ringed-parakeet/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Psittacula krameri')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #007bff"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Rose-ringed Parakeet</h3>
                        <p class="afrikaans-name">Ringnekparkiet</p>
                        <p class="scientific">Psittacula krameri</p>
                        <div class="card-meta">
                            <span class="meta-tag">Parakeets</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sterna dougallii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/roseate-tern/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sterna dougallii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #e83e8c"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Roseate Tern</h3>
                        <p class="afrikaans-name">Rooiborssterretjie</p>
                        <p class="scientific">Sterna dougallii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Terns</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pastor roseus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pastor roseus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #e83e8c"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: linear-gradient(135deg, #ff006e, #8338ec, #3a86ff)"></div><div class="card-color-dot" style="background: #007bff"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Rosy Starling</h3>
                        <p class="afrikaans-name">Rooiborspreeu</p>
                        <p class="scientific">Pastor roseus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Starlings</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Agapornis roseicollis">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Agapornis roseicollis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #e83e8c"></div><div class="card-color-dot" style="background: #007bff"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Rosy-faced Lovebird</h3>
                        <p class="afrikaans-name">Rooiwangparkiet</p>
                        <p class="scientific">Agapornis roseicollis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Lovebirds</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Macronyx ameliae">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Macronyx ameliae')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #e83e8c"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Rosy-throated Longclaw</h3>
                        <p class="afrikaans-name">Rooskeelkalkoentjie</p>
                        <p class="scientific">Macronyx ameliae</p>
                        <div class="card-meta">
                            <span class="meta-tag">Longclaws</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Apalis ruddi">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Apalis ruddi')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Rudd's Apalis</h3>
                        <p class="afrikaans-name">Ruddse Kleinjantjie</p>
                        <p class="scientific">Apalis ruddi</p>
                        <div class="card-meta">
                            <span class="meta-tag">Apalises</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Heteromirafra ruddi">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Heteromirafra ruddi')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Rudd's Lark</h3>
                        <p class="afrikaans-name">Drakensberglewerik</p>
                        <p class="scientific">Heteromirafra ruddi</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Arenaria interpres">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Arenaria interpres')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Ruddy Turnstone</h3>
                        <p class="afrikaans-name">Steenloper</p>
                        <p class="scientific">Arenaria interpres</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calidris pugnax">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calidris pugnax')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Ruff</h3>
                        <p class="afrikaans-name">Kemphaan</p>
                        <p class="scientific">Calidris pugnax</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ardeola rufiventris">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ardeola rufiventris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Rufous-bellied Heron</h3>
                        <p class="afrikaans-name">Rooipensreier</p>
                        <p class="scientific">Ardeola rufiventris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Herons</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Accipiter rufiventris">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Accipiter rufiventris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Rufous-breasted Sparrowhawk</h3>
                        <p class="afrikaans-name">Rooiborssperwer</p>
                        <p class="scientific">Accipiter rufiventris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Hawks</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Caprimulgus rufigena">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Caprimulgus rufigena')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Rufous-cheeked Nightjar</h3>
                        <p class="afrikaans-name">Rooiwangnaguil</p>
                        <p class="scientific">Caprimulgus rufigena</p>
                        <div class="card-meta">
                            <span class="meta-tag">Nightjars</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Malcorus pectoralis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/rufous-eared-warbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Malcorus pectoralis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Rufous-eared Warbler</h3>
                        <p class="afrikaans-name">Rooioorlangstertjie</p>
                        <p class="scientific">Malcorus pectoralis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Warblers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Mirafra africana"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/rufous-naped-lark/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Mirafra africana')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Rufous-naped Lark</h3>
                        <p class="afrikaans-name">Rooineklewerik</p>
                        <p class="scientific">Mirafra africana</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cercotrichas galactotes"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/rufous-tailed-scrub-robin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cercotrichas galactotes')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Rufous-tailed Scrub Robin</h3>
                        <p class="afrikaans-name">Swartrugtinktinkie</p>
                        <p class="scientific">Cercotrichas galactotes</p>
                        <div class="card-meta">
                            <span class="meta-tag">Scrub-Robins</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cisticola galactotes"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/rufous-winged-cisticola/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cisticola galactotes')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Rufous-winged Cisticola</h3>
                        <p class="afrikaans-name">Swartrugtinktinkie</p>
                        <p class="scientific">Cisticola galactotes</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cisticolas</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Gyps rueppelli"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/rüppells-vulture/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Gyps rueppelli')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Rüppell's Vulture</h3>
                        <p class="afrikaans-name">Rüppellse Aasvoël</p>
                        <p class="scientific">Gyps rueppelli</p>
                        <div class="card-meta">
                            <span class="meta-tag">Vultures</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Xema sabini"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/sabines-gull/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Xema sabini')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Sabine's Gull</h3>
                        <p class="afrikaans-name">Mikstertmeeu</p>
                        <p class="scientific">Xema sabini</p>
                        <div class="card-meta">
                            <span class="meta-tag">Gulls</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calendulauda sabota">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calendulauda sabota')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Sabota Lark</h3>
                        <p class="afrikaans-name">Sabotalewerik</p>
                        <p class="scientific">Calendulauda sabota</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ephippiorhynchus senegalensis">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ephippiorhynchus senegalensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Saddle-billed Stork</h3>
                        <p class="afrikaans-name">Saalbekooievaar</p>
                        <p class="scientific">Ephippiorhynchus senegalensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Storks</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Thalassarche salvini">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Thalassarche salvini')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Salvin's Albatross</h3>
                        <p class="afrikaans-name">Salvinalbatros (-malmok)</p>
                        <p class="scientific">Thalassarche salvini</p>
                        <div class="card-meta">
                            <span class="meta-tag">Albatrosses</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pachyptila salvini">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pachyptila salvini')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Salvin's Prion</h3>
                        <p class="afrikaans-name">Marionwalvisvoël</p>
                        <p class="scientific">Pachyptila salvini</p>
                        <div class="card-meta">
                            <span class="meta-tag">Prions</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Riparia riparia"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/sand-martin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Riparia riparia')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Sand Martin</h3>
                        <p class="afrikaans-name">Europese Oewerswael</p>
                        <p class="scientific">Riparia riparia</p>
                        <div class="card-meta">
                            <span class="meta-tag">Martins</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Rivers</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calidris alba"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/sanderling/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calidris alba')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Sanderling</h3>
                        <p class="afrikaans-name">Drietoonstrandloper</p>
                        <p class="scientific">Calidris alba</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Thalasseus sandvicensis">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Thalasseus sandvicensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Sandwich Tern</h3>
                        <p class="afrikaans-name">Grootsterretjie</p>
                        <p class="scientific">Thalasseus sandvicensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Terns</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sporopipes squamifrons">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sporopipes squamifrons')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #e83e8c"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Scaly-feathered Finch</h3>
                        <p class="afrikaans-name">Baardmannetjie</p>
                        <p class="scientific">Sporopipes squamifrons</p>
                        <div class="card-meta">
                            <span class="meta-tag">Weavers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Indicator variegatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/scaly-throated-honeyguide/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Indicator variegatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Scaly-throated Honeyguide</h3>
                        <p class="afrikaans-name">Gevlekte Heuningwyser</p>
                        <p class="scientific">Indicator variegatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Honeyguides</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Chalcomitra senegalensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/scarlet-chested-sunbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Chalcomitra senegalensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Scarlet-chested Sunbird</h3>
                        <p class="afrikaans-name">Rooiborssuikerbekkie</p>
                        <p class="scientific">Chalcomitra senegalensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sunbirds</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Spizocorys sclateri"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/sclaters-lark/2_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Spizocorys sclateri')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Sclater's Lark</h3>
                        <p class="afrikaans-name">Namakwalewerik</p>
                        <p class="scientific">Spizocorys sclateri</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Karoo</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sagittarius serpentarius"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/secretarybird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sagittarius serpentarius')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Secretarybird</h3>
                        <p class="afrikaans-name">Sekretarisvoël</p>
                        <p class="scientific">Sagittarius serpentarius</p>
                        <div class="card-meta">
                            <span class="meta-tag">Secretarybird</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Acrocephalus schoenobaenus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/sedge-warbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Acrocephalus schoenobaenus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Sedge Warbler</h3>
                        <p class="afrikaans-name">Europese Vleisanger</p>
                        <p class="scientific">Acrocephalus schoenobaenus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Reed-Warblers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Centropus senegalensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/senegal-coucal/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Centropus senegalensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Senegal Coucal</h3>
                        <p class="afrikaans-name">Senegalvleiloerie</p>
                        <p class="scientific">Centropus senegalensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Coucals</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Vanellus lugubris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/senegal-lapwing/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Vanellus lugubris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Senegal Lapwing</h3>
                        <p class="afrikaans-name">Kleinswartvlerkkiewiet</p>
                        <p class="scientific">Vanellus lugubris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Lapwings</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Monticola explorator"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/sentinel-rock-thrush/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Monticola explorator')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Sentinel Rock Thrush</h3>
                        <p class="afrikaans-name">Langtoonkliplyster</p>
                        <p class="scientific">Monticola explorator</p>
                        <div class="card-meta">
                            <span class="meta-tag">Thrushes</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Vidua regia"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/shaft-tailed-whydah/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Vidua regia')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Shaft-tailed Whydah</h3>
                        <p class="afrikaans-name">Pylstertrooibekkie</p>
                        <p class="scientific">Vidua regia</p>
                        <div class="card-meta">
                            <span class="meta-tag">Whydahs</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Scleroptila shelleyi"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/shelleys-francolin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Scleroptila shelleyi')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Shelley's Francolin</h3>
                        <p class="afrikaans-name">Laeveldpatrys</p>
                        <p class="scientific">Scleroptila shelleyi</p>
                        <div class="card-meta">
                            <span class="meta-tag">Francolins</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Accipiter badius"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/shikra/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Accipiter badius')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Shikra</h3>
                        <p class="afrikaans-name">Gebande Sperwer</p>
                        <p class="scientific">Accipiter badius</p>
                        <div class="card-meta">
                            <span class="meta-tag">Hawks</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Certhilauda chuana"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/short-clawed-lark/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Certhilauda chuana')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Short-clawed Lark</h3>
                        <p class="afrikaans-name">Kortkloulewerik</p>
                        <p class="scientific">Certhilauda chuana</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anthus brachyurus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/short-tailed-pipit/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anthus brachyurus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Short-tailed Pipit</h3>
                        <p class="afrikaans-name">Kortstertkoester</p>
                        <p class="scientific">Anthus brachyurus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pipits</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Monticola brevipes"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/short-toed-rock-thrush/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Monticola brevipes')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Short-toed Rock Thrush</h3>
                        <p class="afrikaans-name">Korttoonkliplyster</p>
                        <p class="scientific">Monticola brevipes</p>
                        <div class="card-meta">
                            <span class="meta-tag">Thrushes</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Thalassarche cauta"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/shy-albatross/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Thalassarche cauta')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Shy Albatross</h3>
                        <p class="afrikaans-name">Bloubekalbatros (-malmok)</p>
                        <p class="scientific">Thalassarche cauta</p>
                        <div class="card-meta">
                            <span class="meta-tag">Albatrosses</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Emarginata sinuata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/sickle-winged-chat/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Emarginata sinuata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Sickle-winged Chat</h3>
                        <p class="afrikaans-name">Vlaktespekvreter</p>
                        <p class="scientific">Emarginata sinuata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Chats</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Egretta vinaceigula"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/slaty-egret/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Egretta vinaceigula')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Slaty Egret</h3>
                        <p class="afrikaans-name">Rooikeelreier</p>
                        <p class="scientific">Egretta vinaceigula</p>
                        <div class="card-meta">
                            <span class="meta-tag">Herons</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Chroicocephalus genei"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/slender-billed-gull/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Chroicocephalus genei')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Slender-billed Gull</h3>
                        <p class="afrikaans-name">Dunbekmeeu</p>
                        <p class="scientific">Chroicocephalus genei</p>
                        <div class="card-meta">
                            <span class="meta-tag">Gulls</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pachyptila belcheri"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/slender-billed-prion/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pachyptila belcheri')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Slender-billed Prion</h3>
                        <p class="afrikaans-name">Dunbekwalvisvoël</p>
                        <p class="scientific">Pachyptila belcheri</p>
                        <div class="card-meta">
                            <span class="meta-tag">Prions</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Egretta thula"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/snowy-egret/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Egretta thula')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Snowy Egret</h3>
                        <p class="afrikaans-name">Sneeuwitreier</p>
                        <p class="scientific">Egretta thula</p>
                        <div class="card-meta">
                            <span class="meta-tag">Herons</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Chionis albus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/snowy-sheathbill/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Chionis albus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Snowy Sheathbill</h3>
                        <p class="afrikaans-name">Grootskedebek (-peddie)</p>
                        <p class="scientific">Chionis albus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sheathbills</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Philetairus socius"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/sociable-weaver/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Philetairus socius')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Sociable Weaver</h3>
                        <p class="afrikaans-name">Versamelvoël</p>
                        <p class="scientific">Philetairus socius</p>
                        <div class="card-meta">
                            <span class="meta-tag">Weavers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Karoo</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pterodroma mollis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/soft-plumaged-petrel/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pterodroma mollis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Soft-plumaged Petrel</h3>
                        <p class="afrikaans-name">Donsveerstormvoël</p>
                        <p class="scientific">Pterodroma mollis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Petrels</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Andropadus importunus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/sombre-greenbul/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Andropadus importunus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Sombre Greenbul</h3>
                        <p class="afrikaans-name">Gewone Willie</p>
                        <p class="scientific">Andropadus importunus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Greenbuls</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Phoebetria fusca"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/sooty-albatross/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Phoebetria fusca')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Sooty Albatross</h3>
                        <p class="afrikaans-name">Bruinalbatros (-malmok)</p>
                        <p class="scientific">Phoebetria fusca</p>
                        <div class="card-meta">
                            <span class="meta-tag">Albatrosses</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Falco concolor"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/sooty-falcon/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Falco concolor')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Sooty Falcon</h3>
                        <p class="afrikaans-name">Roetvalk</p>
                        <p class="scientific">Falco concolor</p>
                        <div class="card-meta">
                            <span class="meta-tag">Falcons</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Rocky</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ichthyaetus hemprichii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/sooty-gull/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ichthyaetus hemprichii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Sooty Gull</h3>
                        <p class="afrikaans-name">Roetmeeu</p>
                        <p class="scientific">Ichthyaetus hemprichii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Gulls</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ardenna grisea"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/sooty-shearwater/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ardenna grisea')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Sooty Shearwater</h3>
                        <p class="afrikaans-name">Malbaatjie</p>
                        <p class="scientific">Ardenna grisea</p>
                        <div class="card-meta">
                            <span class="meta-tag">Shearwaters</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Onychoprion fuscatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/sooty-tern/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Onychoprion fuscatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Sooty Tern</h3>
                        <p class="afrikaans-name">Roetsterretjie</p>
                        <p class="scientific">Onychoprion fuscatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Terns</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Petrochelidon spilodera"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/south-african-cliff-swallow/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Petrochelidon spilodera')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>South African Cliff Swallow</h3>
                        <p class="afrikaans-name">Familieswael</p>
                        <p class="scientific">Petrochelidon spilodera</p>
                        <div class="card-meta">
                            <span class="meta-tag">Swallows</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Tadorna cana"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/south-african-shelduck/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Tadorna cana')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>South African Shelduck</h3>
                        <p class="afrikaans-name">Kopereend</p>
                        <p class="scientific">Tadorna cana</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ducks</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Stercorarius maccormicki"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/south-polar-skua/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Stercorarius maccormicki')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>South Polar Skua</h3>
                        <p class="afrikaans-name">Suidpoolroofmeeu</p>
                        <p class="scientific">Stercorarius maccormicki</p>
                        <div class="card-meta">
                            <span class="meta-tag">Skuas</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Geronticus calvus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/southern-bald-ibis/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Geronticus calvus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Bald Ibis</h3>
                        <p class="afrikaans-name">Kalkoenibis</p>
                        <p class="scientific">Geronticus calvus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ibises</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Circaetus fasciolatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/southern-banded-snake-eagle/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Circaetus fasciolatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Banded Snake Eagle</h3>
                        <p class="afrikaans-name">Dubbelbandslangarend</p>
                        <p class="scientific">Circaetus fasciolatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Snake-Eagles</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Melaenornis pammelaina"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/southern-black-flycatcher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Melaenornis pammelaina')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Black Flycatcher</h3>
                        <p class="afrikaans-name">Swartvlieëvanger</p>
                        <p class="scientific">Melaenornis pammelaina</p>
                        <div class="card-meta">
                            <span class="meta-tag">Flycatchers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Afrotis afra"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/southern-black-korhaan/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Afrotis afra')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            <span class="badge badge-threatened">VU</span>
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Black Korhaan</h3>
                        <p class="afrikaans-name">Swartvlerkkorhaan</p>
                        <p class="scientific">Afrotis afra</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bustards</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Melaniparus niger">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Melaniparus niger')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Black Tit</h3>
                        <p class="afrikaans-name">Gewone Swartmees</p>
                        <p class="scientific">Melaniparus niger</p>
                        <div class="card-meta">
                            <span class="meta-tag">Tits</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Laniarius ferrugineus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Laniarius ferrugineus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #e83e8c"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Boubou</h3>
                        <p class="afrikaans-name">Suidelike Waterfiskaal</p>
                        <p class="scientific">Laniarius ferrugineus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Boubous</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ploceus xanthopterus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ploceus xanthopterus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Brown-throated Weaver</h3>
                        <p class="afrikaans-name">Bruinkeelwewer</p>
                        <p class="scientific">Ploceus xanthopterus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Weavers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Merops nubicoides">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Merops nubicoides')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Carmine Bee-eater</h3>
                        <p class="afrikaans-name">Rooiborsbyvreter</p>
                        <p class="scientific">Merops nubicoides</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bee-eaters</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cinnyris chalybeus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cinnyris chalybeus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #007bff"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Double-collared Sunbird</h3>
                        <p class="afrikaans-name">Klein-rooibandsuikerbekkie</p>
                        <p class="scientific">Cinnyris chalybeus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sunbirds</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Lanius collaris">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Lanius collaris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Fiscal</h3>
                        <p class="afrikaans-name">Fiskaallaksman</p>
                        <p class="scientific">Lanius collaris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Shrikes</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Fulmarus glacialoides">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Fulmarus glacialoides')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Fulmar</h3>
                        <p class="afrikaans-name">Silwerstormvoël</p>
                        <p class="scientific">Fulmarus glacialoides</p>
                        <div class="card-meta">
                            <span class="meta-tag">Petrels</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Macronectes giganteus">
                        <span class="placeholder">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Macronectes giganteus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Giant Petrel</h3>
                        <p class="afrikaans-name">Reusenellie</p>
                        <p class="scientific">Macronectes giganteus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Petrels</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Passer diffusus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/southern-grey-headed-sparrow/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Passer diffusus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Grey-headed Sparrow</h3>
                        <p class="afrikaans-name">Gryskopmossie</p>
                        <p class="scientific">Passer diffusus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sparrows</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Bucorvus leadbeateri"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/southern-ground-hornbill/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Bucorvus leadbeateri')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Ground- Hornbill</h3>
                        <p class="afrikaans-name">Bromvoël</p>
                        <p class="scientific">Bucorvus leadbeateri</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ground-Hornbills</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Hyliota australis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/southern-hyliota/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Hyliota australis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Hyliota</h3>
                        <p class="afrikaans-name">Mashonahyliota</p>
                        <p class="scientific">Hyliota australis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Hyliotas</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ploceus velatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/southern-masked-weaver/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ploceus velatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Masked Weaver</h3>
                        <p class="afrikaans-name">Swartkeelgeelvink</p>
                        <p class="scientific">Ploceus velatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Weavers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Turdoides bicolor"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/southern-pied-babbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Turdoides bicolor')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Pied Babbler</h3>
                        <p class="afrikaans-name">Witkatlagter</p>
                        <p class="scientific">Turdoides bicolor</p>
                        <div class="card-meta">
                            <span class="meta-tag">Babblers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Netta erythrophthalma"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/southern-pochard/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Netta erythrophthalma')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #007bff"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Pochard</h3>
                        <p class="afrikaans-name">Bruineend</p>
                        <p class="scientific">Netta erythrophthalma</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ducks</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Euplectes orix"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/southern-red-bishop/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Euplectes orix')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Red Bishop</h3>
                        <p class="afrikaans-name">Rooivink</p>
                        <p class="scientific">Euplectes orix</p>
                        <div class="card-meta">
                            <span class="meta-tag">Widowbirds</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Tockus rufirostris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/southern-red-billed-hornbill/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Tockus rufirostris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Red-billed Hornbill</h3>
                        <p class="afrikaans-name">Rooibekneushoringvoël</p>
                        <p class="scientific">Tockus rufirostris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Hornbills</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Savanna</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Eudyptes chrysocome"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/southern-rockhopper-penguin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Eudyptes chrysocome')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Rockhopper Penguin</h3>
                        <p class="afrikaans-name">Geelkuifpikkewyn</p>
                        <p class="scientific">Eudyptes chrysocome</p>
                        <div class="card-meta">
                            <span class="meta-tag">Penguins</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Diomedea epomophora"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/southern-royal-albatross/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Diomedea epomophora')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Royal Albatross</h3>
                        <p class="afrikaans-name">Witvlerkkoningalbatros (-malmok)</p>
                        <p class="scientific">Diomedea epomophora</p>
                        <div class="card-meta">
                            <span class="meta-tag">Albatrosses</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Tchagra tchagra"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/southern-tchagra/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Tchagra tchagra')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Tchagra</h3>
                        <p class="afrikaans-name">Grysborstjagra</p>
                        <p class="scientific">Tchagra tchagra</p>
                        <div class="card-meta">
                            <span class="meta-tag">Tchagras</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Eurocephalus anguitimens"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/southern-white-crowned-shrike/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Eurocephalus anguitimens')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern White-crowned Shrike</h3>
                        <p class="afrikaans-name">Kremetartlaksman</p>
                        <p class="scientific">Eurocephalus anguitimens</p>
                        <div class="card-meta">
                            <span class="meta-tag">Shrikes</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ptilopsis granti"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/southern-white-faced-owl/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ptilopsis granti')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern White-faced Owl</h3>
                        <p class="afrikaans-name">Witwanguil</p>
                        <p class="scientific">Ptilopsis granti</p>
                        <div class="card-meta">
                            <span class="meta-tag">Owls</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Zosterops anderssoni"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/southern-yellow-white-eye/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Zosterops anderssoni')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Yellow White-eye</h3>
                        <p class="afrikaans-name">Geel Glasogie</p>
                        <p class="scientific">Zosterops anderssoni</p>
                        <div class="card-meta">
                            <span class="meta-tag">White-eyes</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Tockus leucomelas"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/southern-yellow-billed-hornbill/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Tockus leucomelas')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Southern Yellow-billed Hornbill</h3>
                        <p class="afrikaans-name">Geelbekneushoringvoël</p>
                        <p class="scientific">Tockus leucomelas</p>
                        <div class="card-meta">
                            <span class="meta-tag">Hornbills</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Colius striatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/speckled-mousebird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Colius striatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Speckled Mousebird</h3>
                        <p class="afrikaans-name">Gevlekte Muisvoël</p>
                        <p class="scientific">Colius striatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Mousebirds</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Columba guinea"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/speckled-pigeon/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Columba guinea')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Speckled Pigeon</h3>
                        <p class="afrikaans-name">Kransduif</p>
                        <p class="scientific">Columba guinea</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pigeons</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Procellaria conspicillata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/spectacled-petrel/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Procellaria conspicillata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Spectacled Petrel</h3>
                        <p class="afrikaans-name">Brilbassiaan</p>
                        <p class="scientific">Procellaria conspicillata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Petrels</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ploceus ocularis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/spectacled-weaver/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ploceus ocularis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Spectacled Weaver</h3>
                        <p class="afrikaans-name">Brilwewer</p>
                        <p class="scientific">Ploceus ocularis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Weavers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Chersomanes albofasciata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/spike-heeled-lark/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Chersomanes albofasciata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Spike-heeled Lark</h3>
                        <p class="afrikaans-name">Vlaktelewerik</p>
                        <p class="scientific">Chersomanes albofasciata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Porzana porzana"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/spotted-crake/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Porzana porzana')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Spotted Crake</h3>
                        <p class="afrikaans-name">Gevlekte Riethaan</p>
                        <p class="scientific">Porzana porzana</p>
                        <div class="card-meta">
                            <span class="meta-tag">Crakes</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Bubo africanus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/spotted-eagle-owl/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Bubo africanus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Spotted Eagle- Owl</h3>
                        <p class="afrikaans-name">Gevlekte Ooruil</p>
                        <p class="scientific">Bubo africanus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Owls</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Rocky</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Muscicapa striata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/spotted-flycatcher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Muscicapa striata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Spotted Flycatcher</h3>
                        <p class="afrikaans-name">Europese Vlieëvanger</p>
                        <p class="scientific">Muscicapa striata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Flycatchers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Geokichla guttata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/spotted-ground-thrush/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Geokichla guttata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Spotted Ground Thrush</h3>
                        <p class="afrikaans-name">Natallyster</p>
                        <p class="scientific">Geokichla guttata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Thrushes</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Tringa erythropus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/spotted-redshank/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Tringa erythropus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Spotted Redshank</h3>
                        <p class="afrikaans-name">Gevlekte Rooipootruiter</p>
                        <p class="scientific">Tringa erythropus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Burhinus capensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/spotted-thick-knee/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Burhinus capensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Spotted Thick-knee</h3>
                        <p class="afrikaans-name">Gewone Dikkop</p>
                        <p class="scientific">Burhinus capensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Thick-knees</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Plectropterus gambensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/spur-winged-goose/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Plectropterus gambensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Spur-winged Goose</h3>
                        <p class="afrikaans-name">Wildemakou</p>
                        <p class="scientific">Plectropterus gambensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ducks</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Vanellus spinosus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/spur-winged-lapwing/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Vanellus spinosus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Spur-winged Lapwing</h3>
                        <p class="afrikaans-name">Spoorvlerkkiewiet</p>
                        <p class="scientific">Vanellus spinosus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Lapwings</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ardeola ralloides"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/squacco-heron/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ardeola ralloides')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Squacco Heron</h3>
                        <p class="afrikaans-name">Ralreier</p>
                        <p class="scientific">Ardeola ralloides</p>
                        <div class="card-meta">
                            <span class="meta-tag">Herons</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Dicrurus ludwigii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/square-tailed-drongo/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Dicrurus ludwigii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Square-tailed Drongo</h3>
                        <p class="afrikaans-name">Kleinbyvanger</p>
                        <p class="scientific">Dicrurus ludwigii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Drongos</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Caprimulgus fossii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/square-tailed-nightjar/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Caprimulgus fossii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Square-tailed Nightjar</h3>
                        <p class="afrikaans-name">Laeveldnaguil</p>
                        <p class="scientific">Caprimulgus fossii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Nightjars</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Spizocorys starki"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/starks-lark/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Spizocorys starki')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Stark's Lark</h3>
                        <p class="afrikaans-name">Woestynlewerik</p>
                        <p class="scientific">Spizocorys starki</p>
                        <div class="card-meta">
                            <span class="meta-tag">Larks</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Karoo</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Aquila nipalensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/steppe-eagle/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Aquila nipalensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Steppe Eagle</h3>
                        <p class="afrikaans-name">Steppe-arend</p>
                        <p class="scientific">Aquila nipalensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Eagles</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calamonastes stierlingi"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/stierlings-wren-warbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calamonastes stierlingi')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Stierling's Wren- Warbler</h3>
                        <p class="afrikaans-name">Stierlingse Sanger</p>
                        <p class="scientific">Calamonastes stierlingi</p>
                        <div class="card-meta">
                            <span class="meta-tag">Warblers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calonectris leucomelas"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/streaked-shearwater/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calonectris leucomelas')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Streaked Shearwater</h3>
                        <p class="afrikaans-name">Gestreepte Pylstormvoël</p>
                        <p class="scientific">Calonectris leucomelas</p>
                        <div class="card-meta">
                            <span class="meta-tag">Shearwaters</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sarothrura boehmi"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/streaky-breasted-flufftail/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sarothrura boehmi')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Streaky-breasted Flufftail</h3>
                        <p class="afrikaans-name">Streepborsvleikuiken</p>
                        <p class="scientific">Sarothrura boehmi</p>
                        <div class="card-meta">
                            <span class="meta-tag">Flufftails</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Crithagra gularis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/streaky-headed-seedeater/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Crithagra gularis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Streaky-headed Seedeater</h3>
                        <p class="afrikaans-name">Streepkopkanarie</p>
                        <p class="scientific">Crithagra gularis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Canaries</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Aenigmatolimnas marginalis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/striped-crake/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Aenigmatolimnas marginalis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Striped Crake</h3>
                        <p class="afrikaans-name">Gestreepte Riethaan</p>
                        <p class="scientific">Aenigmatolimnas marginalis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Crakes</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sarothrura affinis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/striped-flufftail/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sarothrura affinis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Striped Flufftail</h3>
                        <p class="afrikaans-name">Gestreepte Vleikuiken</p>
                        <p class="scientific">Sarothrura affinis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Flufftails</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Halcyon chelicuti"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/striped-kingfisher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Halcyon chelicuti')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Striped Kingfisher</h3>
                        <p class="afrikaans-name">Gestreepte Visvanger</p>
                        <p class="scientific">Halcyon chelicuti</p>
                        <div class="card-meta">
                            <span class="meta-tag">Kingfishers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anthus lineiventris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/striped-pipit/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anthus lineiventris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Striped Pipit</h3>
                        <p class="afrikaans-name">Gestreepte Koester</p>
                        <p class="scientific">Anthus lineiventris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pipits</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Puffinus elegans"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/subantarctic-shearwater/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Puffinus elegans')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Subantarctic Shearwater</h3>
                        <p class="afrikaans-name">Gevlekte Vleikuiken</p>
                        <p class="scientific">Puffinus elegans</p>
                        <div class="card-meta">
                            <span class="meta-tag">Shearwaters</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Stercorarius antarcticus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/subantarctic-skua/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Stercorarius antarcticus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Subantarctic Skua</h3>
                        <p class="afrikaans-name">Bruinroofmeeu</p>
                        <p class="scientific">Stercorarius antarcticus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Skuas</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pternistis swainsonii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/swainsons-spurfowl/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pternistis swainsonii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Swainson's Spurfowl</h3>
                        <p class="afrikaans-name">Bosveldfisant</p>
                        <p class="scientific">Pternistis swainsonii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Francolins</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Merops hirundineus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/swallow-tailed-bee-eater/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Merops hirundineus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #007bff"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Swallow-tailed Bee-eater</h3>
                        <p class="afrikaans-name">Swaelstertbyvreter</p>
                        <p class="scientific">Merops hirundineus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bee-eaters</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Caprimulgus natalensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/swamp-nightjar/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Caprimulgus natalensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Swamp Nightjar</h3>
                        <p class="afrikaans-name">Natalse Naguil</p>
                        <p class="scientific">Caprimulgus natalensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Nightjars</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Coccopygia melanotis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/swee-waxbill/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Coccopygia melanotis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            <span class="badge badge-near-endemic">Near-Endemic</span>
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Swee Waxbill</h3>
                        <p class="afrikaans-name">Rooikopwewer</p>
                        <p class="scientific">Coccopygia melanotis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Waxbills</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Thalasseus bergii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/swift-tern/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Thalasseus bergii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Swift Tern</h3>
                        <p class="afrikaans-name">Geelbeksterretjie</p>
                        <p class="scientific">Thalasseus bergii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Terns</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pseudobulweria rostrata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/tahiti-petrel/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pseudobulweria rostrata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Tahiti Petrel</h3>
                        <p class="afrikaans-name">Tahiti-stormvoël</p>
                        <p class="scientific">Pseudobulweria rostrata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Petrels</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Falco fasciinucha"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/taita-falcon/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Falco fasciinucha')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Taita Falcon</h3>
                        <p class="afrikaans-name">Taitavalk</p>
                        <p class="scientific">Falco fasciinucha</p>
                        <div class="card-meta">
                            <span class="meta-tag">Falcons</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Rocky</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Turtur tympanistria"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/tambourine-dove/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Turtur tympanistria')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #6f42c1"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Tambourine Dove</h3>
                        <p class="afrikaans-name">Witborsduifie</p>
                        <p class="scientific">Turtur tympanistria</p>
                        <div class="card-meta">
                            <span class="meta-tag">Doves</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Aquila rapax"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/tawny-eagle/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Aquila rapax')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Tawny Eagle</h3>
                        <p class="afrikaans-name">Roofarend</p>
                        <p class="scientific">Aquila rapax</p>
                        <div class="card-meta">
                            <span class="meta-tag">Eagles</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Prinia subflava"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/tawny-flanked-prinia/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Prinia subflava')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Tawny-flanked Prinia</h3>
                        <p class="afrikaans-name">Bruinsylangstertjie</p>
                        <p class="scientific">Prinia subflava</p>
                        <div class="card-meta">
                            <span class="meta-tag">Prinias</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cursorius temminckii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/temmincks-courser/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cursorius temminckii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Temminck's Courser</h3>
                        <p class="afrikaans-name">Trekdrawwertjie</p>
                        <p class="scientific">Cursorius temminckii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Coursers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calidris temminckii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/temmincks-stint/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calidris temminckii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Temminck's Stint</h3>
                        <p class="afrikaans-name">Temminckse Strandloper</p>
                        <p class="scientific">Calidris temminckii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Xenus cinereus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/terek-sandpiper/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Xenus cinereus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Terek Sandpiper</h3>
                        <p class="afrikaans-name">Terekruiter</p>
                        <p class="scientific">Xenus cinereus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Phyllastrephus terrestris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/terrestrial-brownbul/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Phyllastrephus terrestris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Terrestrial Brownbul</h3>
                        <p class="afrikaans-name">Boskrapper</p>
                        <p class="scientific">Phyllastrephus terrestris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Greenbuls</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pachycoccyx audeberti"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/thick-billed-cuckoo/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pachycoccyx audeberti')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Thick-billed Cuckoo</h3>
                        <p class="afrikaans-name">Dikbekkoekoek</p>
                        <p class="scientific">Pachycoccyx audeberti</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cuckoos</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Amblyospiza albifrons"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/thick-billed-weaver/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Amblyospiza albifrons')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Thick-billed Weaver</h3>
                        <p class="afrikaans-name">Dikbekwewer</p>
                        <p class="scientific">Amblyospiza albifrons</p>
                        <div class="card-meta">
                            <span class="meta-tag">Weavers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Rhinoptilus cinctus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/three-banded-courser/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Rhinoptilus cinctus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #FFFDD0"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Three-banded Courser</h3>
                        <p class="afrikaans-name">Driebanddrawwertjie</p>
                        <p class="scientific">Rhinoptilus cinctus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Coursers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Charadrius tricollaris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/three-banded-plover/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Charadrius tricollaris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Three-banded Plover</h3>
                        <p class="afrikaans-name">Driebandstrandkiewiet</p>
                        <p class="scientific">Charadrius tricollaris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Plovers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Luscinia luscinia"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/thrush-nightingale/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Luscinia luscinia')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Thrush Nightingale</h3>
                        <p class="afrikaans-name">Lysternagtegaal</p>
                        <p class="scientific">Luscinia luscinia</p>
                        <div class="card-meta">
                            <span class="meta-tag">Thrushes</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cisticola rufilatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/tinkling-cisticola/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cisticola rufilatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Tinkling Cisticola</h3>
                        <p class="afrikaans-name">Rooitinktinkie</p>
                        <p class="scientific">Cisticola rufilatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cisticolas</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Emarginata tractrac"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/tractrac-chat/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Emarginata tractrac')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Tractrac Chat</h3>
                        <p class="afrikaans-name">Woestynspekvreter</p>
                        <p class="scientific">Emarginata tractrac</p>
                        <div class="card-meta">
                            <span class="meta-tag">Chats</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anthus trivialis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/tree-pipit/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anthus trivialis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Tree Pipit</h3>
                        <p class="afrikaans-name">Boomkoester</p>
                        <p class="scientific">Anthus trivialis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pipits</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Diomedea dabbenena"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/tristan-albatross/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Diomedea dabbenena')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">CR</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Tristan Albatross</h3>
                        <p class="afrikaans-name">Tristangrootalbatros (-malmok)</p>
                        <p class="scientific">Diomedea dabbenena</p>
                        <div class="card-meta">
                            <span class="meta-tag">Albatrosses</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Laniarius major"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/tropical-boubou/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Laniarius major')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #e83e8c"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Tropical Boubou</h3>
                        <p class="afrikaans-name">Tropiese Waterfiskaal</p>
                        <p class="scientific">Laniarius major</p>
                        <div class="card-meta">
                            <span class="meta-tag">Boubous</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Savanna</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Puffinus bailloni"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/tropical-shearwater/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Puffinus bailloni')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Tropical Shearwater</h3>
                        <p class="afrikaans-name">Tropiese Kleinpylstormvoël</p>
                        <p class="scientific">Puffinus bailloni</p>
                        <div class="card-meta">
                            <span class="meta-tag">Shearwaters</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Bycanistes bucinator"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/trumpeter-hornbill/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Bycanistes bucinator')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Trumpeter Hornbill</h3>
                        <p class="afrikaans-name">Gewone Boskraai</p>
                        <p class="scientific">Bycanistes bucinator</p>
                        <div class="card-meta">
                            <span class="meta-tag">Hornbills</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Hippolais languida"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/upchers-warbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Hippolais languida')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Upcher's Warbler</h3>
                        <p class="afrikaans-name">Upcher se Sanger</p>
                        <p class="scientific">Hippolais languida</p>
                        <div class="card-meta">
                            <span class="meta-tag">Warblers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Bartramia longicauda"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/upland-sandpiper/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Bartramia longicauda')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Upland Sandpiper</h3>
                        <p class="afrikaans-name">Langstert-ruiter</p>
                        <p class="scientific">Bartramia longicauda</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cinnyris venustus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/variable-sunbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cinnyris venustus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #6f42c1"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Variable Sunbird</h3>
                        <p class="afrikaans-name">Geelpenssuikerbekkie</p>
                        <p class="scientific">Cinnyris venustus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sunbirds</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Aquila verreauxii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/verreauxs-eagle/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Aquila verreauxii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Verreaux's Eagle</h3>
                        <p class="afrikaans-name">Witkruisarend</p>
                        <p class="scientific">Aquila verreauxii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Eagles</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Rocky</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ketupa lactea"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/verreauxs-eagle-owl/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ketupa lactea')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Verreaux's Eagle- Owl</h3>
                        <p class="afrikaans-name">Reuse-ooruil</p>
                        <p class="scientific">Ketupa lactea</p>
                        <div class="card-meta">
                            <span class="meta-tag">Eagles</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cryptillas victorini"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/victorins-warbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cryptillas victorini')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Victorin's Warbler</h3>
                        <p class="afrikaans-name">Rooiborsruigtesanger</p>
                        <p class="scientific">Cryptillas victorini</p>
                        <div class="card-meta">
                            <span class="meta-tag">Warblers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Vidua chalybeata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/village-indigobird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Vidua chalybeata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Village Indigobird</h3>
                        <p class="afrikaans-name">Staalblouvinkie</p>
                        <p class="scientific">Vidua chalybeata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Whydahs</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ploceus cucullatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/village-weaver/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ploceus cucullatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Village Weaver</h3>
                        <p class="afrikaans-name">Bontrugwewer</p>
                        <p class="scientific">Ploceus cucullatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Weavers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cinnyricinclus leucogaster"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/violet-backed-starling/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cinnyricinclus leucogaster')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #6f42c1"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Violet-backed Starling</h3>
                        <p class="afrikaans-name">Witborsspreeu</p>
                        <p class="scientific">Cinnyricinclus leucogaster</p>
                        <div class="card-meta">
                            <span class="meta-tag">Starlings</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Granatina granatina"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/violet-eared-waxbill/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Granatina granatina')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #6f42c1"></div><div class="card-color-dot" style="background: #007bff"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Violet-eared Waxbill</h3>
                        <p class="afrikaans-name">Koningblousysie</p>
                        <p class="scientific">Granatina granatina</p>
                        <div class="card-meta">
                            <span class="meta-tag">Waxbills</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Hieraaetus wahlbergi"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/wahlbergs-eagle/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Hieraaetus wahlbergi')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Wahlberg's Eagle</h3>
                        <p class="afrikaans-name">Bruinarend</p>
                        <p class="scientific">Hieraaetus wahlbergi</p>
                        <div class="card-meta">
                            <span class="meta-tag">Eagles</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cisticola lais"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/wailing-cisticola/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cisticola lais')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Wailing Cisticola</h3>
                        <p class="afrikaans-name">Huiltinktinkie</p>
                        <p class="scientific">Cisticola lais</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cisticolas</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Diomedea exulans"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/wandering-albatross/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Diomedea exulans')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Wandering Albatross</h3>
                        <p class="afrikaans-name">Grootalbatros (-malmok)</p>
                        <p class="scientific">Diomedea exulans</p>
                        <div class="card-meta">
                            <span class="meta-tag">Albatrosses</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Burhinus vermiculatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/water-thick-knee/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Burhinus vermiculatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Water Thick-knee</h3>
                        <p class="afrikaans-name">Waterdikkop</p>
                        <p class="scientific">Burhinus vermiculatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Thick-knees</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Grus carunculata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/wattled-crane/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Grus carunculata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Wattled Crane</h3>
                        <p class="afrikaans-name">Lelkraanvoël</p>
                        <p class="scientific">Grus carunculata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cranes</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Creatophora cinerea"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/wattled-starling/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Creatophora cinerea')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Wattled Starling</h3>
                        <p class="afrikaans-name">Lelspreeu</p>
                        <p class="scientific">Creatophora cinerea</p>
                        <div class="card-meta">
                            <span class="meta-tag">Starlings</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ardenna pacifica"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/wedge-tailed-shearwater/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ardenna pacifica')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Wedge-tailed Shearwater</h3>
                        <p class="afrikaans-name">Keilstertpylstormvoël</p>
                        <p class="scientific">Ardenna pacifica</p>
                        <div class="card-meta">
                            <span class="meta-tag">Shearwaters</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Tyto alba"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/western-barn-owl/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Tyto alba')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Western Barn Owl</h3>
                        <p class="afrikaans-name">Nonnetjie-uil</p>
                        <p class="scientific">Tyto alba</p>
                        <div class="card-meta">
                            <span class="meta-tag">Owls</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Bubulcus ibis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/western-cattle-egret/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Bubulcus ibis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Western Cattle Egret</h3>
                        <p class="afrikaans-name">Veereier (Bosluisvoël)</p>
                        <p class="scientific">Bubulcus ibis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Herons</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Delichon urbicum"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/western-house-martin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Delichon urbicum')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Western House Martin</h3>
                        <p class="afrikaans-name">Huisswael</p>
                        <p class="scientific">Delichon urbicum</p>
                        <div class="card-meta">
                            <span class="meta-tag">Martins</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Rocky</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Circus aeruginosus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/western-marsh-harrier/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Circus aeruginosus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Western Marsh Harrier</h3>
                        <p class="afrikaans-name">Europese Vleivalk(Paddavreter)</p>
                        <p class="scientific">Circus aeruginosus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Harriers</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Motacilla flava"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/western-yellow-wagtail/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Motacilla flava')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Western Yellow Wagtail</h3>
                        <p class="afrikaans-name">Geelkwikkie</p>
                        <p class="scientific">Motacilla flava</p>
                        <div class="card-meta">
                            <span class="meta-tag">Wagtails</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Saxicola rubetra"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/whinchat/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Saxicola rubetra')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Whinchat</h3>
                        <p class="afrikaans-name">Europese Bontrokkie</p>
                        <p class="scientific">Saxicola rubetra</p>
                        <div class="card-meta">
                            <span class="meta-tag">Chats</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Chlidonias hybrida"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/whiskered-tern/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Chlidonias hybrida')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Whiskered Tern</h3>
                        <p class="afrikaans-name">Witbaardsterretjie</p>
                        <p class="scientific">Chlidonias hybrida</p>
                        <div class="card-meta">
                            <span class="meta-tag">Terns</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ciconia ciconia"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-stork/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ciconia ciconia')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White Stork</h3>
                        <p class="afrikaans-name">Witooievaar</p>
                        <p class="scientific">Ciconia ciconia</p>
                        <div class="card-meta">
                            <span class="meta-tag">Storks</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Gygis alba"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-tern/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Gygis alba')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White Tern</h3>
                        <p class="afrikaans-name">Feesterretjie</p>
                        <p class="scientific">Gygis alba</p>
                        <div class="card-meta">
                            <span class="meta-tag">Terns</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Motacilla alba"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-wagtail/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Motacilla alba')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White Wagtail</h3>
                        <p class="afrikaans-name">Wit Kwikkie</p>
                        <p class="scientific">Motacilla alba</p>
                        <div class="card-meta">
                            <span class="meta-tag">Wagtails</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Thalassornis leuconotus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-backed-duck/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Thalassornis leuconotus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-backed Duck</h3>
                        <p class="afrikaans-name">Witrugeend</p>
                        <p class="scientific">Thalassornis leuconotus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ducks</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Colius colius"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-backed-mousebird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Colius colius')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-backed Mousebird</h3>
                        <p class="afrikaans-name">Witkruismuisvoël</p>
                        <p class="scientific">Colius colius</p>
                        <div class="card-meta">
                            <span class="meta-tag">Mousebirds</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Gorsachius leuconotus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-backed-night-heron/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Gorsachius leuconotus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-backed Night Heron</h3>
                        <p class="afrikaans-name">Witrugnagreier</p>
                        <p class="scientific">Gorsachius leuconotus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Herons</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Gyps africanus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-backed-vulture/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Gyps africanus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">CR</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-backed Vulture</h3>
                        <p class="afrikaans-name">Witrugaasvoël</p>
                        <p class="scientific">Gyps africanus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Vultures</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Eupodotis senegalensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-bellied-korhaan/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Eupodotis senegalensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-bellied Korhaan</h3>
                        <p class="afrikaans-name">Witpenskorhaan</p>
                        <p class="scientific">Eupodotis senegalensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bustards</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Fregetta grallaria"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-bellied-storm-petrel/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Fregetta grallaria')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-bellied Storm Petrel</h3>
                        <p class="afrikaans-name">Witpensstormswael</p>
                        <p class="scientific">Fregetta grallaria</p>
                        <div class="card-meta">
                            <span class="meta-tag">Storm-Petrels</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cinnyris talatala"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-bellied-sunbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cinnyris talatala')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-bellied Sunbird</h3>
                        <p class="afrikaans-name">Witpenssuikerbekkie</p>
                        <p class="scientific">Cinnyris talatala</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sunbirds</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Phalacrocorax lucidus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-breasted-cormorant/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Phalacrocorax lucidus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-breasted Cormorant</h3>
                        <p class="afrikaans-name">Witborsduiker</p>
                        <p class="scientific">Phalacrocorax lucidus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cormorants</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ceblepyris pectoralis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-breasted-cuckooshrike/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ceblepyris pectoralis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-breasted Cuckooshrike</h3>
                        <p class="afrikaans-name">Witborskatakoeroe</p>
                        <p class="scientific">Ceblepyris pectoralis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cuckooshrikes</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cossypha heuglini"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-browed-robin-chat/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cossypha heuglini')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-browed Robin-chat</h3>
                        <p class="afrikaans-name">Heuglinse Janfrederik</p>
                        <p class="scientific">Cossypha heuglini</p>
                        <div class="card-meta">
                            <span class="meta-tag">Robin-Chats</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Rivers</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cercotrichas leucophrys"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-browed-scrub-robin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cercotrichas leucophrys')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-browed Scrub Robin</h3>
                        <p class="afrikaans-name">Gestreepte Wipstert</p>
                        <p class="scientific">Cercotrichas leucophrys</p>
                        <div class="card-meta">
                            <span class="meta-tag">Scrub-Robins</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Plocepasser mahali"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-browed-sparrow-weaver/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Plocepasser mahali')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-browed Sparrow-weaver</h3>
                        <p class="afrikaans-name">Koringvoël</p>
                        <p class="scientific">Plocepasser mahali</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sparrow-Weavers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sterna repressa"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-cheeked-tern/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sterna repressa')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-cheeked Tern</h3>
                        <p class="afrikaans-name">Witwangsterretjie</p>
                        <p class="scientific">Sterna repressa</p>
                        <div class="card-meta">
                            <span class="meta-tag">Terns</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Procellaria aequinoctialis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-chinned-petrel/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Procellaria aequinoctialis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-chinned Petrel</h3>
                        <p class="afrikaans-name">Bassiaan</p>
                        <p class="scientific">Procellaria aequinoctialis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Petrels</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Prionops plumatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-crested-helmet-shrike/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Prionops plumatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-crested Helmet-shrike</h3>
                        <p class="afrikaans-name">Withelmlaksman</p>
                        <p class="scientific">Prionops plumatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Helmet-shrikes</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Vanellus albiceps"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-crowned-lapwing/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Vanellus albiceps')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-crowned Lapwing</h3>
                        <p class="afrikaans-name">Witkopkiewiet</p>
                        <p class="scientific">Vanellus albiceps</p>
                        <div class="card-meta">
                            <span class="meta-tag">Lapwings</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Stactolaema leucotis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-eared-barbet/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Stactolaema leucotis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-eared Barbet</h3>
                        <p class="afrikaans-name">Witoorhoutkapper</p>
                        <p class="scientific">Stactolaema leucotis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Barbets</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pelagodroma marina"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-faced-storm-petrel/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pelagodroma marina')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-faced Storm Petrel</h3>
                        <p class="afrikaans-name">Witwangstormswael</p>
                        <p class="scientific">Pelagodroma marina</p>
                        <div class="card-meta">
                            <span class="meta-tag">Storm-Petrels</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Dendrocygna viduata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-faced-whistling-duck/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Dendrocygna viduata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-faced Whistling Duck</h3>
                        <p class="afrikaans-name">Nonnetjie-eend</p>
                        <p class="scientific">Dendrocygna viduata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ducks</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Merops bullockoides"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-fronted-bee-eater/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Merops bullockoides')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #dc3545"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-fronted Bee-eater</h3>
                        <p class="afrikaans-name">Rooikeelbyvreter</p>
                        <p class="scientific">Merops bullockoides</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bee-eaters</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Charadrius marginatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-fronted-plover/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Charadrius marginatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-fronted Plover</h3>
                        <p class="afrikaans-name">Vaalstrandkiewiet</p>
                        <p class="scientific">Charadrius marginatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Plovers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Coastal</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pterodroma lessonii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-headed-petrel/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pterodroma lessonii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-headed Petrel</h3>
                        <p class="afrikaans-name">Witkopstormvoël</p>
                        <p class="scientific">Pterodroma lessonii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Petrels</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Trigonoceps occipitalis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-headed-vulture/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Trigonoceps occipitalis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">CR</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #e83e8c"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-headed Vulture</h3>
                        <p class="afrikaans-name">Witkopaasvoël</p>
                        <p class="scientific">Trigonoceps occipitalis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Vultures</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Corvus albicollis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-necked-raven/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Corvus albicollis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-necked Raven</h3>
                        <p class="afrikaans-name">Withalskraai</p>
                        <p class="scientific">Corvus albicollis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Crows</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Rocky</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Calidris fuscicollis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-rumped-sandpiper/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Calidris fuscicollis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-rumped Sandpiper</h3>
                        <p class="afrikaans-name">Witrugstrandloper</p>
                        <p class="scientific">Calidris fuscicollis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Apus caffer"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-rumped-swift/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Apus caffer')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-rumped Swift</h3>
                        <p class="afrikaans-name">Witkruiswindswael</p>
                        <p class="scientific">Apus caffer</p>
                        <div class="card-meta">
                            <span class="meta-tag">Swifts</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pogonocichla stellata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-starred-robin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pogonocichla stellata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-starred Robin</h3>
                        <p class="afrikaans-name">Witkoljanfrederik</p>
                        <p class="scientific">Pogonocichla stellata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Robins</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Phaethon lepturus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-tailed-tropicbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Phaethon lepturus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-tailed Tropicbird</h3>
                        <p class="afrikaans-name">Witpylstert</p>
                        <p class="scientific">Phaethon lepturus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Tropicbirds</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Merops albicollis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-throated-bee-eater/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Merops albicollis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-throated Bee-eater</h3>
                        <p class="afrikaans-name">Witkeelbyvreter</p>
                        <p class="scientific">Merops albicollis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bee-eaters</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Crithagra albogularis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-throated-canary/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Crithagra albogularis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-throated Canary</h3>
                        <p class="afrikaans-name">Witkeelkanarie</p>
                        <p class="scientific">Crithagra albogularis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Canaries</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Irania gutturalis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-throated-robin/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Irania gutturalis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-throated Robin</h3>
                        <p class="afrikaans-name">Geelkeelsandpatrys</p>
                        <p class="scientific">Irania gutturalis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Robins</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cossypha humeralis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-throated-robin-chat/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cossypha humeralis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-throated Robin-chat</h3>
                        <p class="afrikaans-name">Witkeeljanfrederik</p>
                        <p class="scientific">Cossypha humeralis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Robin-Chats</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Savanna</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Hirundo albigularis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-throated-swallow/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Hirundo albigularis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-throated Swallow</h3>
                        <p class="afrikaans-name">Witkeelswael</p>
                        <p class="scientific">Hirundo albigularis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Swallows</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Sarothrura ayresi"><img class="card-thumbnail" width="320" height="240" src="https://inaturalist-open-data.s3.amazonaws.com/photos/13707797/medium.jpg" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Sarothrura ayresi')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            <span class="badge badge-threatened">EN</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-winged Flufftail</h3>
                        <p class="afrikaans-name">Witvlerkvleikuiken</p>
                        <p class="scientific">Sarothrura ayresi</p>
                        <div class="card-meta">
                            <span class="meta-tag">Flufftails</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Chlidonias leucopterus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-winged-tern/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Chlidonias leucopterus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-winged Tern</h3>
                        <p class="afrikaans-name">Witvlerksterretjie</p>
                        <p class="scientific">Chlidonias leucopterus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Terns</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Euplectes albonotatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/white-winged-widowbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Euplectes albonotatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>White-winged Widowbird</h3>
                        <p class="afrikaans-name">Witvlerkflap</p>
                        <p class="scientific">Euplectes albonotatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Widowbirds</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Phylloscopus trochilus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/willow-warbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Phylloscopus trochilus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Willow Warbler</h3>
                        <p class="afrikaans-name">Hofsanger</p>
                        <p class="scientific">Phylloscopus trochilus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Warblers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Phalaropus tricolor"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/wilsons-phalarope/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Phalaropus tricolor')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Wilson's Phalarope</h3>
                        <p class="afrikaans-name">Bontfraiingpoot</p>
                        <p class="scientific">Phalaropus tricolor</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Oceanites oceanicus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/wilsons-storm-petrel/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Oceanites oceanicus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Wilson's Storm Petrel</h3>
                        <p class="afrikaans-name">Gewone Stormswael</p>
                        <p class="scientific">Oceanites oceanicus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Storm-Petrels</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Marine</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cisticola ayresii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/wing-snapping-cisticola/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cisticola ayresii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Wing-snapping Cisticola</h3>
                        <p class="afrikaans-name">Kleinste Klopkloppie</p>
                        <p class="scientific">Cisticola ayresii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cisticolas</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Hirundo smithii"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/wire-tailed-swallow/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Hirundo smithii')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #fd7e14"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Wire-tailed Swallow</h3>
                        <p class="afrikaans-name">Draadstertswael</p>
                        <p class="scientific">Hirundo smithii</p>
                        <div class="card-meta">
                            <span class="meta-tag">Swallows</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Tringa glareola"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/wood-sandpiper/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Tringa glareola')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Wood Sandpiper</h3>
                        <p class="afrikaans-name">Bosruiter</p>
                        <p class="scientific">Tringa glareola</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandpipers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Phylloscopus sibilatrix"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/wood-warbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Phylloscopus sibilatrix')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Wood Warbler</h3>
                        
                        <p class="scientific">Phylloscopus sibilatrix</p>
                        <div class="card-meta">
                            <span class="meta-tag">Warblers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Lanius senator"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/woodchat-shrike/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Lanius senator')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Woodchat Shrike</h3>
                        
                        <p class="scientific">Lanius senator</p>
                        <div class="card-meta">
                            <span class="meta-tag">Shrikes</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Halcyon senegalensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/woodland-kingfisher/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Halcyon senegalensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #007bff"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Woodland Kingfisher</h3>
                        <p class="afrikaans-name">Bosveldvisvanger</p>
                        <p class="scientific">Halcyon senegalensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Kingfishers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Batis fratrum"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/woodwards-batis/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Batis fratrum')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Woodwards' Batis</h3>
                        <p class="afrikaans-name">Woodwardse Bosbontrokkie</p>
                        <p class="scientific">Batis fratrum</p>
                        <div class="card-meta">
                            <span class="meta-tag">Batises</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Euplectes capensis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/yellow-bishop/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Euplectes capensis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Yellow Bishop</h3>
                        <p class="afrikaans-name">Kaapse Flap</p>
                        <p class="scientific">Euplectes capensis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Widowbirds</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Crithagra flaviventris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/yellow-canary/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Crithagra flaviventris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Yellow Canary</h3>
                        <p class="afrikaans-name">Geelkanarie</p>
                        <p class="scientific">Crithagra flaviventris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Canaries</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ploceus subaureus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/yellow-weaver/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ploceus subaureus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Yellow Weaver</h3>
                        <p class="afrikaans-name">Geelwewer</p>
                        <p class="scientific">Ploceus subaureus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Weavers</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Rivers</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Eremomela icteropygialis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/yellow-bellied-eremomela/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Eremomela icteropygialis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Yellow-bellied Eremomela</h3>
                        <p class="afrikaans-name">Geelpensbossanger</p>
                        <p class="scientific">Eremomela icteropygialis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Eremomelas</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Chlorocichla flaviventris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/yellow-bellied-greenbul/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Chlorocichla flaviventris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Yellow-bellied Greenbul</h3>
                        <p class="afrikaans-name">Geelborswillie</p>
                        <p class="scientific">Chlorocichla flaviventris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Bulbuls</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anas undulata"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/yellow-billed-duck/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anas undulata')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #007bff"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Yellow-billed Duck</h3>
                        <p class="afrikaans-name">Geelbekeend</p>
                        <p class="scientific">Anas undulata</p>
                        <div class="card-meta">
                            <span class="meta-tag">Ducks</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Ardea intermedia"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/yellow-billed-egret/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Ardea intermedia')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Yellow-billed Egret</h3>
                        <p class="afrikaans-name">Geelbekwitreier</p>
                        <p class="scientific">Ardea intermedia</p>
                        <div class="card-meta">
                            <span class="meta-tag">Herons</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Milvus aegyptius"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/yellow-billed-kite/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Milvus aegyptius')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Yellow-billed Kite</h3>
                        
                        <p class="scientific">Milvus aegyptius</p>
                        <div class="card-meta">
                            <span class="meta-tag">Kites</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Buphagus africanus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/yellow-billed-oxpecker/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Buphagus africanus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #dc3545"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Yellow-billed Oxpecker</h3>
                        <p class="afrikaans-name">Geelbekrenostervoël</p>
                        <p class="scientific">Buphagus africanus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Oxpeckers</span>
                            <span class="meta-tag">Medium</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Mycteria ibis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/yellow-billed-stork/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Mycteria ibis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #f5f5f5"></div><div class="card-color-dot" style="background: #e83e8c"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Yellow-billed Stork</h3>
                        <p class="afrikaans-name">Nimmersat</p>
                        <p class="scientific">Mycteria ibis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Storks</span>
                            <span class="meta-tag">Very Large</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Apalis flavida"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/yellow-breasted-apalis/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Apalis flavida')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #888"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #28a745"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Yellow-breasted Apalis</h3>
                        <p class="afrikaans-name">Geelborskleinjantjie</p>
                        <p class="scientific">Apalis flavida</p>
                        <div class="card-meta">
                            <span class="meta-tag">Apalises</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Anthus chloris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/yellow-breasted-pipit/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Anthus chloris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            <span class="badge badge-endemic">Endemic</span>
                            
                            <span class="badge badge-threatened">VU</span>
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Yellow-breasted Pipit</h3>
                        <p class="afrikaans-name">Geelborskoester</p>
                        <p class="scientific">Anthus chloris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Pipits</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Euplectes afer"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/yellow-crowned-bishop/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Euplectes afer')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #8B4513"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Yellow-crowned Bishop</h3>
                        <p class="afrikaans-name">Goudgeelvink</p>
                        <p class="scientific">Euplectes afer</p>
                        <div class="card-meta">
                            <span class="meta-tag">Widowbirds</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Wetland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Crithagra mozambica"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/yellow-fronted-canary/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Crithagra mozambica')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Yellow-fronted Canary</h3>
                        <p class="afrikaans-name">Geeloogkanarie</p>
                        <p class="scientific">Crithagra mozambica</p>
                        <div class="card-meta">
                            <span class="meta-tag">Canaries</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Fynbos</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pogoniulus chrysoconus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/yellow-fronted-tinkerbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pogoniulus chrysoconus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Yellow-fronted Tinkerbird</h3>
                        <p class="afrikaans-name">Geelblestinker</p>
                        <p class="scientific">Pogoniulus chrysoconus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Tinkerbirds</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pogoniulus bilineatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/yellow-rumped-tinkerbird/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pogoniulus bilineatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #333"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #f5f5f5"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Yellow-rumped Tinkerbird</h3>
                        <p class="afrikaans-name">Swartblestinker</p>
                        <p class="scientific">Pogoniulus bilineatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Tinkerbirds</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Phyllastrephus flavostriatus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/yellow-streaked-greenbul/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Phyllastrephus flavostriatus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Yellow-streaked Greenbul</h3>
                        <p class="afrikaans-name">Geelstreepboskruiper</p>
                        <p class="scientific">Phyllastrephus flavostriatus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Greenbuls</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Macronyx croceus"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/yellow-throated-longclaw/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Macronyx croceus')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #333"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Yellow-throated Longclaw</h3>
                        <p class="afrikaans-name">Geelkeelkalkoentjie</p>
                        <p class="scientific">Macronyx croceus</p>
                        <div class="card-meta">
                            <span class="meta-tag">Longclaws</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Gymnoris superciliaris"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/yellow-throated-petronia/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Gymnoris superciliaris')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #ffc107"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Yellow-throated Petronia</h3>
                        <p class="afrikaans-name">Geelvlekmossie</p>
                        <p class="scientific">Gymnoris superciliaris</p>
                        <div class="card-meta">
                            <span class="meta-tag">Petronias</span>
                            <span class="meta-tag">Small</span>
                            <span class="meta-tag">Woodland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Pterocles gutturalis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/yellow-throated-sandgrouse/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Pterocles gutturalis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Yellow-throated Sandgrouse</h3>
                        <p class="afrikaans-name">Geelkeelsandpatrys</p>
                        <p class="scientific">Pterocles gutturalis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Sandgrouse</span>
                            <span class="meta-tag">Large</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Phylloscopus ruficapilla"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/yellow-throated-woodland-warbler/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Phylloscopus ruficapilla')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #ffc107"></div><div class="card-color-dot" style="background: #28a745"></div><div class="card-color-dot" style="background: #888"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Yellow-throated Woodland Warbler</h3>
                        <p class="afrikaans-name">Geelkeelsanger</p>
                        <p class="scientific">Phylloscopus ruficapilla</p>
                        <div class="card-meta">
                            <span class="meta-tag">Warblers</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Forest</span>
                        </div>
                    </div></div><div class="card ">
                    <div class="card-image" data-species="Cisticola juncidis"><img class="card-thumbnail" width="320" height="240" src="https://cdn.sabirds.app/zitting-cisticola/1_medium.webp" alt="">
                        <span class="placeholder" style="display: none;">🐦</span>
                        <button class="lifelist-btn " onclick="toggleSeen(event, 'Cisticola juncidis')" title="Log sighting">
                            +
                        </button>
                        <div class="card-badges">
                            
                            
                            
                            <span class="badge badge-sound">🔊</span>
                        </div>
                        <div class="card-color-dots">
                            <div class="card-color-dot" style="background: #8B4513"></div><div class="card-color-dot" style="background: #f5deb3"></div>
                        </div>
                    </div>
                    <div class="card-content">
                        <h3>Zitting Cisticola</h3>
                        <p class="afrikaans-name">Landeryklopkloppie</p>
                        <p class="scientific">Cisticola juncidis</p>
                        <div class="card-meta">
                            <span class="meta-tag">Cisticolas</span>
                            <span class="meta-tag">Tiny</span>
                            <span class="meta-tag">Grassland</span>
                        </div>
                    </div>
                </div>
            </div>
                    '''

In [4]:
soup = BeautifulSoup(html_content, 'html.parser')
birds = soup.find_all('div', class_="card")
df = pd.DataFrame()
for bird in birds:
    # print(extract_bird_details(str(bird)))
    df = pd.concat([df, pd.DataFrame(extract_bird_details(str(bird)))], ignore_index=True)
df

,common_name,scientific_name,afrikaans_name,image,family,size,habitat,colors
0,Abdim's Stork,Ciconia abdimii,Kleinswartooievaar,https://cdn.sabirds.app/abdims-stork/1_medium....,Storks,Very Large,Grassland,"[Black, White, Grey]"
1,Acacia Pied Barbet,Tricholaema leucomelas,Bonthoutkapper,https://cdn.sabirds.app/acacia-pied-barbet/1_m...,Barbets,Small,Fynbos,"[Black, White, Red]"
2,African Barred Owlet,Glaucidium capense,Gebande Uil,https://cdn.sabirds.app/african-barred-owlet/1...,Owls,Medium,Forest,"[Brown, White]"
3,African Black Duck,Anas sparsa,Swarteend,https://cdn.sabirds.app/african-black-duck/1_m...,Ducks,Large,Wetland,"[Brown, Black]"
4,African Black Swift,Apus barbatus,Swartwindswael,https://cdn.sabirds.app/african-black-swift/1_...,Swifts,Small,Rocky,[Black]
...,...,...,...,...,...,...,...,...
871,Yellow-throated Longclaw,Macronyx croceus,Geelkeelkalkoentjie,https://cdn.sabirds.app/yellow-throated-longcl...,Longclaws,Small,Grassland,"[Brown, Yellow, Black]"
872,Yellow-throated Petronia,Gymnoris superciliaris,Geelvlekmossie,https://cdn.sabirds.app/yellow-throated-petron...,Petronias,Small,Woodland,"[Brown, Yellow]"
873,Yellow-throated Sandgrouse,Pterocles gutturalis,Geelkeelsandpatrys,https://cdn.sabirds.app/yellow-throated-sandgr...,Sandgrouse,Large,Grassland,"[Brown, Yellow, Buff]"
874,Yellow-throated Woodland Warbler,Phylloscopus ruficapilla,Geelkeelsanger,https://cdn.sabirds.app/yellow-throated-woodla...,Warblers,Tiny,Forest,"[Yellow, Green, Grey]"


In [28]:
filepath = "C:\\Users\\scuba\\Downloads\\BIRD APP\\"

In [29]:
df.to_csv(filepath + "BIRD_DATA (WITHOUT AUDIO).csv", index=False)

In [30]:
df.to_json(filepath + "BIRD_DATA (WITHOUT AUDIO).json", orient='records', lines=True)

# DROPDOWNS

## family

In [13]:
family = ['All'] + list(df.family.sort_values().unique())
family

['All',
 'Albatrosses',
 'Apalises',
 'Avadavats',
 'Avocets',
 'Babblers',
 'Barbets',
 'Batises',
 'Bee-eaters',
 'Bitterns',
 'Bokmakieries',
 'Boobies',
 'Boubous',
 'Broadbills',
 'Brubrus',
 'Buffalo-Weavers',
 'Bulbuls',
 'Buntings',
 'Bush-Warblers',
 'Bush-shrikes',
 'Bustards',
 'Buttonquails',
 'Buzzards',
 'Camaropteras',
 'Canaries',
 'Chats',
 'Cisticolas',
 'Coots',
 'Cormorants',
 'Coucals',
 'Coursers',
 'Crab-Plovers',
 'Crakes',
 'Cranes',
 'Creepers',
 'Crombecs',
 'Crows',
 'Cuckoos',
 'Cuckooshrikes',
 'Curlews',
 'Darters',
 'Doves',
 'Drongos',
 'Ducks',
 'Eagles',
 'Eremomelas',
 'Fairy-Flycatchers',
 'Falcons',
 'Finches',
 'Finfoots',
 'Firefinches',
 'Flamingos',
 'Flufftails',
 'Flycatchers',
 'Francolins',
 'Frigatebirds',
 'Gamebirds',
 'Gannets',
 'Geese',
 'Go-away-birds',
 'Godwits',
 'Goshawks',
 'Grassbirds',
 'Grebes',
 'Greenbuls',
 'Ground-Hornbills',
 'Guineafowl',
 'Gulls',
 'Gulls & Terns',
 'Hamerkop',
 'Harriers',
 'Hawks',
 'Helmet-shrikes',

## size

In [31]:
size = ['None'] + list(df['size'].unique())
size    

['None', 'Very Large', 'Small', 'Medium', 'Large', 'Tiny']

## habitat

In [32]:
habitat = ['None'] + list(df['habitat'].unique())
habitat

['None',
 'Grassland',
 'Fynbos',
 'Forest',
 'Wetland',
 'Rocky',
 'Woodland',
 'Savanna',
 'Coastal',
 'Marine',
 'Rivers',
 'Shrubland',
 'Karoo',
 'Gardens']

## FIND THE TOP 5 SIMILAR BIRDS

In [10]:
bird = df.iloc[7]
bird['common_name']

'African Cuckoo'

In [61]:
# Same family
df_family = df[df.family==df.iloc[7]['family']]
df_family

,common_name,scientific_name,afrikaans_name,image,family,size,habitat,colors
7,African Cuckoo,Cuculus gularis,Afrikaanse Koekoek,https://cdn.sabirds.app/african-cuckoo/1_mediu...,Cuckoos,Medium,Fynbos,"[Grey, White, Black]"
11,African Emerald Cuckoo,Chrysococcyx cupreus,Mooimeisie,https://cdn.sabirds.app/african-emerald-cuckoo...,Cuckoos,Small,Forest,"[Green, Yellow]"
84,Barred Long-tailed Cuckoo,Cercococcyx montanus,Langstertkoekoek,https://cdn.sabirds.app/barred-long-tailed-cuc...,Cuckoos,Medium,Forest,"[Red, Grey, Brown, Black]"
95,Black Cuckoo,Cuculus clamosus,Swartkoekoek,https://cdn.sabirds.app/black-cuckoo/1_medium....,Cuckoos,Medium,Forest,[Black]
236,Common Cuckoo,Cuculus canorus,Europese Koekoek,https://cdn.sabirds.app/common-cuckoo/1_medium...,Cuckoos,Medium,Forest,"[Grey, White, Black]"
267,Cuckoo Finch,Anomalospiza imberbis,Koekoekvink,None,Cuckoos,Small,Grassland,"[Yellow, Brown, Black]"
278,Diederik Cuckoo,Chrysococcyx caprius,Diederikkie,None,Cuckoos,Small,Fynbos,"[Grey, Brown, Black]"
348,Great Spotted Cuckoo,Clamator glandarius,Gevlekte Koekoek,https://cdn.sabirds.app/great-spotted-cuckoo/1...,Cuckoos,Medium,Fynbos,"[Grey, White, Brown]"
360,Green Malkoha,Ceuthmochares australis,Groenvleiloerie,None,Cuckoos,Medium,Forest,"[Green, Grey, Red]"
411,Jacobin Cuckoo,Clamator jacobinus,Bontnuwejaarsvoël,https://cdn.sabirds.app/jacobin-cuckoo/1_mediu...,Cuckoos,Medium,Grassland,"[Black, White]"


In [62]:
# Sane size
df_size = df[df['size']==df.iloc[7]['size']]
df_size

,common_name,scientific_name,afrikaans_name,image,family,size,habitat,colors
2,African Barred Owlet,Glaucidium capense,Gebande Uil,https://cdn.sabirds.app/african-barred-owlet/1...,Owls,Medium,Forest,"[Brown, White]"
6,African Crake,Crecopsis egregia,Afrikaanse Riethaan,https://cdn.sabirds.app/african-crake/1_medium...,Crakes,Medium,Woodland,"[Brown, Grey, Red]"
7,African Cuckoo,Cuculus gularis,Afrikaanse Koekoek,https://cdn.sabirds.app/african-cuckoo/1_mediu...,Cuckoos,Medium,Fynbos,"[Grey, White, Black]"
15,African Golden Oriole,Oriolus auratus,Afrikaanse Wielewaal,https://cdn.sabirds.app/african-golden-oriole/...,Orioles,Medium,Woodland,"[Yellow, Black]"
18,African Green Pigeon,Treron calvus,Papegaaiduif,https://cdn.sabirds.app/african-green-pigeon/1...,Pigeons,Medium,Forest,"[Green, Yellow, Purple]"
...,...,...,...,...,...,...,...,...
842,White-winged Tern,Chlidonias leucopterus,Witvlerksterretjie,https://cdn.sabirds.app/white-winged-tern/1_me...,Terns,Medium,Wetland,"[White, Black, Grey]"
845,Wilson's Phalarope,Phalaropus tricolor,Bontfraiingpoot,https://cdn.sabirds.app/wilsons-phalarope/1_me...,Sandpipers,Medium,Wetland,"[Brown, Grey, White]"
849,Wood Sandpiper,Tringa glareola,Bosruiter,https://cdn.sabirds.app/wood-sandpiper/1_mediu...,Sandpipers,Medium,Wetland,"[Brown, White]"
852,Woodland Kingfisher,Halcyon senegalensis,Bosveldvisvanger,https://cdn.sabirds.app/woodland-kingfisher/1_...,Kingfishers,Medium,Woodland,"[Blue, Black, Red]"


In [63]:
# Same habitat
df_habitat = df[df['habitat']==df.iloc[7]['habitat']]
df

,common_name,scientific_name,afrikaans_name,image,family,size,habitat,colors
0,Abdim's Stork,Ciconia abdimii,Kleinswartooievaar,https://cdn.sabirds.app/abdims-stork/1_medium....,Storks,Very Large,Grassland,"[Black, White, Grey]"
1,Acacia Pied Barbet,Tricholaema leucomelas,Bonthoutkapper,https://cdn.sabirds.app/acacia-pied-barbet/1_m...,Barbets,Small,Fynbos,"[Black, White, Red]"
2,African Barred Owlet,Glaucidium capense,Gebande Uil,https://cdn.sabirds.app/african-barred-owlet/1...,Owls,Medium,Forest,"[Brown, White]"
3,African Black Duck,Anas sparsa,Swarteend,https://cdn.sabirds.app/african-black-duck/1_m...,Ducks,Large,Wetland,"[Brown, Black]"
4,African Black Swift,Apus barbatus,Swartwindswael,https://cdn.sabirds.app/african-black-swift/1_...,Swifts,Small,Rocky,[Black]
...,...,...,...,...,...,...,...,...
871,Yellow-throated Longclaw,Macronyx croceus,Geelkeelkalkoentjie,https://cdn.sabirds.app/yellow-throated-longcl...,Longclaws,Small,Grassland,"[Brown, Yellow, Black]"
872,Yellow-throated Petronia,Gymnoris superciliaris,Geelvlekmossie,https://cdn.sabirds.app/yellow-throated-petron...,Petronias,Small,Woodland,"[Brown, Yellow]"
873,Yellow-throated Sandgrouse,Pterocles gutturalis,Geelkeelsandpatrys,https://cdn.sabirds.app/yellow-throated-sandgr...,Sandgrouse,Large,Grassland,"[Brown, Yellow, Buff]"
874,Yellow-throated Woodland Warbler,Phylloscopus ruficapilla,Geelkeelsanger,https://cdn.sabirds.app/yellow-throated-woodla...,Warblers,Tiny,Forest,"[Yellow, Green, Grey]"


In [64]:
# Similar colors
df_colors =pd.DataFrame()
for n in range(len(df)):
    if set(df.iloc[7]['colors']).issubset(set(df.iloc[n]['colors'])):
        # print(n, df.iloc[n]['common_name'], df.iloc[n]['colors'])
        df_colors = pd.concat([df1, pd.DataFrame(df.iloc[n]).T], ignore_index=True)
df_colors

,common_name,scientific_name,afrikaans_name,image,family,size,habitat,colors
0,Abdim's Stork,Ciconia abdimii,Kleinswartooievaar,https://cdn.sabirds.app/abdims-stork/1_medium....,Storks,Very Large,Grassland,"[Black, White, Grey]"
1,African Cuckoo,Cuculus gularis,Afrikaanse Koekoek,https://cdn.sabirds.app/african-cuckoo/1_mediu...,Cuckoos,Medium,Fynbos,"[Grey, White, Black]"
2,African Grey Hornbill,Lophoceros nasutus,Grysneushoringvoël,https://cdn.sabirds.app/african-grey-hornbill/...,Hornbills,Medium,Fynbos,"[Grey, Black, White]"
3,Amsterdam Albatross,Diomedea amsterdamensis,Amsterdamalbatros,https://cdn.sabirds.app/amsterdam-albatross/1_...,Albatrosses,Very Large,Marine,"[White, Black, Grey]"
4,Ashy Tit,Melaniparus cinerascens,Akasiagrysmees,https://cdn.sabirds.app/ashy-tit/1_medium.webp,Tits,Small,Fynbos,"[Grey, Black, White]"
...,...,...,...,...,...,...,...,...
68,White-crested Helmet-shrike,Prionops plumatus,Withelmlaksman,https://cdn.sabirds.app/white-crested-helmet-s...,Helmet-shrikes,Small,Woodland,"[White, Grey, Black]"
69,White-winged Tern,Chlidonias leucopterus,Witvlerksterretjie,https://cdn.sabirds.app/white-winged-tern/1_me...,Terns,Medium,Wetland,"[White, Black, Grey]"
70,Woodchat Shrike,Lanius senator,None,https://cdn.sabirds.app/woodchat-shrike/1_medi...,Shrikes,Small,Fynbos,"[Grey, Black, White]"
71,Woodwards' Batis,Batis fratrum,Woodwardse Bosbontrokkie,https://cdn.sabirds.app/woodwards-batis/1_medi...,Batises,Tiny,Forest,"[Black, White, Grey]"


In [96]:
features = df.iloc[7]['colors'] + [df.iloc[7]['size']]
features

['Grey', 'White', 'Black', 'Medium']

In [98]:
df_features =pd.DataFrame()
for n in range(len(df)):
    bird_features = df.iloc[n]['colors'] + [df.iloc[n]['size']]
    if set(features).issubset(set(bird_features)):
        # print(n, df.iloc[n]['common_name'], bird_features)
        df_features = pd.concat([df_features, pd.DataFrame(df.iloc[n]).T], ignore_index=True) 
df_features  

,common_name,scientific_name,afrikaans_name,image,family,size,habitat,colors
0,African Cuckoo,Cuculus gularis,Afrikaanse Koekoek,https://cdn.sabirds.app/african-cuckoo/1_mediu...,Cuckoos,Medium,Fynbos,"[Grey, White, Black]"
1,African Grey Hornbill,Lophoceros nasutus,Grysneushoringvoël,https://cdn.sabirds.app/african-grey-hornbill/...,Hornbills,Medium,Fynbos,"[Grey, Black, White]"
2,Black Tern,Chlidonias niger,Swartsterretjie,https://cdn.sabirds.app/black-tern/1_medium.webp,Terns,Medium,Wetland,"[Black, Grey, White]"
3,Black-naped Tern,Sterna sumatrana,Swartneksterretjie,https://cdn.sabirds.app/black-naped-tern/1_med...,Terns,Medium,Marine,"[Black, White, Grey]"
4,Blacksmith Lapwing,Vanellus armatus,Bontkiewiet,https://cdn.sabirds.app/blacksmith-lapwing/1_m...,Lapwings,Medium,Wetland,"[Black, White, Grey]"
5,Bridled Tern,Onychoprion anaethetus,Brilsterretjie,https://cdn.sabirds.app/bridled-tern/1_medium....,Terns,Medium,Marine,"[White, Grey, Black]"
6,Brown Noddy,Anous stolidus,Grootbruinsterretjie,https://cdn.sabirds.app/brown-noddy/1_medium.webp,Noddies,Medium,Marine,"[Brown, White, Grey, Black]"
7,Common Cuckoo,Cuculus canorus,Europese Koekoek,https://cdn.sabirds.app/common-cuckoo/1_medium...,Cuckoos,Medium,Forest,"[Grey, White, Black]"
8,Damara Tern,Sternula balaenarum,Damarasterretjie,None,Terns,Medium,Coastal,"[White, Grey, Black]"
9,Gull-billed Tern,Gelochelidon nilotica,Oostelike Sterretjie,https://cdn.sabirds.app/gull-billed-tern/1_med...,Terns,Medium,Coastal,"[White, Grey, Black]"


In [91]:
set(features)

{'Black', 'Cuckoos', 'Fynbos', 'Grey', 'Medium', 'White'}